In [3]:
import pandas as pd
train = pd.read_csv("./train.csv")
train

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",per:employee_of,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree
32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia


### ver1. 개선 
#### 문제
subject_entity와 object_entity를 바꿔주면 label도 바꿔줘야 한다.<br>
<br>
ex. 김경징이 처형되지 않고 귀양을 가게 되자 .... 1637년 2월 22일 인조는 김경징과 장신에게 사약을 내렸다.<br>
    subject : 김경징   object : 1637년 2월 22일    label: per:date_of_death<br>
  =>subject : 1637년 2월 22일   object : 김경징    label: date_of_death:per<br>
  
#### 해결책 
1. org:alternate_names, per:colleagues.. 처럼 entity가 바뀌어도 label이 바뀔 필요 없는 문장들에 대해서만 entity를 바꿔준다<br>
2. label이 no_relation이 아닌 경우 :을 기준으로 label을 바꿔준다 ✅

In [4]:
# label별로 문장을 보고 label을 바꿔줘야 하는 지 체크
idx = []
for i in range(32470):
    if train['label'][i] == "per:colleagues":#"org:alternate_names"... 이런식으로 바꿔가면서
        print(train.iloc[i])
        print(train['sentence'][i])
        break

id                                                               20
sentence          1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희 당선을 위해 무려 60...
subject_entity    {'word': '김종필', 'start_idx': 14, 'end_idx': 16...
object_entity     {'word': '박정희', 'start_idx': 30, 'end_idx': 32...
label                                                per:colleagues
source                                                    wikipedia
Name: 20, dtype: object
1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희 당선을 위해 무려 600억원이나 썼다고 밝혔다.


In [5]:
train_copy = train.copy()
train_changed = train.copy() # 증강할 데이터
train_changed['subject_entity'] = train_copy['object_entity']
train_changed['object_entity'] = train_copy['subject_entity']
train_changed 

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...","{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...","{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...","{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '박정부', 'start_idx': 22, 'end_idx': 24...","{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '1967', 'start_idx': 0, 'end_idx': 3,...","{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...",no_relation,wikipedia
...,...,...,...,...,...,...
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...","{'word': '유기준', 'start_idx': 93, 'end_idx': 95...",per:employee_of,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...",per:colleagues,wikipedia
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...","{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...",org:top_members/employees,wikitree
32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...",no_relation,wikipedia


In [6]:
idx = []
for i in range(32470):
    if train_changed['label'][i] != 'no_relation':
        idx.append(i)

print(len(idx))
#print(idx)

22936


22936개의 문장이 no_relation이 아님

In [7]:
idx = [2, 3, 5, 7, 8, 9, 10, 11, 12, 13, 15, 17, 18, 20, 21, 22, 23, 24, 25, 28, 29, 30, 31, 33, 34, 35, 36, 38, 39, 40, 42, 43, 45, 46, 47, 48, 49, 50, 52, 53, 56, 57, 58, 59, 60, 62, 65, 67, 68, 71, 72, 74, 75, 76, 77, 78, 79, 80, 82, 84, 85, 86, 88, 91, 92, 93, 94, 95, 97, 99, 100, 101, 102, 103, 104, 105, 111, 112, 113, 115, 116, 118, 119, 120, 121, 122, 123, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 142, 144, 146, 149, 150, 151, 153, 154, 155, 156, 157, 158, 161, 162, 163, 164, 167, 169, 174, 176, 178, 179, 180, 181, 183, 184, 185, 186, 187, 189, 190, 191, 193, 195, 197, 198, 199, 201, 202, 203, 204, 205, 206, 207, 208, 210, 211, 212, 213, 214, 215, 217, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 237, 238, 239, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 256, 257, 258, 259, 260, 262, 263, 265, 266, 268, 269, 271, 272, 273, 276, 278, 279, 280, 281, 282, 284, 286, 288, 289, 291, 292, 294, 295, 297, 298, 299, 300, 301, 302, 304, 306, 308, 309, 310, 311, 312, 313, 314, 315, 320, 321, 322, 323, 324, 326, 327, 329, 332, 333, 336, 341, 345, 346, 349, 350, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 364, 366, 367, 368, 369, 371, 373, 374, 375, 376, 378, 380, 381, 382, 383, 384, 385, 386, 387, 388, 390, 392, 393, 394, 396, 397, 400, 401, 402, 403, 404, 405, 406, 407, 408, 410, 411, 412, 415, 416, 417, 418, 419, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 433, 435, 436, 438, 439, 441, 442, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 465, 466, 467, 468, 469, 471, 472, 473, 474, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 488, 493, 494, 497, 498, 499, 501, 502, 503, 504, 507, 508, 509, 511, 512, 513, 514, 515, 516, 517, 518, 519, 521, 524, 525, 526, 528, 529, 530, 533, 537, 538, 540, 542, 543, 544, 545, 549, 551, 552, 553, 556, 557, 558, 559, 560, 561, 563, 564, 565, 567, 568, 569, 570, 571, 573, 574, 575, 576, 579, 580, 581, 582, 584, 586, 587, 588, 589, 590, 591, 592, 593, 595, 596, 598, 599, 600, 601, 602, 604, 605, 606, 607, 608, 610, 613, 614, 615, 616, 617, 619, 620, 621, 623, 624, 625, 626, 629, 631, 632, 633, 634, 636, 637, 640, 641, 643, 644, 645, 646, 647, 648, 649, 650, 651, 653, 655, 656, 657, 659, 660, 661, 662, 663, 664, 666, 667, 671, 673, 674, 675, 676, 677, 678, 679, 683, 684, 685, 686, 688, 690, 691, 692, 693, 695, 697, 698, 700, 702, 703, 704, 705, 706, 707, 708, 710, 711, 713, 716, 718, 720, 721, 723, 725, 729, 730, 731, 732, 733, 734, 736, 744, 747, 749, 751, 752, 754, 755, 756, 758, 760, 761, 762, 763, 765, 766, 767, 768, 770, 771, 773, 774, 775, 776, 777, 780, 784, 786, 788, 789, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 804, 805, 806, 807, 808, 809, 810, 811, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 831, 832, 833, 835, 836, 837, 838, 839, 842, 843, 844, 845, 847, 848, 849, 852, 855, 856, 857, 858, 860, 862, 864, 865, 866, 867, 868, 869, 872, 874, 875, 876, 877, 878, 879, 880, 882, 883, 884, 885, 887, 888, 889, 890, 891, 892, 894, 895, 896, 897, 898, 899, 901, 902, 903, 904, 905, 907, 908, 909, 910, 911, 912, 913, 914, 917, 918, 920, 921, 922, 925, 926, 929, 930, 931, 932, 934, 935, 936, 937, 938, 941, 942, 944, 945, 946, 947, 948, 949, 950, 952, 953, 954, 956, 957, 958, 959, 960, 961, 962, 963, 965, 967, 969, 971, 972, 973, 975, 976, 978, 979, 980, 981, 982, 983, 984, 985, 988, 989, 990, 991, 992, 993, 994, 995, 996, 998, 1001, 1002, 1004, 1005, 1006, 1007, 1008, 1010, 1013, 1014, 1016, 1017, 1019, 1021, 1023, 1024, 1025, 1026, 1030, 1032, 1033, 1035, 1036, 1037, 1040, 1041, 1042, 1044, 1045, 1047, 1048, 1050, 1051, 1054, 1055, 1056, 1059, 1060, 1061, 1063, 1064, 1066, 1068, 1069, 1070, 1072, 1074, 1075, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1092, 1093, 1094, 1096, 1097, 1099, 1101, 1102, 1103, 1104, 1105, 1107, 1108, 1109, 1110, 1113, 1114, 1115, 1117, 1118, 1119, 1121, 1122, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1133, 1136, 1137, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1148, 1149, 1150, 1152, 1153, 1154, 1156, 1157, 1158, 1160, 1161, 1162, 1163, 1164, 1165, 1166, 1168, 1169, 1171, 1172, 1174, 1176, 1178, 1180, 1182, 1184, 1185, 1186, 1187, 1188, 1190, 1191, 1192, 1195, 1197, 1198, 1199, 1201, 1202, 1203, 1204, 1207, 1208, 1209, 1210, 1212, 1213, 1214, 1215, 1216, 1219, 1221, 1222, 1226, 1228, 1229, 1230, 1232, 1233, 1234, 1235, 1236, 1240, 1241, 1243, 1244, 1245, 1246, 1247, 1249, 1250, 1251, 1253, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1271, 1273, 1274, 1276, 1277, 1279, 1280, 1281, 1282, 1283, 1284, 1286, 1289, 1290, 1292, 1293, 1294, 1297, 1298, 1300, 1301, 1302, 1303, 1304, 1305, 1307, 1310, 1312, 1313, 1314, 1316, 1317, 1318, 1319, 1320, 1324, 1325, 1326, 1328, 1329, 1330, 1331, 1332, 1334, 1335, 1336, 1339, 1340, 1342, 1343, 1344, 1345, 1347, 1348, 1349, 1351, 1353, 1354, 1355, 1357, 1360, 1362, 1366, 1367, 1369, 1370, 1372, 1373, 1374, 1375, 1376, 1377, 1378, 1379, 1381, 1383, 1385, 1386, 1388, 1389, 1391, 1392, 1393, 1394, 1395, 1396, 1397, 1398, 1400, 1401, 1402, 1403, 1406, 1407, 1408, 1409, 1410, 1411, 1412, 1413, 1414, 1416, 1418, 1419, 1420, 1421, 1422, 1423, 1424, 1427, 1428, 1429, 1431, 1432, 1433, 1435, 1438, 1439, 1440, 1441, 1446, 1447, 1450, 1452, 1454, 1455, 1456, 1457, 1458, 1459, 1460, 1461, 1465, 1467, 1469, 1470, 1471, 1472, 1473, 1477, 1479, 1480, 1481, 1482, 1485, 1490, 1491, 1492, 1493, 1494, 1495, 1496, 1497, 1501, 1502, 1503, 1504, 1505, 1506, 1508, 1509, 1510, 1512, 1513, 1514, 1516, 1518, 1521, 1524, 1525, 1526, 1527, 1528, 1533, 1534, 1535, 1536, 1537, 1539, 1540, 1541, 1542, 1543, 1544, 1545, 1546, 1547, 1549, 1550, 1551, 1552, 1553, 1555, 1556, 1557, 1558, 1559, 1560, 1563, 1564, 1565, 1566, 1569, 1571, 1573, 1574, 1575, 1576, 1577, 1578, 1579, 1580, 1581, 1582, 1583, 1584, 1586, 1587, 1588, 1590, 1591, 1592, 1593, 1595, 1596, 1597, 1598, 1599, 1601, 1602, 1603, 1604, 1605, 1606, 1607, 1609, 1610, 1612, 1615, 1616, 1617, 1621, 1622, 1623, 1624, 1626, 1627, 1628, 1629, 1631, 1632, 1633, 1634, 1636, 1637, 1638, 1639, 1641, 1642, 1646, 1648, 1649, 1653, 1654, 1655, 1656, 1657, 1658, 1660, 1661, 1662, 1663, 1664, 1666, 1667, 1668, 1669, 1671, 1672, 1674, 1675, 1676, 1677, 1678, 1680, 1683, 1686, 1687, 1688, 1690, 1691, 1692, 1693, 1696, 1697, 1698, 1699, 1700, 1702, 1705, 1706, 1708, 1710, 1712, 1713, 1715, 1717, 1718, 1722, 1723, 1724, 1725, 1726, 1728, 1729, 1730, 1732, 1733, 1734, 1737, 1738, 1739, 1741, 1742, 1743, 1744, 1745, 1747, 1748, 1749, 1752, 1754, 1757, 1759, 1760, 1763, 1765, 1768, 1769, 1770, 1772, 1773, 1774, 1775, 1776, 1777, 1778, 1780, 1781, 1782, 1784, 1785, 1786, 1787, 1789, 1790, 1791, 1792, 1793, 1795, 1796, 1797, 1798, 1800, 1803, 1804, 1805, 1807, 1808, 1810, 1811, 1812, 1813, 1815, 1818, 1819, 1822, 1823, 1824, 1825, 1826, 1828, 1830, 1832, 1833, 1835, 1839, 1840, 1842, 1844, 1845, 1846, 1847, 1848, 1849, 1850, 1851, 1852, 1853, 1854, 1855, 1856, 1857, 1858, 1861, 1863, 1864, 1865, 1866, 1867, 1868, 1870, 1871, 1874, 1876, 1877, 1879, 1880, 1883, 1885, 1886, 1887, 1888, 1889, 1890, 1891, 1892, 1893, 1894, 1895, 1896, 1897, 1898, 1899, 1900, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1911, 1912, 1913, 1914, 1915, 1917, 1918, 1919, 1920, 1921, 1923, 1925, 1926, 1927, 1928, 1929, 1931, 1932, 1934, 1935, 1936, 1937, 1938, 1939, 1942, 1943, 1944, 1946, 1950, 1951, 1954, 1955, 1956, 1957, 1958, 1960, 1962, 1963, 1964, 1967, 1969, 1970, 1972, 1974, 1977, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1988, 1990, 1991, 1992, 1993, 1996, 1998, 2000, 2002, 2004, 2006, 2007, 2011, 2013, 2014, 2016, 2017, 2019, 2020, 2022, 2024, 2026, 2027, 2028, 2029, 2030, 2032, 2033, 2035, 2036, 2037, 2038, 2040, 2041, 2042, 2043, 2045, 2048, 2050, 2051, 2054, 2055, 2057, 2059, 2064, 2065, 2067, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2079, 2080, 2081, 2082, 2085, 2086, 2087, 2088, 2089, 2090, 2092, 2093, 2096, 2097, 2099, 2100, 2103, 2104, 2105, 2106, 2108, 2109, 2110, 2111, 2112, 2114, 2115, 2117, 2118, 2119, 2120, 2121, 2122, 2123, 2125, 2126, 2127, 2128, 2129, 2130, 2131, 2133, 2134, 2135, 2138, 2139, 2140, 2141, 2142, 2145, 2146, 2147, 2148, 2149, 2150, 2153, 2154, 2155, 2156, 2157, 2160, 2161, 2162, 2166, 2168, 2169, 2170, 2171, 2172, 2173, 2174, 2175, 2177, 2180, 2182, 2183, 2184, 2185, 2187, 2188, 2189, 2190, 2191, 2192, 2193, 2194, 2197, 2198, 2199, 2201, 2202, 2204, 2205, 2208, 2210, 2211, 2212, 2213, 2214, 2217, 2221, 2222, 2223, 2226, 2227, 2228, 2230, 2231, 2232, 2233, 2234, 2235, 2236, 2237, 2239, 2240, 2241, 2242, 2243, 2244, 2246, 2247, 2249, 2250, 2252, 2253, 2254, 2255, 2256, 2257, 2259, 2260, 2261, 2262, 2263, 2265, 2266, 2267, 2268, 2270, 2272, 2273, 2274, 2277, 2278, 2279, 2280, 2281, 2282, 2285, 2286, 2287, 2289, 2290, 2291, 2293, 2294, 2295, 2296, 2297, 2298, 2299, 2301, 2303, 2304, 2305, 2306, 2307, 2308, 2311, 2312, 2313, 2314, 2315, 2317, 2318, 2319, 2320, 2321, 2325, 2326, 2336, 2338, 2340, 2341, 2343, 2344, 2346, 2347, 2349, 2351, 2352, 2355, 2356, 2357, 2358, 2359, 2363, 2365, 2366, 2367, 2369, 2370, 2371, 2372, 2374, 2375, 2376, 2377, 2380, 2381, 2382, 2384, 2387, 2388, 2391, 2393, 2394, 2395, 2396, 2397, 2398, 2399, 2401, 2402, 2404, 2405, 2406, 2407, 2408, 2410, 2411, 2413, 2414, 2415, 2416, 2417, 2419, 2420, 2422, 2424, 2425, 2426, 2428, 2429, 2431, 2433, 2434, 2437, 2438, 2440, 2441, 2443, 2444, 2445, 2448, 2449, 2450, 2451, 2454, 2455, 2456, 2459, 2461, 2462, 2464, 2465, 2467, 2468, 2470, 2471, 2472, 2473, 2474, 2476, 2478, 2479, 2480, 2481, 2482, 2484, 2488, 2489, 2490, 2491, 2492, 2494, 2495, 2498, 2499, 2501, 2502, 2505, 2506, 2509, 2510, 2511, 2512, 2513, 2514, 2516, 2518, 2519, 2520, 2521, 2524, 2525, 2527, 2528, 2529, 2530, 2532, 2535, 2536, 2537, 2539, 2541, 2542, 2543, 2544, 2545, 2547, 2549, 2550, 2552, 2555, 2557, 2558, 2560, 2563, 2564, 2566, 2567, 2569, 2570, 2571, 2572, 2573, 2574, 2576, 2577, 2580, 2581, 2582, 2583, 2584, 2585, 2586, 2587, 2588, 2589, 2590, 2591, 2592, 2594, 2595, 2597, 2598, 2599, 2600, 2602, 2603, 2604, 2605, 2606, 2607, 2608, 2611, 2612, 2614, 2615, 2616, 2618, 2619, 2621, 2622, 2625, 2626, 2627, 2629, 2630, 2631, 2633, 2635, 2636, 2639, 2640, 2644, 2645, 2646, 2647, 2650, 2651, 2652, 2653, 2655, 2656, 2658, 2659, 2660, 2661, 2663, 2665, 2667, 2668, 2669, 2670, 2671, 2672, 2673, 2674, 2675, 2676, 2678, 2679, 2680, 2682, 2683, 2685, 2686, 2688, 2690, 2691, 2692, 2693, 2696, 2697, 2699, 2702, 2703, 2704, 2706, 2707, 2709, 2710, 2711, 2712, 2713, 2714, 2716, 2719, 2720, 2721, 2724, 2725, 2726, 2727, 2728, 2732, 2733, 2734, 2735, 2736, 2737, 2738, 2739, 2740, 2741, 2744, 2745, 2746, 2747, 2748, 2749, 2750, 2752, 2753, 2754, 2755, 2756, 2758, 2759, 2760, 2764, 2766, 2770, 2771, 2773, 2774, 2776, 2777, 2779, 2780, 2782, 2784, 2785, 2788, 2789, 2792, 2793, 2794, 2798, 2801, 2802, 2803, 2804, 2805, 2807, 2808, 2810, 2811, 2812, 2813, 2815, 2816, 2817, 2818, 2819, 2820, 2821, 2822, 2823, 2824, 2825, 2826, 2828, 2829, 2830, 2832, 2833, 2834, 2836, 2837, 2838, 2839, 2840, 2841, 2842, 2844, 2845, 2846, 2848, 2849, 2853, 2854, 2856, 2857, 2858, 2859, 2860, 2861, 2862, 2864, 2865, 2866, 2870, 2871, 2872, 2873, 2874, 2875, 2878, 2879, 2880, 2881, 2882, 2884, 2885, 2886, 2887, 2890, 2891, 2892, 2893, 2894, 2895, 2896, 2898, 2899, 2904, 2905, 2909, 2910, 2911, 2912, 2913, 2914, 2917, 2918, 2919, 2920, 2921, 2922, 2924, 2927, 2928, 2929, 2930, 2931, 2932, 2937, 2938, 2939, 2940, 2941, 2942, 2944, 2945, 2946, 2947, 2949, 2950, 2952, 2954, 2957, 2958, 2959, 2960, 2961, 2962, 2966, 2967, 2968, 2969, 2970, 2972, 2974, 2975, 2976, 2977, 2978, 2979, 2980, 2982, 2983, 2984, 2985, 2986, 2987, 2988, 2992, 2993, 2995, 2996, 2997, 2998, 2999, 3001, 3004, 3005, 3009, 3010, 3011, 3012, 3014, 3016, 3017, 3018, 3019, 3020, 3021, 3022, 3023, 3025, 3026, 3028, 3029, 3030, 3031, 3032, 3033, 3037, 3041, 3042, 3043, 3044, 3046, 3048, 3049, 3050, 3051, 3052, 3053, 3054, 3056, 3057, 3058, 3059, 3060, 3062, 3063, 3064, 3065, 3066, 3067, 3068, 3069, 3070, 3074, 3076, 3079, 3080, 3081, 3084, 3085, 3086, 3087, 3088, 3089, 3091, 3092, 3096, 3097, 3098, 3099, 3100, 3101, 3103, 3107, 3108, 3109, 3110, 3111, 3113, 3114, 3116, 3117, 3118, 3119, 3120, 3121, 3122, 3126, 3127, 3128, 3130, 3131, 3132, 3133, 3135, 3137, 3138, 3139, 3140, 3143, 3144, 3145, 3146, 3147, 3148, 3149, 3150, 3153, 3154, 3155, 3158, 3160, 3162, 3166, 3167, 3169, 3170, 3171, 3172, 3173, 3174, 3175, 3176, 3177, 3178, 3179, 3180, 3181, 3182, 3183, 3184, 3185, 3186, 3187, 3188, 3189, 3190, 3191, 3194, 3195, 3196, 3197, 3198, 3199, 3201, 3203, 3204, 3205, 3208, 3210, 3211, 3212, 3213, 3216, 3219, 3220, 3222, 3223, 3224, 3225, 3226, 3227, 3228, 3229, 3230, 3231, 3235, 3236, 3237, 3238, 3239, 3241, 3243, 3244, 3245, 3246, 3247, 3248, 3249, 3250, 3251, 3254, 3256, 3257, 3258, 3259, 3260, 3262, 3263, 3264, 3265, 3266, 3268, 3271, 3272, 3275, 3276, 3279, 3280, 3281, 3282, 3283, 3284, 3285, 3287, 3289, 3290, 3291, 3292, 3293, 3294, 3296, 3297, 3298, 3299, 3300, 3301, 3302, 3303, 3305, 3307, 3308, 3309, 3310, 3314, 3315, 3317, 3318, 3319, 3321, 3323, 3326, 3328, 3329, 3330, 3333, 3334, 3336, 3338, 3340, 3341, 3345, 3347, 3349, 3350, 3351, 3352, 3353, 3354, 3355, 3356, 3357, 3358, 3360, 3361, 3362, 3363, 3364, 3365, 3366, 3367, 3368, 3370, 3371, 3372, 3374, 3376, 3377, 3378, 3379, 3381, 3382, 3383, 3384, 3385, 3386, 3387, 3388, 3389, 3390, 3391, 3392, 3394, 3396, 3397, 3399, 3401, 3402, 3404, 3405, 3406, 3408, 3409, 3410, 3411, 3412, 3414, 3415, 3416, 3417, 3419, 3422, 3423, 3425, 3426, 3428, 3429, 3430, 3431, 3432, 3433, 3434, 3435, 3436, 3437, 3438, 3439, 3441, 3442, 3443, 3444, 3445, 3447, 3449, 3450, 3452, 3453, 3454, 3455, 3456, 3458, 3459, 3460, 3461, 3462, 3463, 3465, 3467, 3468, 3470, 3472, 3473, 3474, 3475, 3476, 3477, 3478, 3479, 3483, 3485, 3486, 3488, 3489, 3490, 3491, 3492, 3493, 3494, 3495, 3497, 3498, 3499, 3500, 3502, 3504, 3505, 3510, 3511, 3512, 3513, 3515, 3517, 3518, 3519, 3520, 3521, 3522, 3523, 3524, 3525, 3526, 3527, 3529, 3530, 3531, 3532, 3533, 3534, 3535, 3536, 3538, 3539, 3540, 3541, 3542, 3543, 3544, 3545, 3546, 3548, 3549, 3550, 3551, 3554, 3555, 3556, 3557, 3559, 3560, 3561, 3562, 3563, 3564, 3565, 3566, 3569, 3570, 3572, 3575, 3577, 3578, 3580, 3581, 3582, 3583, 3584, 3585, 3587, 3589, 3591, 3592, 3593, 3594, 3595, 3596, 3598, 3599, 3600, 3602, 3603, 3604, 3605, 3606, 3607, 3608, 3609, 3610, 3611, 3612, 3613, 3614, 3615, 3616, 3617, 3618, 3619, 3620, 3622, 3623, 3624, 3626, 3628, 3629, 3630, 3631, 3636, 3637, 3639, 3640, 3641, 3642, 3643, 3644, 3645, 3647, 3648, 3649, 3650, 3652, 3654, 3655, 3656, 3657, 3659, 3661, 3662, 3663, 3664, 3665, 3666, 3667, 3668, 3669, 3670, 3671, 3674, 3676, 3680, 3681, 3682, 3683, 3687, 3689, 3691, 3693, 3695, 3696, 3697, 3699, 3701, 3702, 3703, 3705, 3707, 3708, 3709, 3711, 3713, 3714, 3715, 3716, 3717, 3718, 3719, 3720, 3721, 3723, 3725, 3729, 3730, 3731, 3733, 3734, 3735, 3736, 3737, 3738, 3739, 3740, 3743, 3744, 3745, 3746, 3747, 3748, 3750, 3752, 3753, 3754, 3757, 3758, 3759, 3760, 3761, 3762, 3767, 3768, 3769, 3770, 3771, 3772, 3773, 3776, 3777, 3779, 3780, 3781, 3783, 3784, 3786, 3787, 3788, 3789, 3790, 3791, 3792, 3796, 3797, 3798, 3799, 3800, 3801, 3804, 3806, 3807, 3808, 3810, 3812, 3813, 3814, 3815, 3817, 3818, 3820, 3821, 3822, 3826, 3827, 3828, 3829, 3830, 3831, 3832, 3833, 3834, 3835, 3838, 3839, 3842, 3844, 3845, 3847, 3848, 3849, 3850, 3851, 3852, 3854, 3855, 3858, 3859, 3860, 3861, 3862, 3863, 3864, 3865, 3869, 3871, 3872, 3873, 3875, 3876, 3877, 3878, 3880, 3882, 3883, 3884, 3885, 3887, 3889, 3890, 3892, 3893, 3894, 3895, 3896, 3898, 3900, 3901, 3907, 3908, 3909, 3910, 3913, 3914, 3915, 3916, 3917, 3918, 3919, 3921, 3923, 3924, 3925, 3926, 3927, 3928, 3929, 3930, 3932, 3933, 3937, 3939, 3940, 3942, 3944, 3945, 3946, 3948, 3949, 3950, 3952, 3953, 3954, 3958, 3959, 3960, 3963, 3964, 3965, 3966, 3968, 3970, 3971, 3972, 3973, 3976, 3978, 3980, 3981, 3982, 3983, 3984, 3985, 3986, 3987, 3988, 3989, 3992, 3993, 3995, 3996, 3997, 3999, 4002, 4003, 4005, 4010, 4011, 4012, 4013, 4014, 4015, 4016, 4017, 4018, 4019, 4021, 4023, 4024, 4025, 4033, 4034, 4036, 4038, 4039, 4040, 4041, 4043, 4046, 4047, 4048, 4050, 4052, 4053, 4054, 4055, 4056, 4057, 4058, 4059, 4060, 4061, 4063, 4065, 4066, 4067, 4069, 4071, 4072, 4073, 4074, 4077, 4078, 4079, 4082, 4083, 4084, 4086, 4087, 4088, 4090, 4091, 4093, 4095, 4096, 4097, 4100, 4101, 4102, 4104, 4105, 4107, 4109, 4110, 4114, 4115, 4116, 4117, 4118, 4119, 4121, 4122, 4123, 4125, 4126, 4127, 4129, 4136, 4139, 4140, 4141, 4142, 4143, 4144, 4146, 4147, 4148, 4149, 4150, 4151, 4152, 4154, 4155, 4157, 4160, 4161, 4162, 4163, 4164, 4165, 4166, 4167, 4168, 4170, 4171, 4172, 4174, 4175, 4176, 4178, 4180, 4181, 4182, 4183, 4184, 4185, 4186, 4187, 4189, 4191, 4192, 4193, 4194, 4196, 4197, 4198, 4199, 4201, 4202, 4204, 4206, 4207, 4208, 4209, 4211, 4212, 4213, 4214, 4215, 4216, 4217, 4221, 4222, 4223, 4224, 4225, 4226, 4227, 4228, 4231, 4232, 4234, 4235, 4236, 4237, 4238, 4239, 4242, 4243, 4244, 4245, 4246, 4247, 4248, 4249, 4250, 4253, 4254, 4255, 4257, 4258, 4259, 4260, 4261, 4262, 4263, 4264, 4265, 4266, 4268, 4269, 4270, 4271, 4274, 4275, 4277, 4278, 4279, 4280, 4281, 4282, 4283, 4284, 4285, 4286, 4288, 4289, 4290, 4291, 4293, 4294, 4295, 4296, 4299, 4300, 4301, 4303, 4304, 4305, 4306, 4307, 4311, 4313, 4314, 4315, 4316, 4317, 4319, 4320, 4321, 4324, 4325, 4326, 4327, 4328, 4329, 4330, 4331, 4332, 4333, 4334, 4335, 4336, 4338, 4339, 4340, 4341, 4344, 4345, 4346, 4347, 4348, 4349, 4350, 4351, 4352, 4353, 4354, 4355, 4357, 4358, 4359, 4362, 4364, 4365, 4366, 4367, 4370, 4371, 4372, 4375, 4380, 4381, 4382, 4383, 4384, 4385, 4386, 4388, 4390, 4391, 4392, 4393, 4398, 4399, 4400, 4401, 4402, 4404, 4405, 4406, 4407, 4409, 4410, 4411, 4413, 4414, 4415, 4416, 4417, 4418, 4419, 4420, 4421, 4422, 4423, 4424, 4426, 4428, 4429, 4430, 4432, 4435, 4437, 4438, 4439, 4440, 4442, 4444, 4448, 4449, 4450, 4451, 4453, 4454, 4455, 4456, 4457, 4458, 4460, 4462, 4465, 4470, 4473, 4475, 4476, 4477, 4478, 4480, 4481, 4482, 4483, 4484, 4485, 4486, 4487, 4488, 4489, 4492, 4493, 4494, 4495, 4496, 4498, 4499, 4500, 4501, 4502, 4503, 4504, 4505, 4506, 4507, 4510, 4511, 4512, 4513, 4514, 4515, 4516, 4517, 4518, 4519, 4520, 4521, 4523, 4524, 4525, 4526, 4529, 4534, 4535, 4536, 4537, 4538, 4541, 4542, 4544, 4547, 4548, 4549, 4550, 4555, 4556, 4557, 4559, 4562, 4563, 4564, 4565, 4568, 4569, 4570, 4571, 4572, 4573, 4574, 4575, 4576, 4578, 4579, 4581, 4582, 4583, 4584, 4585, 4586, 4587, 4588, 4589, 4590, 4591, 4592, 4593, 4594, 4596, 4597, 4598, 4604, 4605, 4606, 4607, 4609, 4610, 4611, 4613, 4614, 4616, 4618, 4619, 4620, 4621, 4622, 4623, 4626, 4627, 4628, 4631, 4632, 4633, 4634, 4635, 4637, 4638, 4640, 4641, 4642, 4643, 4645, 4646, 4647, 4650, 4652, 4654, 4656, 4658, 4662, 4663, 4664, 4665, 4666, 4667, 4668, 4669, 4670, 4671, 4674, 4675, 4677, 4680, 4681, 4682, 4683, 4684, 4685, 4688, 4689, 4690, 4691, 4692, 4693, 4694, 4695, 4696, 4697, 4699, 4700, 4701, 4704, 4705, 4706, 4708, 4709, 4711, 4713, 4714, 4715, 4716, 4718, 4719, 4720, 4721, 4724, 4725, 4726, 4727, 4728, 4729, 4730, 4731, 4734, 4737, 4738, 4739, 4740, 4741, 4742, 4744, 4746, 4747, 4748, 4749, 4751, 4752, 4754, 4755, 4756, 4757, 4758, 4759, 4760, 4762, 4763, 4764, 4765, 4766, 4767, 4768, 4769, 4770, 4771, 4773, 4774, 4775, 4776, 4777, 4778, 4779, 4782, 4783, 4784, 4785, 4786, 4787, 4788, 4789, 4790, 4791, 4792, 4793, 4794, 4796, 4797, 4798, 4800, 4801, 4803, 4805, 4806, 4807, 4808, 4809, 4810, 4811, 4813, 4814, 4816, 4819, 4820, 4823, 4825, 4826, 4827, 4829, 4831, 4832, 4834, 4836, 4840, 4841, 4842, 4843, 4844, 4845, 4846, 4848, 4851, 4852, 4853, 4854, 4855, 4856, 4858, 4859, 4861, 4862, 4863, 4865, 4867, 4868, 4869, 4870, 4874, 4875, 4876, 4877, 4878, 4880, 4881, 4882, 4883, 4885, 4886, 4887, 4888, 4889, 4890, 4892, 4895, 4896, 4898, 4899, 4903, 4904, 4906, 4908, 4909, 4910, 4911, 4913, 4916, 4917, 4918, 4919, 4920, 4921, 4922, 4923, 4924, 4925, 4926, 4927, 4928, 4929, 4931, 4932, 4933, 4934, 4935, 4936, 4937, 4938, 4940, 4941, 4942, 4943, 4944, 4945, 4946, 4947, 4948, 4949, 4950, 4951, 4952, 4953, 4954, 4956, 4957, 4958, 4959, 4960, 4963, 4964, 4965, 4967, 4968, 4969, 4970, 4971, 4972, 4973, 4976, 4977, 4979, 4980, 4981, 4982, 4983, 4984, 4986, 4988, 4989, 4991, 4992, 4994, 4995, 4996, 4997, 4998, 4999, 5000, 5002, 5003, 5004, 5006, 5007, 5009, 5010, 5011, 5012, 5014, 5017, 5020, 5021, 5022, 5023, 5024, 5025, 5026, 5028, 5029, 5031, 5033, 5035, 5038, 5039, 5041, 5042, 5043, 5045, 5047, 5048, 5051, 5052, 5053, 5054, 5055, 5056, 5058, 5059, 5060, 5061, 5062, 5064, 5065, 5066, 5067, 5068, 5069, 5070, 5071, 5074, 5075, 5076, 5077, 5078, 5080, 5081, 5082, 5084, 5085, 5087, 5088, 5090, 5091, 5092, 5094, 5095, 5096, 5098, 5099, 5100, 5102, 5104, 5106, 5107, 5111, 5112, 5113, 5114, 5115, 5117, 5118, 5119, 5120, 5122, 5124, 5126, 5128, 5129, 5130, 5134, 5138, 5139, 5140, 5141, 5142, 5144, 5146, 5148, 5149, 5150, 5155, 5156, 5157, 5158, 5159, 5160, 5161, 5164, 5165, 5166, 5172, 5173, 5175, 5176, 5181, 5182, 5185, 5186, 5187, 5188, 5189, 5192, 5194, 5195, 5197, 5198, 5199, 5200, 5202, 5203, 5204, 5205, 5206, 5210, 5212, 5213, 5215, 5217, 5218, 5221, 5222, 5223, 5225, 5226, 5228, 5229, 5230, 5231, 5232, 5233, 5234, 5235, 5236, 5237, 5239, 5240, 5243, 5245, 5246, 5248, 5249, 5250, 5251, 5252, 5253, 5254, 5255, 5256, 5257, 5258, 5259, 5260, 5261, 5262, 5263, 5264, 5265, 5266, 5267, 5269, 5271, 5274, 5276, 5278, 5279, 5280, 5284, 5287, 5288, 5289, 5290, 5292, 5293, 5294, 5296, 5297, 5298, 5300, 5301, 5302, 5305, 5306, 5307, 5308, 5310, 5312, 5313, 5315, 5316, 5318, 5319, 5320, 5321, 5324, 5325, 5326, 5327, 5329, 5331, 5332, 5333, 5335, 5336, 5337, 5338, 5339, 5341, 5342, 5343, 5345, 5346, 5348, 5349, 5350, 5351, 5354, 5355, 5356, 5357, 5359, 5361, 5362, 5363, 5364, 5367, 5369, 5370, 5371, 5372, 5373, 5374, 5375, 5377, 5378, 5380, 5381, 5382, 5383, 5384, 5386, 5389, 5391, 5392, 5394, 5395, 5397, 5398, 5399, 5400, 5401, 5402, 5403, 5404, 5405, 5408, 5409, 5411, 5412, 5413, 5414, 5415, 5416, 5417, 5419, 5420, 5421, 5422, 5425, 5426, 5427, 5428, 5430, 5431, 5432, 5434, 5436, 5437, 5438, 5439, 5440, 5442, 5443, 5446, 5447, 5448, 5449, 5450, 5452, 5455, 5456, 5458, 5459, 5460, 5461, 5462, 5463, 5465, 5467, 5470, 5472, 5473, 5475, 5476, 5477, 5478, 5479, 5481, 5484, 5485, 5487, 5489, 5490, 5491, 5492, 5494, 5496, 5497, 5498, 5499, 5500, 5503, 5504, 5505, 5507, 5508, 5509, 5510, 5511, 5512, 5514, 5515, 5516, 5517, 5518, 5519, 5520, 5521, 5522, 5523, 5524, 5525, 5527, 5528, 5529, 5531, 5532, 5534, 5535, 5536, 5539, 5540, 5543, 5545, 5546, 5547, 5551, 5552, 5553, 5556, 5557, 5561, 5563, 5565, 5566, 5567, 5568, 5570, 5572, 5574, 5578, 5579, 5580, 5581, 5582, 5583, 5584, 5585, 5586, 5587, 5588, 5591, 5593, 5595, 5596, 5598, 5599, 5600, 5601, 5604, 5605, 5606, 5607, 5608, 5610, 5611, 5612, 5614, 5615, 5616, 5617, 5619, 5620, 5621, 5622, 5623, 5625, 5628, 5629, 5630, 5632, 5633, 5634, 5635, 5636, 5637, 5638, 5639, 5640, 5641, 5642, 5644, 5645, 5646, 5647, 5651, 5652, 5653, 5654, 5655, 5656, 5657, 5658, 5660, 5661, 5662, 5664, 5665, 5667, 5668, 5669, 5671, 5672, 5673, 5674, 5676, 5677, 5678, 5679, 5681, 5682, 5685, 5686, 5687, 5689, 5690, 5692, 5693, 5694, 5696, 5699, 5700, 5702, 5703, 5704, 5707, 5708, 5710, 5711, 5712, 5713, 5714, 5715, 5716, 5717, 5719, 5721, 5722, 5723, 5724, 5725, 5726, 5727, 5730, 5732, 5733, 5735, 5736, 5737, 5739, 5740, 5741, 5743, 5744, 5748, 5749, 5750, 5752, 5753, 5756, 5757, 5758, 5759, 5760, 5761, 5763, 5765, 5766, 5767, 5768, 5769, 5771, 5772, 5773, 5776, 5777, 5780, 5781, 5782, 5784, 5787, 5789, 5790, 5791, 5793, 5794, 5796, 5798, 5799, 5801, 5802, 5803, 5806, 5807, 5808, 5809, 5811, 5812, 5813, 5815, 5816, 5817, 5820, 5822, 5823, 5824, 5829, 5830, 5831, 5832, 5835, 5836, 5837, 5840, 5842, 5843, 5844, 5845, 5846, 5848, 5850, 5852, 5853, 5855, 5858, 5861, 5862, 5866, 5867, 5868, 5869, 5870, 5872, 5875, 5876, 5877, 5878, 5879, 5880, 5881, 5882, 5883, 5885, 5886, 5888, 5889, 5890, 5891, 5892, 5893, 5896, 5897, 5899, 5900, 5901, 5903, 5904, 5905, 5907, 5908, 5909, 5910, 5911, 5913, 5914, 5915, 5917, 5918, 5919, 5920, 5922, 5923, 5924, 5926, 5927, 5929, 5930, 5931, 5932, 5933, 5934, 5938, 5939, 5940, 5942, 5943, 5945, 5947, 5948, 5949, 5950, 5953, 5954, 5955, 5957, 5958, 5960, 5961, 5962, 5965, 5966, 5967, 5968, 5969, 5970, 5971, 5972, 5973, 5974, 5975, 5977, 5978, 5979, 5980, 5981, 5982, 5983, 5984, 5986, 5989, 5991, 5992, 5993, 5994, 5996, 5998, 6000, 6001, 6002, 6003, 6004, 6005, 6006, 6008, 6010, 6011, 6012, 6013, 6014, 6015, 6016, 6017, 6018, 6019, 6022, 6023, 6024, 6026, 6027, 6030, 6032, 6033, 6034, 6035, 6036, 6037, 6042, 6045, 6047, 6048, 6049, 6050, 6051, 6052, 6053, 6054, 6056, 6057, 6058, 6059, 6060, 6061, 6062, 6063, 6066, 6068, 6069, 6070, 6071, 6073, 6075, 6077, 6078, 6082, 6083, 6084, 6085, 6086, 6087, 6088, 6089, 6090, 6091, 6092, 6093, 6097, 6100, 6101, 6102, 6103, 6104, 6105, 6107, 6108, 6109, 6110, 6111, 6112, 6114, 6116, 6117, 6120, 6121, 6124, 6125, 6126, 6128, 6129, 6132, 6134, 6135, 6136, 6137, 6138, 6139, 6140, 6141, 6149, 6150, 6152, 6153, 6154, 6155, 6156, 6157, 6158, 6159, 6160, 6164, 6168, 6169, 6170, 6171, 6172, 6173, 6174, 6175, 6178, 6179, 6180, 6181, 6182, 6183, 6185, 6186, 6188, 6189, 6193, 6194, 6195, 6196, 6197, 6198, 6200, 6201, 6202, 6204, 6205, 6206, 6208, 6209, 6210, 6211, 6212, 6214, 6215, 6216, 6218, 6221, 6223, 6224, 6225, 6228, 6229, 6230, 6231, 6232, 6233, 6238, 6239, 6242, 6243, 6245, 6247, 6248, 6249, 6250, 6251, 6252, 6253, 6256, 6258, 6259, 6260, 6261, 6264, 6265, 6266, 6268, 6269, 6270, 6272, 6273, 6274, 6276, 6277, 6278, 6280, 6281, 6284, 6285, 6286, 6288, 6289, 6290, 6291, 6293, 6295, 6296, 6298, 6299, 6301, 6302, 6304, 6305, 6308, 6309, 6310, 6311, 6312, 6313, 6314, 6317, 6318, 6320, 6321, 6322, 6323, 6324, 6326, 6328, 6329, 6330, 6331, 6332, 6333, 6334, 6340, 6341, 6342, 6343, 6344, 6345, 6346, 6347, 6349, 6351, 6353, 6354, 6355, 6356, 6357, 6358, 6360, 6361, 6362, 6363, 6365, 6366, 6368, 6371, 6372, 6373, 6374, 6376, 6377, 6378, 6379, 6380, 6381, 6382, 6384, 6385, 6388, 6389, 6390, 6391, 6392, 6393, 6394, 6395, 6396, 6397, 6399, 6401, 6402, 6403, 6404, 6405, 6406, 6407, 6408, 6409, 6410, 6411, 6414, 6415, 6419, 6420, 6421, 6423, 6424, 6426, 6427, 6428, 6429, 6430, 6431, 6432, 6433, 6434, 6435, 6436, 6438, 6439, 6440, 6442, 6445, 6446, 6447, 6448, 6449, 6450, 6451, 6452, 6453, 6454, 6455, 6456, 6461, 6462, 6463, 6464, 6465, 6466, 6468, 6470, 6471, 6472, 6474, 6475, 6477, 6478, 6479, 6480, 6482, 6484, 6485, 6487, 6488, 6490, 6491, 6492, 6493, 6494, 6495, 6496, 6498, 6499, 6501, 6502, 6504, 6505, 6506, 6508, 6509, 6511, 6513, 6514, 6515, 6516, 6517, 6520, 6522, 6523, 6524, 6526, 6527, 6528, 6530, 6531, 6532, 6534, 6535, 6536, 6537, 6538, 6540, 6541, 6542, 6544, 6545, 6547, 6548, 6549, 6552, 6555, 6557, 6558, 6559, 6562, 6563, 6564, 6565, 6566, 6568, 6569, 6570, 6571, 6572, 6573, 6574, 6575, 6576, 6577, 6578, 6579, 6580, 6583, 6584, 6585, 6586, 6587, 6588, 6591, 6593, 6594, 6595, 6596, 6597, 6598, 6599, 6600, 6602, 6603, 6604, 6605, 6606, 6607, 6608, 6609, 6610, 6611, 6612, 6613, 6614, 6615, 6616, 6617, 6618, 6620, 6622, 6625, 6627, 6628, 6629, 6630, 6631, 6632, 6633, 6634, 6635, 6636, 6637, 6639, 6640, 6642, 6643, 6644, 6646, 6647, 6649, 6650, 6652, 6654, 6655, 6657, 6658, 6660, 6662, 6664, 6666, 6667, 6668, 6669, 6670, 6671, 6672, 6675, 6677, 6680, 6682, 6683, 6685, 6686, 6688, 6689, 6691, 6692, 6693, 6694, 6695, 6700, 6701, 6702, 6703, 6705, 6706, 6709, 6711, 6712, 6713, 6714, 6715, 6716, 6717, 6718, 6720, 6721, 6722, 6723, 6725, 6726, 6727, 6729, 6732, 6733, 6734, 6735, 6736, 6738, 6742, 6743, 6744, 6746, 6748, 6750, 6751, 6752, 6753, 6755, 6756, 6757, 6758, 6759, 6762, 6763, 6764, 6767, 6768, 6769, 6770, 6774, 6776, 6777, 6778, 6779, 6780, 6782, 6783, 6785, 6787, 6788, 6789, 6790, 6791, 6792, 6793, 6794, 6795, 6796, 6797, 6805, 6806, 6807, 6809, 6811, 6812, 6813, 6814, 6816, 6818, 6819, 6820, 6821, 6828, 6829, 6830, 6831, 6832, 6834, 6835, 6836, 6837, 6839, 6840, 6842, 6843, 6844, 6845, 6846, 6847, 6848, 6849, 6850, 6851, 6852, 6853, 6855, 6856, 6857, 6859, 6860, 6861, 6862, 6864, 6865, 6867, 6868, 6869, 6870, 6872, 6874, 6878, 6879, 6880, 6882, 6883, 6884, 6885, 6886, 6888, 6889, 6891, 6892, 6893, 6896, 6897, 6898, 6900, 6901, 6903, 6906, 6907, 6911, 6912, 6913, 6914, 6915, 6916, 6917, 6920, 6921, 6922, 6924, 6925, 6926, 6927, 6928, 6929, 6930, 6932, 6933, 6934, 6935, 6936, 6937, 6942, 6945, 6947, 6949, 6950, 6951, 6952, 6953, 6954, 6955, 6956, 6957, 6958, 6959, 6960, 6961, 6963, 6965, 6967, 6968, 6970, 6971, 6972, 6973, 6974, 6975, 6976, 6977, 6979, 6981, 6984, 6987, 6988, 6990, 6991, 6993, 6994, 6995, 6997, 7001, 7002, 7003, 7004, 7006, 7007, 7008, 7010, 7013, 7014, 7016, 7018, 7019, 7020, 7021, 7023, 7024, 7025, 7026, 7027, 7028, 7029, 7030, 7035, 7036, 7037, 7038, 7040, 7041, 7043, 7044, 7046, 7047, 7048, 7049, 7050, 7051, 7052, 7053, 7057, 7058, 7059, 7060, 7062, 7063, 7064, 7066, 7067, 7068, 7070, 7071, 7072, 7074, 7076, 7078, 7079, 7083, 7084, 7085, 7086, 7087, 7090, 7091, 7092, 7093, 7094, 7095, 7096, 7098, 7100, 7102, 7105, 7107, 7108, 7110, 7111, 7112, 7113, 7115, 7116, 7117, 7118, 7119, 7121, 7122, 7123, 7124, 7125, 7126, 7127, 7130, 7132, 7133, 7135, 7136, 7137, 7138, 7142, 7143, 7145, 7146, 7151, 7154, 7155, 7156, 7157, 7158, 7161, 7162, 7164, 7166, 7167, 7168, 7169, 7172, 7173, 7174, 7175, 7178, 7180, 7183, 7184, 7186, 7187, 7188, 7189, 7190, 7193, 7197, 7198, 7199, 7201, 7204, 7205, 7206, 7207, 7208, 7209, 7210, 7211, 7212, 7214, 7216, 7217, 7218, 7219, 7220, 7221, 7222, 7224, 7225, 7227, 7228, 7229, 7230, 7231, 7232, 7233, 7235, 7240, 7241, 7242, 7243, 7244, 7245, 7246, 7247, 7254, 7256, 7257, 7258, 7259, 7263, 7264, 7265, 7266, 7267, 7268, 7269, 7270, 7271, 7272, 7273, 7275, 7277, 7279, 7280, 7281, 7282, 7283, 7285, 7286, 7287, 7288, 7289, 7290, 7293, 7294, 7295, 7296, 7298, 7301, 7303, 7305, 7307, 7308, 7309, 7310, 7311, 7312, 7313, 7315, 7316, 7317, 7318, 7320, 7322, 7323, 7325, 7326, 7327, 7330, 7331, 7332, 7334, 7336, 7337, 7338, 7339, 7340, 7342, 7343, 7345, 7347, 7348, 7349, 7350, 7351, 7352, 7353, 7356, 7357, 7358, 7360, 7362, 7363, 7365, 7366, 7367, 7370, 7371, 7372, 7373, 7374, 7375, 7378, 7379, 7380, 7381, 7382, 7383, 7385, 7386, 7387, 7389, 7390, 7391, 7393, 7396, 7398, 7400, 7401, 7402, 7405, 7409, 7410, 7411, 7412, 7413, 7415, 7417, 7418, 7419, 7421, 7422, 7423, 7424, 7425, 7426, 7427, 7430, 7432, 7434, 7435, 7436, 7437, 7438, 7439, 7440, 7441, 7443, 7445, 7446, 7447, 7450, 7451, 7452, 7455, 7460, 7463, 7465, 7468, 7469, 7470, 7472, 7473, 7474, 7475, 7476, 7481, 7483, 7487, 7488, 7489, 7490, 7492, 7493, 7494, 7496, 7497, 7498, 7499, 7500, 7501, 7502, 7503, 7506, 7507, 7508, 7510, 7514, 7515, 7516, 7518, 7519, 7520, 7521, 7526, 7528, 7529, 7530, 7532, 7533, 7534, 7535, 7536, 7538, 7539, 7540, 7542, 7545, 7546, 7547, 7548, 7549, 7550, 7551, 7552, 7553, 7554, 7558, 7560, 7561, 7562, 7563, 7565, 7566, 7567, 7568, 7569, 7570, 7572, 7574, 7575, 7578, 7579, 7582, 7583, 7584, 7587, 7588, 7589, 7590, 7592, 7593, 7594, 7595, 7596, 7597, 7598, 7600, 7601, 7604, 7606, 7607, 7608, 7611, 7612, 7613, 7614, 7616, 7618, 7620, 7621, 7622, 7626, 7628, 7631, 7632, 7634, 7635, 7636, 7637, 7638, 7640, 7641, 7642, 7644, 7646, 7648, 7649, 7650, 7651, 7652, 7654, 7655, 7656, 7658, 7659, 7661, 7663, 7665, 7666, 7667, 7668, 7669, 7670, 7671, 7672, 7673, 7674, 7675, 7676, 7677, 7678, 7680, 7681, 7682, 7684, 7686, 7688, 7689, 7690, 7691, 7692, 7695, 7696, 7697, 7698, 7699, 7701, 7702, 7703, 7704, 7705, 7706, 7707, 7708, 7712, 7713, 7714, 7715, 7716, 7717, 7718, 7719, 7720, 7721, 7722, 7723, 7724, 7725, 7726, 7727, 7728, 7729, 7730, 7731, 7732, 7734, 7737, 7738, 7741, 7743, 7744, 7745, 7748, 7749, 7750, 7751, 7753, 7754, 7756, 7759, 7760, 7761, 7762, 7764, 7765, 7766, 7767, 7770, 7771, 7774, 7775, 7776, 7778, 7781, 7784, 7785, 7787, 7790, 7791, 7794, 7795, 7797, 7798, 7799, 7800, 7802, 7803, 7804, 7806, 7807, 7808, 7809, 7810, 7812, 7813, 7815, 7816, 7817, 7818, 7819, 7820, 7823, 7825, 7826, 7827, 7828, 7829, 7830, 7832, 7833, 7834, 7835, 7837, 7838, 7839, 7842, 7843, 7844, 7846, 7847, 7849, 7850, 7851, 7852, 7853, 7854, 7855, 7857, 7858, 7860, 7861, 7862, 7863, 7864, 7865, 7867, 7868, 7869, 7870, 7871, 7872, 7873, 7875, 7876, 7878, 7879, 7880, 7881, 7882, 7883, 7885, 7886, 7888, 7889, 7890, 7891, 7892, 7896, 7897, 7898, 7899, 7900, 7903, 7904, 7906, 7907, 7908, 7911, 7912, 7913, 7914, 7915, 7916, 7917, 7918, 7919, 7920, 7921, 7924, 7925, 7926, 7927, 7928, 7931, 7932, 7933, 7935, 7936, 7937, 7938, 7939, 7941, 7942, 7943, 7944, 7945, 7946, 7947, 7948, 7949, 7950, 7951, 7952, 7954, 7955, 7957, 7959, 7960, 7961, 7962, 7968, 7971, 7972, 7973, 7974, 7978, 7979, 7982, 7984, 7985, 7986, 7987, 7988, 7989, 7990, 7991, 7992, 7993, 7994, 7995, 7997, 7998, 7999, 8000, 8003, 8006, 8008, 8009, 8010, 8011, 8012, 8013, 8016, 8018, 8020, 8021, 8022, 8023, 8025, 8026, 8027, 8028, 8030, 8033, 8034, 8035, 8037, 8038, 8039, 8040, 8041, 8044, 8045, 8047, 8048, 8050, 8051, 8052, 8053, 8054, 8055, 8057, 8058, 8059, 8061, 8062, 8063, 8065, 8066, 8067, 8069, 8070, 8071, 8072, 8074, 8075, 8076, 8078, 8079, 8080, 8081, 8083, 8084, 8085, 8088, 8089, 8090, 8092, 8094, 8095, 8096, 8097, 8098, 8099, 8100, 8102, 8103, 8104, 8105, 8106, 8107, 8111, 8112, 8113, 8114, 8115, 8116, 8117, 8118, 8119, 8120, 8123, 8124, 8126, 8128, 8129, 8130, 8133, 8134, 8138, 8140, 8144, 8147, 8148, 8150, 8152, 8153, 8154, 8156, 8157, 8158, 8159, 8162, 8164, 8167, 8169, 8170, 8171, 8172, 8175, 8176, 8177, 8179, 8180, 8181, 8183, 8185, 8186, 8187, 8188, 8189, 8190, 8191, 8192, 8194, 8196, 8197, 8200, 8201, 8202, 8203, 8204, 8205, 8206, 8207, 8208, 8209, 8212, 8213, 8214, 8216, 8217, 8218, 8219, 8220, 8221, 8222, 8223, 8225, 8228, 8229, 8230, 8231, 8233, 8235, 8236, 8238, 8239, 8240, 8241, 8242, 8243, 8244, 8245, 8246, 8248, 8249, 8251, 8253, 8254, 8255, 8256, 8257, 8258, 8260, 8262, 8263, 8264, 8265, 8266, 8268, 8269, 8270, 8271, 8272, 8273, 8276, 8277, 8279, 8280, 8282, 8283, 8285, 8286, 8288, 8289, 8290, 8291, 8294, 8296, 8297, 8298, 8300, 8301, 8303, 8305, 8307, 8309, 8310, 8312, 8316, 8318, 8320, 8321, 8322, 8323, 8324, 8327, 8328, 8329, 8330, 8333, 8334, 8339, 8340, 8341, 8344, 8345, 8347, 8348, 8349, 8351, 8353, 8354, 8356, 8357, 8358, 8360, 8361, 8362, 8363, 8365, 8366, 8370, 8371, 8372, 8373, 8374, 8375, 8380, 8381, 8384, 8387, 8389, 8390, 8391, 8392, 8394, 8395, 8396, 8397, 8398, 8399, 8401, 8404, 8405, 8406, 8407, 8409, 8410, 8411, 8413, 8414, 8416, 8417, 8418, 8419, 8421, 8422, 8424, 8425, 8426, 8427, 8429, 8430, 8431, 8432, 8433, 8435, 8436, 8438, 8439, 8440, 8441, 8442, 8443, 8444, 8445, 8446, 8447, 8448, 8450, 8452, 8455, 8456, 8457, 8459, 8460, 8461, 8463, 8465, 8466, 8467, 8468, 8469, 8470, 8471, 8472, 8473, 8475, 8476, 8477, 8478, 8479, 8480, 8482, 8483, 8485, 8486, 8487, 8489, 8490, 8493, 8495, 8496, 8497, 8498, 8499, 8501, 8503, 8504, 8506, 8507, 8508, 8511, 8512, 8513, 8515, 8516, 8517, 8518, 8519, 8520, 8521, 8524, 8527, 8528, 8531, 8532, 8534, 8535, 8536, 8537, 8538, 8540, 8541, 8542, 8543, 8544, 8545, 8547, 8548, 8549, 8550, 8551, 8552, 8553, 8554, 8555, 8556, 8557, 8559, 8560, 8561, 8562, 8563, 8564, 8566, 8568, 8569, 8570, 8571, 8573, 8574, 8576, 8577, 8580, 8581, 8583, 8584, 8586, 8587, 8588, 8589, 8592, 8593, 8595, 8596, 8597, 8603, 8604, 8605, 8607, 8608, 8610, 8611, 8613, 8615, 8616, 8617, 8618, 8619, 8621, 8622, 8623, 8625, 8626, 8627, 8628, 8629, 8631, 8632, 8633, 8634, 8636, 8640, 8642, 8643, 8645, 8647, 8648, 8649, 8650, 8652, 8653, 8656, 8657, 8658, 8659, 8660, 8661, 8662, 8664, 8665, 8667, 8669, 8670, 8671, 8674, 8676, 8679, 8680, 8681, 8682, 8683, 8684, 8685, 8687, 8692, 8696, 8697, 8698, 8699, 8703, 8704, 8705, 8706, 8707, 8708, 8709, 8710, 8713, 8714, 8716, 8717, 8718, 8719, 8720, 8722, 8723, 8724, 8725, 8726, 8727, 8728, 8729, 8730, 8732, 8733, 8734, 8735, 8736, 8739, 8741, 8742, 8744, 8745, 8746, 8747, 8749, 8750, 8751, 8753, 8754, 8756, 8758, 8759, 8760, 8761, 8763, 8764, 8765, 8766, 8767, 8768, 8769, 8770, 8773, 8774, 8776, 8777, 8778, 8779, 8780, 8781, 8782, 8783, 8784, 8785, 8786, 8787, 8788, 8789, 8791, 8792, 8796, 8797, 8798, 8799, 8800, 8801, 8803, 8804, 8805, 8807, 8808, 8811, 8812, 8813, 8814, 8815, 8819, 8821, 8822, 8825, 8826, 8827, 8828, 8830, 8831, 8834, 8835, 8836, 8838, 8839, 8840, 8842, 8843, 8845, 8847, 8850, 8852, 8853, 8855, 8856, 8857, 8858, 8859, 8860, 8862, 8863, 8864, 8866, 8867, 8869, 8870, 8871, 8874, 8875, 8878, 8879, 8880, 8881, 8882, 8883, 8888, 8889, 8891, 8894, 8895, 8897, 8898, 8900, 8901, 8902, 8903, 8904, 8905, 8906, 8909, 8911, 8912, 8913, 8914, 8916, 8918, 8919, 8920, 8921, 8923, 8924, 8925, 8927, 8929, 8930, 8931, 8938, 8939, 8941, 8942, 8944, 8945, 8946, 8947, 8951, 8953, 8954, 8956, 8960, 8961, 8962, 8965, 8966, 8967, 8968, 8970, 8971, 8973, 8977, 8978, 8979, 8981, 8982, 8984, 8986, 8987, 8988, 8989, 8990, 8992, 8993, 8996, 8997, 8999, 9000, 9001, 9002, 9004, 9005, 9006, 9007, 9009, 9011, 9012, 9013, 9014, 9016, 9018, 9019, 9020, 9021, 9022, 9023, 9026, 9027, 9028, 9029, 9031, 9033, 9034, 9036, 9038, 9040, 9041, 9043, 9044, 9045, 9046, 9047, 9048, 9049, 9050, 9051, 9053, 9055, 9056, 9057, 9058, 9059, 9060, 9061, 9063, 9064, 9065, 9067, 9068, 9069, 9070, 9072, 9074, 9075, 9076, 9077, 9081, 9084, 9085, 9087, 9089, 9090, 9091, 9092, 9094, 9095, 9097, 9099, 9100, 9101, 9102, 9103, 9104, 9105, 9106, 9107, 9108, 9109, 9110, 9111, 9112, 9113, 9114, 9115, 9116, 9117, 9119, 9121, 9122, 9123, 9124, 9125, 9126, 9127, 9133, 9135, 9136, 9137, 9138, 9139, 9141, 9143, 9144, 9145, 9147, 9148, 9149, 9150, 9152, 9155, 9156, 9158, 9161, 9165, 9166, 9167, 9169, 9170, 9172, 9173, 9174, 9175, 9176, 9177, 9178, 9179, 9180, 9181, 9182, 9183, 9184, 9186, 9187, 9190, 9191, 9192, 9196, 9199, 9200, 9201, 9203, 9204, 9206, 9208, 9209, 9210, 9211, 9212, 9214, 9216, 9217, 9218, 9220, 9221, 9222, 9223, 9225, 9226, 9229, 9232, 9233, 9234, 9236, 9237, 9238, 9239, 9240, 9243, 9244, 9245, 9246, 9248, 9249, 9251, 9252, 9253, 9254, 9255, 9256, 9261, 9262, 9264, 9266, 9267, 9268, 9269, 9270, 9272, 9273, 9274, 9275, 9277, 9278, 9279, 9280, 9281, 9282, 9285, 9287, 9288, 9290, 9291, 9294, 9295, 9296, 9297, 9298, 9301, 9303, 9304, 9306, 9307, 9308, 9309, 9310, 9311, 9312, 9313, 9314, 9315, 9318, 9319, 9321, 9322, 9323, 9324, 9325, 9326, 9327, 9328, 9329, 9330, 9333, 9334, 9335, 9336, 9339, 9345, 9346, 9348, 9349, 9350, 9353, 9354, 9355, 9356, 9357, 9358, 9359, 9361, 9362, 9363, 9366, 9367, 9368, 9369, 9370, 9371, 9373, 9374, 9375, 9378, 9380, 9382, 9383, 9385, 9386, 9388, 9389, 9390, 9391, 9392, 9393, 9395, 9397, 9399, 9400, 9401, 9402, 9403, 9406, 9407, 9408, 9410, 9411, 9414, 9415, 9416, 9417, 9418, 9421, 9422, 9423, 9424, 9425, 9426, 9428, 9429, 9431, 9432, 9433, 9434, 9436, 9437, 9438, 9439, 9441, 9442, 9443, 9445, 9446, 9447, 9448, 9449, 9450, 9451, 9452, 9453, 9457, 9458, 9459, 9460, 9461, 9464, 9465, 9466, 9467, 9468, 9470, 9472, 9473, 9474, 9475, 9476, 9477, 9478, 9479, 9480, 9481, 9482, 9483, 9486, 9487, 9488, 9489, 9491, 9492, 9494, 9496, 9497, 9499, 9500, 9501, 9502, 9503, 9504, 9505, 9508, 9509, 9510, 9511, 9512, 9514, 9515, 9516, 9521, 9523, 9524, 9525, 9527, 9528, 9529, 9530, 9531, 9532, 9533, 9534, 9535, 9536, 9537, 9538, 9540, 9541, 9542, 9543, 9544, 9545, 9546, 9547, 9549, 9550, 9551, 9552, 9553, 9554, 9555, 9559, 9560, 9561, 9564, 9567, 9568, 9571, 9573, 9574, 9576, 9577, 9578, 9579, 9580, 9582, 9583, 9584, 9585, 9586, 9587, 9589, 9590, 9591, 9592, 9594, 9598, 9599, 9601, 9604, 9605, 9606, 9607, 9608, 9609, 9610, 9612, 9613, 9614, 9615, 9616, 9617, 9619, 9620, 9621, 9622, 9623, 9624, 9626, 9628, 9629, 9630, 9631, 9632, 9633, 9635, 9636, 9637, 9638, 9640, 9641, 9642, 9643, 9645, 9646, 9647, 9648, 9649, 9652, 9654, 9655, 9658, 9659, 9660, 9661, 9662, 9663, 9664, 9665, 9666, 9667, 9668, 9669, 9672, 9673, 9676, 9678, 9679, 9680, 9682, 9684, 9685, 9686, 9687, 9692, 9693, 9694, 9696, 9697, 9698, 9699, 9700, 9701, 9702, 9703, 9704, 9706, 9707, 9708, 9709, 9710, 9711, 9712, 9714, 9716, 9717, 9718, 9719, 9720, 9722, 9723, 9725, 9726, 9727, 9728, 9729, 9731, 9732, 9733, 9735, 9736, 9737, 9739, 9740, 9741, 9742, 9743, 9745, 9748, 9750, 9752, 9753, 9754, 9755, 9756, 9758, 9759, 9761, 9762, 9763, 9764, 9766, 9768, 9769, 9770, 9771, 9773, 9774, 9776, 9777, 9778, 9779, 9780, 9782, 9783, 9784, 9786, 9787, 9788, 9790, 9791, 9794, 9795, 9796, 9798, 9801, 9802, 9804, 9805, 9806, 9808, 9809, 9811, 9812, 9814, 9815, 9817, 9818, 9819, 9822, 9823, 9824, 9825, 9826, 9829, 9830, 9831, 9833, 9836, 9837, 9838, 9839, 9840, 9841, 9842, 9844, 9845, 9846, 9848, 9849, 9850, 9851, 9852, 9853, 9854, 9855, 9856, 9857, 9858, 9859, 9860, 9862, 9864, 9865, 9868, 9871, 9874, 9875, 9877, 9878, 9879, 9880, 9882, 9884, 9885, 9887, 9889, 9890, 9891, 9894, 9895, 9897, 9898, 9899, 9900, 9902, 9907, 9908, 9910, 9912, 9913, 9915, 9916, 9917, 9918, 9921, 9923, 9924, 9925, 9927, 9929, 9930, 9931, 9932, 9934, 9935, 9937, 9938, 9940, 9941, 9942, 9943, 9945, 9946, 9947, 9948, 9949, 9950, 9951, 9952, 9953, 9954, 9956, 9959, 9960, 9961, 9963, 9964, 9965, 9966, 9969, 9970, 9971, 9972, 9973, 9975, 9976, 9977, 9978, 9981, 9983, 9984, 9985, 9986, 9988, 9989, 9990, 9991, 9993, 9995, 9996, 9997, 9999, 10000, 10002, 10003, 10004, 10005, 10006, 10008, 10010, 10011, 10014, 10015, 10017, 10018, 10019, 10020, 10021, 10023, 10024, 10025, 10026, 10027, 10028, 10029, 10030, 10032, 10033, 10034, 10035, 10036, 10039, 10040, 10041, 10043, 10044, 10045, 10049, 10051, 10053, 10056, 10057, 10058, 10059, 10060, 10061, 10062, 10063, 10064, 10065, 10066, 10067, 10068, 10069, 10070, 10071, 10073, 10074, 10075, 10076, 10077, 10078, 10080, 10081, 10085, 10086, 10087, 10090, 10093, 10094, 10095, 10096, 10097, 10099, 10100, 10101, 10102, 10103, 10104, 10105, 10106, 10107, 10108, 10109, 10110, 10111, 10112, 10113, 10115, 10116, 10119, 10120, 10121, 10123, 10124, 10125, 10126, 10127, 10129, 10130, 10131, 10132, 10133, 10135, 10136, 10137, 10139, 10140, 10141, 10142, 10143, 10145, 10146, 10147, 10149, 10150, 10152, 10154, 10155, 10156, 10157, 10158, 10159, 10160, 10161, 10162, 10163, 10164, 10165, 10166, 10167, 10168, 10169, 10170, 10171, 10172, 10173, 10174, 10175, 10176, 10177, 10178, 10180, 10182, 10183, 10184, 10185, 10190, 10191, 10192, 10193, 10194, 10195, 10196, 10197, 10201, 10203, 10204, 10209, 10210, 10211, 10213, 10215, 10216, 10217, 10219, 10220, 10222, 10223, 10224, 10225, 10226, 10227, 10230, 10231, 10234, 10235, 10236, 10238, 10239, 10240, 10241, 10242, 10243, 10244, 10245, 10246, 10247, 10248, 10249, 10251, 10252, 10253, 10254, 10257, 10258, 10259, 10260, 10261, 10262, 10264, 10265, 10266, 10267, 10268, 10271, 10272, 10275, 10276, 10277, 10279, 10280, 10284, 10285, 10286, 10288, 10289, 10290, 10291, 10292, 10293, 10294, 10298, 10299, 10300, 10301, 10302, 10303, 10304, 10305, 10306, 10307, 10308, 10309, 10310, 10311, 10312, 10313, 10314, 10315, 10317, 10319, 10321, 10322, 10323, 10324, 10325, 10326, 10327, 10328, 10329, 10330, 10331, 10332, 10333, 10334, 10337, 10338, 10339, 10342, 10343, 10344, 10345, 10348, 10350, 10351, 10352, 10353, 10354, 10355, 10356, 10357, 10358, 10361, 10362, 10363, 10364, 10365, 10366, 10368, 10369, 10372, 10374, 10375, 10379, 10380, 10381, 10383, 10384, 10387, 10392, 10394, 10395, 10397, 10399, 10401, 10402, 10403, 10406, 10409, 10410, 10411, 10413, 10414, 10415, 10416, 10417, 10418, 10419, 10420, 10423, 10424, 10425, 10426, 10427, 10428, 10431, 10432, 10433, 10434, 10435, 10436, 10440, 10441, 10442, 10444, 10445, 10446, 10447, 10449, 10452, 10453, 10454, 10455, 10456, 10457, 10458, 10460, 10461, 10463, 10465, 10466, 10467, 10468, 10472, 10474, 10475, 10478, 10479, 10482, 10483, 10484, 10485, 10486, 10487, 10488, 10489, 10490, 10491, 10494, 10495, 10497, 10498, 10499, 10500, 10501, 10503, 10504, 10505, 10507, 10508, 10509, 10512, 10514, 10515, 10516, 10517, 10521, 10522, 10525, 10527, 10528, 10531, 10532, 10534, 10535, 10536, 10537, 10538, 10539, 10540, 10541, 10542, 10546, 10547, 10548, 10549, 10550, 10551, 10553, 10557, 10562, 10564, 10566, 10567, 10568, 10570, 10571, 10572, 10573, 10574, 10575, 10577, 10578, 10579, 10581, 10582, 10583, 10584, 10586, 10589, 10590, 10591, 10592, 10593, 10594, 10595, 10596, 10597, 10598, 10601, 10603, 10604, 10605, 10606, 10607, 10608, 10609, 10610, 10612, 10613, 10614, 10616, 10620, 10624, 10626, 10627, 10628, 10630, 10631, 10632, 10634, 10636, 10639, 10640, 10641, 10642, 10643, 10644, 10645, 10646, 10647, 10649, 10651, 10652, 10653, 10654, 10655, 10656, 10657, 10658, 10660, 10661, 10662, 10663, 10664, 10665, 10666, 10667, 10668, 10670, 10671, 10672, 10673, 10675, 10676, 10677, 10678, 10679, 10680, 10682, 10683, 10685, 10686, 10687, 10688, 10689, 10691, 10692, 10693, 10694, 10695, 10701, 10702, 10705, 10708, 10709, 10710, 10711, 10712, 10713, 10718, 10719, 10722, 10723, 10724, 10725, 10726, 10727, 10728, 10730, 10732, 10733, 10736, 10738, 10739, 10740, 10743, 10744, 10745, 10746, 10747, 10749, 10750, 10753, 10754, 10755, 10756, 10757, 10758, 10761, 10762, 10764, 10765, 10767, 10768, 10769, 10770, 10771, 10772, 10773, 10774, 10775, 10776, 10777, 10778, 10779, 10782, 10786, 10787, 10789, 10790, 10792, 10793, 10794, 10795, 10796, 10797, 10798, 10800, 10801, 10803, 10804, 10805, 10806, 10808, 10810, 10812, 10814, 10815, 10816, 10817, 10819, 10820, 10821, 10822, 10824, 10825, 10826, 10827, 10828, 10829, 10830, 10831, 10834, 10835, 10836, 10839, 10840, 10841, 10842, 10843, 10845, 10846, 10847, 10848, 10849, 10851, 10853, 10854, 10855, 10856, 10857, 10858, 10859, 10860, 10863, 10866, 10867, 10868, 10869, 10872, 10873, 10876, 10877, 10878, 10880, 10881, 10882, 10883, 10884, 10885, 10887, 10888, 10889, 10890, 10891, 10892, 10893, 10894, 10896, 10897, 10898, 10899, 10900, 10902, 10903, 10906, 10908, 10910, 10912, 10913, 10914, 10915, 10916, 10917, 10918, 10919, 10922, 10923, 10924, 10926, 10927, 10929, 10930, 10933, 10934, 10936, 10937, 10939, 10940, 10941, 10942, 10943, 10944, 10945, 10946, 10947, 10949, 10951, 10954, 10955, 10957, 10958, 10959, 10961, 10962, 10963, 10965, 10966, 10968, 10969, 10970, 10972, 10973, 10974, 10975, 10977, 10978, 10979, 10980, 10981, 10982, 10983, 10985, 10986, 10987, 10988, 10991, 10992, 10994, 10995, 10997, 11002, 11003, 11004, 11005, 11009, 11011, 11013, 11015, 11016, 11017, 11019, 11021, 11022, 11023, 11024, 11025, 11026, 11027, 11028, 11031, 11034, 11035, 11036, 11037, 11038, 11039, 11040, 11041, 11042, 11043, 11044, 11045, 11047, 11048, 11049, 11050, 11051, 11052, 11054, 11056, 11057, 11058, 11059, 11060, 11061, 11062, 11063, 11064, 11065, 11071, 11072, 11073, 11074, 11075, 11077, 11078, 11079, 11080, 11081, 11082, 11083, 11084, 11086, 11087, 11089, 11092, 11093, 11094, 11097, 11098, 11099, 11100, 11101, 11102, 11103, 11104, 11105, 11106, 11108, 11109, 11110, 11111, 11113, 11115, 11116, 11118, 11119, 11120, 11121, 11122, 11123, 11130, 11131, 11133, 11134, 11135, 11136, 11138, 11139, 11140, 11141, 11142, 11143, 11144, 11145, 11146, 11147, 11148, 11149, 11151, 11155, 11157, 11158, 11159, 11160, 11161, 11164, 11165, 11166, 11167, 11170, 11171, 11172, 11173, 11174, 11176, 11177, 11178, 11179, 11180, 11181, 11182, 11183, 11186, 11187, 11188, 11189, 11190, 11191, 11194, 11195, 11197, 11198, 11201, 11203, 11204, 11205, 11206, 11207, 11208, 11209, 11210, 11212, 11213, 11214, 11217, 11218, 11219, 11221, 11224, 11225, 11226, 11228, 11230, 11231, 11232, 11234, 11235, 11237, 11238, 11239, 11240, 11242, 11243, 11244, 11245, 11246, 11248, 11249, 11250, 11251, 11253, 11254, 11255, 11256, 11259, 11260, 11261, 11262, 11263, 11264, 11265, 11267, 11270, 11271, 11272, 11273, 11274, 11275, 11276, 11277, 11279, 11280, 11281, 11283, 11284, 11286, 11288, 11290, 11291, 11292, 11294, 11296, 11297, 11298, 11299, 11301, 11302, 11303, 11304, 11305, 11306, 11307, 11309, 11311, 11312, 11313, 11314, 11315, 11316, 11319, 11321, 11322, 11323, 11326, 11327, 11328, 11329, 11330, 11331, 11332, 11333, 11334, 11335, 11336, 11337, 11339, 11342, 11343, 11344, 11345, 11346, 11347, 11348, 11350, 11351, 11352, 11353, 11354, 11355, 11358, 11360, 11361, 11362, 11363, 11364, 11365, 11366, 11367, 11368, 11369, 11370, 11372, 11374, 11376, 11379, 11380, 11382, 11383, 11384, 11386, 11387, 11388, 11389, 11390, 11392, 11393, 11396, 11397, 11398, 11401, 11402, 11403, 11404, 11405, 11406, 11407, 11408, 11409, 11413, 11414, 11416, 11417, 11419, 11420, 11421, 11422, 11424, 11425, 11426, 11427, 11428, 11429, 11430, 11432, 11433, 11436, 11437, 11439, 11440, 11441, 11442, 11443, 11444, 11445, 11446, 11447, 11448, 11450, 11452, 11453, 11454, 11455, 11456, 11460, 11461, 11463, 11464, 11465, 11466, 11467, 11468, 11470, 11472, 11473, 11474, 11475, 11476, 11477, 11478, 11481, 11482, 11483, 11484, 11487, 11488, 11491, 11493, 11494, 11495, 11496, 11497, 11498, 11499, 11500, 11501, 11502, 11503, 11504, 11505, 11507, 11508, 11509, 11510, 11511, 11512, 11513, 11515, 11516, 11517, 11518, 11519, 11520, 11521, 11522, 11526, 11527, 11528, 11529, 11530, 11532, 11535, 11536, 11537, 11538, 11539, 11540, 11541, 11542, 11543, 11545, 11547, 11549, 11550, 11551, 11552, 11555, 11556, 11558, 11559, 11561, 11562, 11563, 11565, 11566, 11567, 11568, 11569, 11571, 11572, 11573, 11574, 11577, 11579, 11580, 11581, 11582, 11584, 11585, 11586, 11587, 11588, 11589, 11591, 11592, 11593, 11594, 11595, 11597, 11600, 11602, 11603, 11604, 11605, 11606, 11608, 11609, 11611, 11613, 11614, 11615, 11616, 11617, 11618, 11621, 11624, 11626, 11628, 11629, 11630, 11631, 11633, 11634, 11637, 11640, 11642, 11643, 11644, 11645, 11646, 11647, 11648, 11649, 11650, 11652, 11653, 11655, 11656, 11657, 11658, 11659, 11661, 11665, 11666, 11667, 11668, 11669, 11671, 11672, 11673, 11676, 11677, 11679, 11680, 11682, 11683, 11684, 11685, 11686, 11687, 11688, 11689, 11690, 11691, 11692, 11693, 11694, 11697, 11698, 11699, 11700, 11701, 11702, 11703, 11704, 11705, 11706, 11707, 11708, 11711, 11713, 11714, 11716, 11717, 11718, 11720, 11721, 11722, 11724, 11725, 11727, 11728, 11730, 11734, 11735, 11736, 11737, 11738, 11739, 11741, 11742, 11743, 11744, 11745, 11746, 11747, 11749, 11750, 11752, 11753, 11754, 11755, 11756, 11758, 11759, 11762, 11763, 11764, 11765, 11766, 11767, 11769, 11770, 11771, 11772, 11773, 11774, 11775, 11776, 11778, 11779, 11780, 11781, 11782, 11783, 11784, 11786, 11787, 11788, 11789, 11790, 11791, 11793, 11795, 11796, 11797, 11798, 11799, 11800, 11801, 11802, 11803, 11805, 11807, 11809, 11810, 11811, 11812, 11813, 11814, 11816, 11817, 11818, 11819, 11820, 11821, 11824, 11825, 11826, 11827, 11829, 11830, 11831, 11832, 11833, 11834, 11836, 11837, 11838, 11839, 11840, 11841, 11842, 11847, 11848, 11849, 11850, 11851, 11853, 11854, 11855, 11858, 11860, 11862, 11865, 11866, 11867, 11869, 11870, 11871, 11873, 11874, 11875, 11876, 11877, 11878, 11879, 11880, 11882, 11884, 11887, 11888, 11889, 11890, 11892, 11893, 11894, 11899, 11900, 11901, 11902, 11903, 11904, 11905, 11906, 11909, 11910, 11911, 11913, 11917, 11918, 11919, 11920, 11921, 11922, 11925, 11926, 11927, 11928, 11929, 11930, 11934, 11935, 11941, 11943, 11944, 11945, 11946, 11947, 11948, 11949, 11950, 11952, 11954, 11955, 11958, 11959, 11960, 11961, 11963, 11964, 11965, 11966, 11970, 11971, 11975, 11976, 11977, 11978, 11979, 11980, 11981, 11982, 11983, 11986, 11987, 11988, 11989, 11990, 11991, 11992, 11996, 11997, 11998, 11999, 12000, 12002, 12003, 12004, 12005, 12006, 12007, 12009, 12010, 12011, 12012, 12013, 12014, 12015, 12016, 12017, 12018, 12020, 12021, 12022, 12023, 12025, 12026, 12027, 12030, 12031, 12032, 12033, 12036, 12038, 12039, 12041, 12042, 12044, 12045, 12046, 12047, 12048, 12049, 12050, 12051, 12053, 12055, 12056, 12057, 12058, 12059, 12060, 12061, 12063, 12064, 12065, 12067, 12069, 12070, 12071, 12073, 12077, 12078, 12079, 12081, 12082, 12083, 12085, 12087, 12091, 12092, 12094, 12095, 12098, 12100, 12103, 12105, 12106, 12108, 12109, 12110, 12111, 12113, 12114, 12115, 12116, 12117, 12118, 12119, 12120, 12123, 12124, 12125, 12127, 12130, 12131, 12135, 12137, 12140, 12141, 12142, 12143, 12145, 12146, 12147, 12149, 12150, 12151, 12152, 12153, 12154, 12155, 12157, 12158, 12159, 12160, 12161, 12162, 12163, 12165, 12167, 12168, 12169, 12170, 12171, 12172, 12174, 12175, 12176, 12177, 12178, 12179, 12180, 12181, 12182, 12183, 12184, 12189, 12190, 12191, 12192, 12193, 12194, 12195, 12196, 12197, 12198, 12199, 12200, 12201, 12202, 12203, 12204, 12205, 12206, 12207, 12208, 12209, 12211, 12212, 12213, 12214, 12215, 12216, 12217, 12218, 12220, 12223, 12226, 12227, 12229, 12234, 12235, 12236, 12237, 12239, 12243, 12244, 12245, 12247, 12249, 12250, 12251, 12252, 12257, 12261, 12262, 12264, 12267, 12268, 12269, 12270, 12271, 12275, 12277, 12278, 12279, 12281, 12282, 12283, 12284, 12285, 12286, 12287, 12288, 12290, 12291, 12294, 12295, 12297, 12300, 12302, 12303, 12304, 12308, 12309, 12310, 12311, 12313, 12314, 12315, 12317, 12318, 12319, 12320, 12321, 12322, 12323, 12324, 12325, 12327, 12331, 12332, 12333, 12334, 12335, 12336, 12337, 12341, 12343, 12344, 12345, 12346, 12347, 12350, 12351, 12352, 12353, 12355, 12356, 12358, 12359, 12360, 12361, 12366, 12367, 12369, 12370, 12372, 12374, 12375, 12377, 12378, 12379, 12380, 12382, 12383, 12386, 12387, 12389, 12390, 12391, 12392, 12393, 12394, 12395, 12397, 12398, 12399, 12400, 12401, 12402, 12404, 12407, 12409, 12410, 12414, 12416, 12417, 12418, 12421, 12423, 12424, 12425, 12426, 12427, 12428, 12429, 12430, 12432, 12433, 12434, 12435, 12436, 12437, 12438, 12441, 12442, 12443, 12444, 12446, 12447, 12448, 12452, 12453, 12455, 12458, 12459, 12460, 12462, 12463, 12464, 12465, 12466, 12467, 12469, 12470, 12471, 12473, 12474, 12476, 12477, 12478, 12479, 12481, 12482, 12483, 12485, 12487, 12488, 12491, 12492, 12496, 12497, 12498, 12499, 12500, 12503, 12504, 12505, 12507, 12508, 12510, 12511, 12512, 12513, 12514, 12515, 12516, 12518, 12519, 12520, 12521, 12522, 12524, 12525, 12526, 12527, 12528, 12529, 12530, 12531, 12532, 12533, 12534, 12535, 12536, 12538, 12539, 12540, 12541, 12542, 12543, 12545, 12546, 12547, 12549, 12550, 12551, 12554, 12555, 12556, 12557, 12558, 12559, 12560, 12561, 12562, 12563, 12565, 12566, 12567, 12569, 12572, 12573, 12576, 12577, 12578, 12579, 12580, 12582, 12583, 12584, 12586, 12587, 12588, 12589, 12590, 12591, 12592, 12594, 12595, 12596, 12598, 12600, 12601, 12603, 12604, 12605, 12607, 12611, 12612, 12613, 12616, 12617, 12618, 12619, 12620, 12622, 12624, 12625, 12626, 12627, 12628, 12629, 12630, 12631, 12633, 12636, 12637, 12638, 12639, 12640, 12642, 12643, 12644, 12645, 12647, 12648, 12650, 12651, 12654, 12655, 12656, 12659, 12660, 12661, 12662, 12664, 12667, 12668, 12669, 12671, 12672, 12674, 12675, 12676, 12677, 12679, 12684, 12686, 12687, 12688, 12689, 12690, 12691, 12692, 12694, 12696, 12697, 12698, 12699, 12700, 12701, 12704, 12705, 12706, 12707, 12711, 12712, 12713, 12714, 12715, 12716, 12717, 12719, 12720, 12722, 12723, 12724, 12725, 12726, 12727, 12729, 12730, 12733, 12734, 12736, 12737, 12738, 12739, 12740, 12741, 12744, 12746, 12747, 12748, 12749, 12751, 12752, 12753, 12754, 12755, 12756, 12757, 12759, 12760, 12764, 12767, 12768, 12771, 12772, 12773, 12774, 12775, 12778, 12779, 12781, 12782, 12783, 12784, 12785, 12786, 12787, 12790, 12792, 12793, 12795, 12796, 12798, 12799, 12800, 12801, 12802, 12805, 12806, 12807, 12808, 12809, 12810, 12811, 12812, 12814, 12815, 12817, 12819, 12821, 12822, 12823, 12824, 12826, 12828, 12829, 12830, 12831, 12833, 12835, 12836, 12837, 12838, 12840, 12841, 12843, 12844, 12846, 12847, 12848, 12849, 12851, 12852, 12853, 12854, 12855, 12856, 12857, 12858, 12859, 12860, 12861, 12863, 12865, 12867, 12868, 12869, 12871, 12872, 12873, 12874, 12876, 12877, 12880, 12882, 12883, 12884, 12885, 12886, 12887, 12889, 12890, 12891, 12893, 12894, 12895, 12896, 12898, 12899, 12900, 12902, 12903, 12904, 12905, 12907, 12909, 12910, 12911, 12912, 12916, 12918, 12919, 12922, 12923, 12924, 12925, 12926, 12927, 12928, 12930, 12933, 12934, 12935, 12937, 12938, 12939, 12940, 12941, 12942, 12943, 12945, 12946, 12950, 12951, 12952, 12953, 12954, 12955, 12957, 12958, 12959, 12960, 12961, 12964, 12966, 12969, 12974, 12975, 12976, 12977, 12978, 12982, 12984, 12985, 12986, 12987, 12989, 12991, 12992, 12993, 12996, 12998, 12999, 13000, 13003, 13004, 13005, 13006, 13007, 13008, 13009, 13010, 13011, 13012, 13015, 13016, 13017, 13018, 13019, 13020, 13021, 13024, 13025, 13027, 13028, 13030, 13031, 13033, 13035, 13036, 13037, 13039, 13040, 13041, 13042, 13047, 13049, 13050, 13051, 13052, 13053, 13054, 13055, 13056, 13058, 13059, 13060, 13061, 13062, 13063, 13064, 13065, 13066, 13067, 13068, 13069, 13070, 13073, 13075, 13076, 13078, 13079, 13080, 13083, 13084, 13087, 13088, 13089, 13090, 13091, 13092, 13094, 13095, 13096, 13097, 13099, 13100, 13102, 13103, 13105, 13106, 13108, 13110, 13111, 13113, 13114, 13115, 13117, 13118, 13119, 13122, 13123, 13126, 13127, 13130, 13131, 13132, 13133, 13134, 13135, 13136, 13138, 13139, 13140, 13142, 13144, 13146, 13150, 13151, 13155, 13157, 13158, 13159, 13162, 13163, 13164, 13165, 13166, 13170, 13171, 13172, 13174, 13175, 13178, 13180, 13181, 13182, 13183, 13184, 13185, 13187, 13190, 13192, 13193, 13195, 13197, 13199, 13200, 13201, 13202, 13203, 13204, 13206, 13208, 13209, 13210, 13211, 13212, 13213, 13214, 13215, 13216, 13217, 13218, 13219, 13222, 13224, 13225, 13228, 13229, 13230, 13231, 13233, 13234, 13237, 13239, 13241, 13242, 13243, 13244, 13245, 13249, 13250, 13251, 13252, 13253, 13255, 13256, 13257, 13258, 13259, 13260, 13261, 13262, 13263, 13264, 13265, 13267, 13268, 13269, 13271, 13273, 13274, 13275, 13278, 13281, 13282, 13283, 13284, 13286, 13287, 13289, 13290, 13291, 13292, 13293, 13294, 13296, 13298, 13299, 13301, 13302, 13303, 13304, 13305, 13306, 13307, 13308, 13309, 13310, 13311, 13314, 13316, 13317, 13318, 13319, 13320, 13321, 13322, 13323, 13325, 13327, 13328, 13329, 13331, 13333, 13335, 13337, 13338, 13339, 13340, 13342, 13343, 13345, 13346, 13347, 13348, 13349, 13350, 13353, 13354, 13355, 13356, 13357, 13358, 13359, 13360, 13361, 13362, 13363, 13364, 13367, 13368, 13369, 13370, 13372, 13373, 13374, 13375, 13378, 13379, 13380, 13381, 13382, 13383, 13384, 13385, 13386, 13387, 13388, 13389, 13391, 13392, 13395, 13396, 13397, 13398, 13399, 13400, 13401, 13402, 13403, 13404, 13406, 13407, 13408, 13410, 13411, 13412, 13415, 13417, 13418, 13419, 13423, 13424, 13425, 13427, 13428, 13431, 13432, 13433, 13434, 13435, 13437, 13438, 13443, 13444, 13445, 13446, 13447, 13448, 13449, 13453, 13454, 13455, 13457, 13458, 13459, 13460, 13462, 13464, 13466, 13467, 13468, 13469, 13470, 13471, 13474, 13475, 13477, 13478, 13479, 13480, 13481, 13482, 13483, 13484, 13485, 13487, 13488, 13490, 13491, 13492, 13493, 13495, 13496, 13498, 13499, 13500, 13502, 13503, 13504, 13505, 13508, 13511, 13512, 13513, 13515, 13516, 13517, 13518, 13521, 13524, 13525, 13526, 13527, 13528, 13529, 13530, 13532, 13533, 13534, 13535, 13536, 13537, 13538, 13539, 13540, 13541, 13542, 13543, 13544, 13545, 13546, 13547, 13548, 13549, 13550, 13551, 13552, 13553, 13554, 13555, 13556, 13557, 13558, 13559, 13560, 13561, 13563, 13565, 13566, 13568, 13570, 13573, 13574, 13578, 13579, 13580, 13581, 13582, 13584, 13585, 13586, 13587, 13589, 13590, 13591, 13593, 13594, 13595, 13596, 13597, 13598, 13601, 13603, 13605, 13606, 13607, 13608, 13609, 13611, 13613, 13614, 13617, 13618, 13619, 13620, 13621, 13622, 13623, 13624, 13625, 13626, 13627, 13628, 13629, 13630, 13631, 13633, 13634, 13635, 13636, 13637, 13638, 13639, 13641, 13642, 13643, 13644, 13646, 13647, 13648, 13649, 13651, 13653, 13655, 13658, 13660, 13661, 13662, 13663, 13664, 13665, 13667, 13668, 13669, 13670, 13672, 13674, 13675, 13676, 13680, 13681, 13682, 13683, 13684, 13685, 13686, 13687, 13688, 13689, 13690, 13691, 13692, 13693, 13695, 13697, 13699, 13700, 13701, 13702, 13703, 13705, 13706, 13708, 13709, 13710, 13711, 13712, 13713, 13714, 13715, 13716, 13717, 13718, 13719, 13720, 13722, 13723, 13724, 13725, 13726, 13727, 13728, 13729, 13731, 13732, 13733, 13735, 13736, 13737, 13740, 13741, 13744, 13745, 13746, 13748, 13750, 13751, 13754, 13755, 13757, 13758, 13759, 13760, 13761, 13763, 13764, 13765, 13766, 13768, 13769, 13770, 13771, 13772, 13773, 13774, 13775, 13776, 13777, 13778, 13781, 13782, 13783, 13785, 13786, 13787, 13788, 13789, 13790, 13791, 13792, 13793, 13794, 13795, 13796, 13798, 13800, 13801, 13802, 13803, 13805, 13806, 13807, 13808, 13809, 13811, 13812, 13813, 13815, 13816, 13817, 13818, 13819, 13824, 13826, 13827, 13828, 13830, 13832, 13835, 13836, 13837, 13839, 13840, 13843, 13844, 13845, 13846, 13849, 13851, 13854, 13857, 13858, 13859, 13860, 13861, 13862, 13864, 13866, 13867, 13868, 13869, 13871, 13872, 13873, 13874, 13875, 13876, 13878, 13879, 13880, 13881, 13882, 13883, 13885, 13886, 13888, 13889, 13892, 13893, 13895, 13896, 13898, 13899, 13900, 13901, 13902, 13905, 13906, 13908, 13909, 13910, 13911, 13912, 13913, 13914, 13915, 13916, 13917, 13918, 13919, 13921, 13923, 13924, 13926, 13928, 13930, 13931, 13932, 13933, 13934, 13935, 13936, 13937, 13938, 13942, 13943, 13944, 13946, 13947, 13948, 13949, 13950, 13951, 13952, 13953, 13954, 13956, 13957, 13958, 13959, 13961, 13963, 13965, 13966, 13967, 13969, 13971, 13972, 13973, 13975, 13976, 13977, 13979, 13981, 13983, 13984, 13985, 13986, 13987, 13989, 13990, 13992, 13993, 13994, 13995, 13997, 13998, 13999, 14000, 14001, 14002, 14003, 14004, 14005, 14006, 14007, 14008, 14009, 14010, 14011, 14012, 14015, 14016, 14017, 14019, 14020, 14021, 14022, 14023, 14026, 14027, 14029, 14030, 14031, 14032, 14033, 14034, 14036, 14037, 14040, 14041, 14043, 14044, 14045, 14049, 14050, 14051, 14052, 14053, 14055, 14057, 14058, 14061, 14062, 14063, 14064, 14065, 14066, 14068, 14069, 14072, 14074, 14075, 14076, 14078, 14079, 14080, 14081, 14082, 14084, 14088, 14090, 14091, 14092, 14094, 14095, 14097, 14098, 14100, 14101, 14102, 14103, 14104, 14108, 14110, 14111, 14112, 14115, 14116, 14120, 14121, 14123, 14125, 14126, 14127, 14129, 14132, 14134, 14135, 14136, 14140, 14142, 14143, 14144, 14146, 14148, 14150, 14151, 14152, 14153, 14155, 14157, 14158, 14159, 14160, 14161, 14162, 14163, 14164, 14167, 14168, 14169, 14170, 14171, 14172, 14173, 14174, 14176, 14177, 14178, 14180, 14181, 14183, 14185, 14186, 14188, 14189, 14190, 14191, 14192, 14193, 14194, 14195, 14196, 14197, 14199, 14201, 14202, 14203, 14205, 14208, 14210, 14211, 14212, 14216, 14217, 14218, 14219, 14221, 14224, 14225, 14226, 14227, 14228, 14230, 14231, 14232, 14233, 14234, 14235, 14239, 14240, 14241, 14242, 14243, 14245, 14246, 14247, 14249, 14251, 14252, 14253, 14256, 14257, 14258, 14259, 14260, 14261, 14262, 14265, 14266, 14267, 14268, 14270, 14271, 14273, 14274, 14277, 14280, 14282, 14283, 14284, 14285, 14286, 14287, 14288, 14290, 14292, 14293, 14295, 14297, 14298, 14299, 14300, 14301, 14302, 14303, 14304, 14306, 14307, 14308, 14310, 14312, 14313, 14316, 14317, 14318, 14319, 14320, 14322, 14323, 14324, 14325, 14326, 14328, 14329, 14330, 14331, 14332, 14333, 14336, 14337, 14342, 14343, 14344, 14346, 14347, 14348, 14349, 14350, 14352, 14355, 14357, 14358, 14359, 14361, 14362, 14364, 14366, 14367, 14368, 14369, 14370, 14372, 14374, 14375, 14378, 14379, 14381, 14382, 14384, 14387, 14394, 14395, 14396, 14397, 14399, 14400, 14402, 14403, 14405, 14407, 14410, 14411, 14412, 14413, 14414, 14416, 14417, 14419, 14420, 14422, 14423, 14426, 14427, 14429, 14431, 14432, 14433, 14434, 14437, 14438, 14439, 14440, 14441, 14443, 14444, 14445, 14446, 14449, 14450, 14451, 14453, 14454, 14455, 14456, 14457, 14458, 14461, 14464, 14465, 14466, 14468, 14469, 14470, 14471, 14472, 14473, 14474, 14475, 14476, 14478, 14479, 14482, 14484, 14486, 14487, 14489, 14494, 14495, 14496, 14497, 14502, 14504, 14506, 14507, 14508, 14509, 14510, 14512, 14513, 14514, 14515, 14518, 14519, 14521, 14522, 14523, 14524, 14525, 14526, 14527, 14528, 14529, 14530, 14531, 14534, 14535, 14536, 14537, 14539, 14540, 14541, 14543, 14544, 14545, 14546, 14547, 14548, 14549, 14550, 14551, 14553, 14555, 14556, 14557, 14558, 14559, 14560, 14564, 14566, 14568, 14570, 14572, 14574, 14575, 14577, 14578, 14579, 14580, 14581, 14582, 14583, 14584, 14586, 14587, 14588, 14589, 14590, 14592, 14593, 14596, 14597, 14598, 14600, 14603, 14604, 14605, 14606, 14607, 14608, 14610, 14613, 14614, 14615, 14616, 14617, 14618, 14619, 14623, 14624, 14625, 14626, 14627, 14631, 14634, 14636, 14638, 14640, 14641, 14642, 14644, 14645, 14646, 14647, 14648, 14649, 14650, 14652, 14653, 14654, 14655, 14657, 14658, 14659, 14660, 14664, 14667, 14668, 14670, 14671, 14673, 14675, 14676, 14677, 14678, 14679, 14682, 14683, 14684, 14685, 14687, 14688, 14689, 14690, 14691, 14692, 14693, 14694, 14697, 14698, 14700, 14701, 14703, 14704, 14705, 14706, 14707, 14708, 14710, 14711, 14716, 14717, 14719, 14720, 14721, 14722, 14724, 14726, 14727, 14728, 14729, 14730, 14732, 14733, 14735, 14737, 14738, 14739, 14741, 14742, 14743, 14744, 14745, 14746, 14748, 14749, 14750, 14751, 14752, 14753, 14755, 14756, 14757, 14758, 14759, 14760, 14762, 14764, 14766, 14767, 14768, 14771, 14772, 14773, 14774, 14776, 14779, 14780, 14781, 14783, 14785, 14786, 14787, 14788, 14789, 14790, 14793, 14796, 14797, 14799, 14800, 14802, 14803, 14804, 14805, 14806, 14808, 14809, 14810, 14811, 14812, 14813, 14815, 14816, 14817, 14818, 14819, 14821, 14822, 14823, 14824, 14826, 14827, 14828, 14829, 14831, 14834, 14835, 14836, 14837, 14838, 14840, 14841, 14842, 14844, 14845, 14849, 14850, 14851, 14852, 14853, 14854, 14856, 14858, 14859, 14861, 14863, 14864, 14866, 14867, 14868, 14870, 14871, 14872, 14873, 14874, 14875, 14878, 14880, 14881, 14882, 14883, 14884, 14885, 14886, 14887, 14890, 14891, 14893, 14894, 14895, 14896, 14897, 14899, 14900, 14902, 14903, 14904, 14905, 14906, 14907, 14908, 14910, 14911, 14913, 14914, 14916, 14918, 14919, 14920, 14921, 14922, 14923, 14924, 14926, 14927, 14928, 14930, 14932, 14933, 14934, 14935, 14937, 14940, 14941, 14944, 14945, 14948, 14949, 14950, 14951, 14952, 14953, 14954, 14955, 14956, 14958, 14962, 14965, 14966, 14967, 14968, 14969, 14971, 14972, 14973, 14974, 14975, 14976, 14977, 14979, 14980, 14981, 14982, 14985, 14986, 14987, 14988, 14989, 14992, 14996, 14998, 14999, 15000, 15001, 15002, 15003, 15004, 15006, 15009, 15011, 15014, 15015, 15016, 15017, 15018, 15019, 15020, 15021, 15022, 15023, 15024, 15025, 15026, 15027, 15030, 15031, 15032, 15033, 15034, 15035, 15036, 15037, 15041, 15042, 15043, 15044, 15045, 15046, 15047, 15048, 15050, 15052, 15053, 15054, 15056, 15057, 15058, 15059, 15062, 15063, 15064, 15065, 15066, 15068, 15069, 15070, 15072, 15073, 15075, 15076, 15078, 15080, 15081, 15083, 15084, 15085, 15086, 15087, 15088, 15090, 15093, 15094, 15095, 15097, 15098, 15099, 15100, 15101, 15102, 15103, 15104, 15105, 15106, 15108, 15109, 15111, 15112, 15113, 15115, 15116, 15118, 15119, 15121, 15122, 15124, 15125, 15129, 15130, 15133, 15134, 15135, 15136, 15137, 15138, 15139, 15140, 15141, 15144, 15145, 15147, 15148, 15151, 15154, 15156, 15157, 15158, 15159, 15161, 15162, 15165, 15167, 15168, 15169, 15170, 15171, 15174, 15175, 15176, 15177, 15178, 15179, 15180, 15181, 15185, 15187, 15188, 15189, 15190, 15192, 15193, 15194, 15195, 15196, 15198, 15199, 15200, 15202, 15203, 15204, 15205, 15206, 15207, 15208, 15209, 15210, 15211, 15212, 15215, 15216, 15217, 15218, 15219, 15220, 15221, 15222, 15223, 15224, 15227, 15228, 15229, 15230, 15231, 15232, 15234, 15235, 15236, 15237, 15239, 15240, 15241, 15242, 15243, 15246, 15247, 15248, 15251, 15252, 15253, 15254, 15255, 15256, 15258, 15259, 15261, 15263, 15264, 15265, 15266, 15267, 15268, 15269, 15270, 15271, 15273, 15274, 15276, 15277, 15278, 15279, 15280, 15281, 15282, 15283, 15286, 15287, 15289, 15291, 15293, 15294, 15295, 15297, 15298, 15299, 15301, 15302, 15303, 15304, 15307, 15308, 15309, 15310, 15312, 15315, 15320, 15321, 15322, 15324, 15325, 15326, 15328, 15329, 15330, 15331, 15332, 15333, 15335, 15336, 15338, 15339, 15342, 15344, 15345, 15346, 15347, 15349, 15353, 15354, 15355, 15356, 15358, 15359, 15360, 15363, 15366, 15368, 15369, 15370, 15371, 15375, 15376, 15377, 15378, 15379, 15380, 15381, 15382, 15385, 15386, 15387, 15389, 15390, 15391, 15393, 15394, 15396, 15398, 15399, 15401, 15404, 15406, 15408, 15409, 15410, 15412, 15414, 15415, 15416, 15417, 15418, 15419, 15420, 15421, 15422, 15423, 15424, 15425, 15426, 15427, 15432, 15436, 15437, 15438, 15439, 15440, 15441, 15442, 15443, 15445, 15446, 15447, 15449, 15450, 15451, 15454, 15455, 15456, 15457, 15460, 15462, 15463, 15464, 15465, 15467, 15468, 15469, 15471, 15472, 15473, 15474, 15475, 15478, 15479, 15480, 15482, 15483, 15484, 15485, 15488, 15489, 15490, 15494, 15495, 15496, 15497, 15498, 15499, 15500, 15501, 15502, 15503, 15504, 15505, 15508, 15509, 15510, 15515, 15516, 15517, 15519, 15520, 15521, 15522, 15523, 15524, 15526, 15530, 15532, 15533, 15536, 15537, 15538, 15539, 15541, 15542, 15544, 15545, 15546, 15547, 15548, 15549, 15550, 15551, 15552, 15553, 15555, 15556, 15557, 15558, 15559, 15560, 15561, 15563, 15564, 15565, 15567, 15569, 15570, 15571, 15573, 15574, 15576, 15577, 15578, 15581, 15585, 15586, 15587, 15589, 15591, 15592, 15594, 15595, 15596, 15598, 15599, 15600, 15602, 15604, 15606, 15607, 15608, 15609, 15611, 15612, 15613, 15618, 15619, 15620, 15621, 15622, 15623, 15624, 15625, 15626, 15627, 15628, 15629, 15630, 15631, 15633, 15634, 15637, 15638, 15639, 15641, 15642, 15643, 15644, 15645, 15646, 15647, 15648, 15649, 15650, 15652, 15654, 15656, 15657, 15658, 15661, 15662, 15663, 15664, 15666, 15667, 15668, 15669, 15671, 15672, 15674, 15676, 15677, 15680, 15682, 15683, 15684, 15685, 15687, 15688, 15689, 15690, 15691, 15692, 15693, 15695, 15696, 15699, 15700, 15701, 15702, 15703, 15704, 15705, 15710, 15712, 15714, 15718, 15719, 15721, 15722, 15723, 15724, 15725, 15726, 15728, 15730, 15731, 15733, 15735, 15737, 15738, 15740, 15741, 15742, 15743, 15744, 15745, 15746, 15747, 15748, 15751, 15752, 15753, 15754, 15755, 15756, 15758, 15759, 15760, 15761, 15762, 15765, 15766, 15767, 15768, 15769, 15770, 15771, 15772, 15774, 15775, 15777, 15778, 15779, 15785, 15786, 15787, 15788, 15790, 15791, 15792, 15793, 15796, 15797, 15798, 15799, 15800, 15801, 15802, 15803, 15806, 15807, 15808, 15809, 15811, 15812, 15813, 15815, 15817, 15821, 15822, 15823, 15825, 15827, 15828, 15829, 15830, 15831, 15834, 15835, 15837, 15838, 15840, 15841, 15843, 15844, 15845, 15847, 15848, 15849, 15851, 15852, 15853, 15855, 15856, 15857, 15858, 15861, 15864, 15866, 15869, 15871, 15872, 15873, 15874, 15875, 15876, 15877, 15880, 15881, 15882, 15883, 15884, 15886, 15887, 15890, 15891, 15892, 15893, 15894, 15895, 15896, 15897, 15899, 15900, 15901, 15902, 15904, 15905, 15906, 15907, 15908, 15909, 15910, 15912, 15913, 15914, 15915, 15917, 15918, 15919, 15920, 15922, 15925, 15926, 15927, 15928, 15929, 15930, 15931, 15932, 15933, 15934, 15935, 15936, 15937, 15939, 15942, 15944, 15945, 15946, 15948, 15949, 15951, 15953, 15954, 15955, 15957, 15958, 15959, 15961, 15962, 15964, 15965, 15966, 15967, 15968, 15969, 15970, 15972, 15973, 15975, 15978, 15980, 15981, 15982, 15984, 15985, 15987, 15988, 15989, 15990, 15991, 15992, 15994, 15995, 15996, 15997, 15998, 15999, 16000, 16001, 16002, 16003, 16005, 16007, 16008, 16009, 16010, 16012, 16014, 16015, 16017, 16020, 16021, 16022, 16023, 16024, 16025, 16027, 16028, 16029, 16030, 16031, 16033, 16035, 16038, 16039, 16040, 16041, 16043, 16044, 16047, 16048, 16049, 16050, 16051, 16054, 16055, 16058, 16059, 16060, 16061, 16062, 16063, 16064, 16065, 16066, 16069, 16070, 16073, 16074, 16075, 16076, 16077, 16081, 16082, 16084, 16086, 16088, 16089, 16090, 16091, 16092, 16093, 16094, 16095, 16096, 16098, 16099, 16100, 16101, 16103, 16105, 16106, 16108, 16109, 16112, 16113, 16114, 16116, 16117, 16118, 16119, 16120, 16121, 16122, 16123, 16126, 16127, 16128, 16129, 16130, 16132, 16133, 16134, 16135, 16137, 16138, 16141, 16142, 16143, 16148, 16149, 16150, 16151, 16152, 16154, 16155, 16156, 16157, 16158, 16159, 16160, 16163, 16165, 16166, 16170, 16171, 16173, 16174, 16175, 16176, 16178, 16181, 16182, 16183, 16184, 16185, 16188, 16190, 16193, 16194, 16195, 16196, 16197, 16198, 16199, 16201, 16202, 16203, 16205, 16206, 16207, 16209, 16210, 16214, 16215, 16216, 16218, 16219, 16220, 16221, 16222, 16223, 16224, 16226, 16228, 16229, 16230, 16231, 16232, 16235, 16236, 16237, 16238, 16239, 16240, 16241, 16242, 16243, 16244, 16246, 16248, 16250, 16251, 16252, 16253, 16254, 16256, 16257, 16258, 16259, 16262, 16264, 16265, 16266, 16268, 16270, 16271, 16272, 16273, 16275, 16276, 16278, 16279, 16280, 16281, 16283, 16284, 16285, 16286, 16287, 16288, 16289, 16290, 16291, 16292, 16293, 16296, 16297, 16299, 16300, 16301, 16304, 16305, 16306, 16309, 16310, 16312, 16314, 16316, 16317, 16319, 16320, 16321, 16322, 16323, 16324, 16325, 16326, 16329, 16330, 16331, 16332, 16333, 16335, 16337, 16338, 16339, 16341, 16342, 16343, 16345, 16346, 16347, 16349, 16351, 16353, 16355, 16358, 16359, 16363, 16364, 16366, 16369, 16370, 16371, 16372, 16373, 16376, 16377, 16378, 16379, 16383, 16384, 16385, 16386, 16387, 16388, 16389, 16390, 16391, 16392, 16395, 16396, 16398, 16399, 16400, 16401, 16402, 16403, 16405, 16406, 16407, 16408, 16409, 16410, 16411, 16412, 16413, 16414, 16415, 16416, 16418, 16419, 16421, 16422, 16423, 16425, 16426, 16427, 16429, 16431, 16432, 16434, 16435, 16437, 16438, 16440, 16442, 16443, 16444, 16445, 16446, 16449, 16450, 16451, 16453, 16454, 16457, 16458, 16459, 16460, 16461, 16462, 16463, 16464, 16465, 16466, 16468, 16469, 16470, 16471, 16473, 16474, 16475, 16476, 16477, 16478, 16479, 16480, 16481, 16482, 16483, 16484, 16485, 16486, 16487, 16489, 16491, 16496, 16497, 16498, 16499, 16500, 16501, 16505, 16506, 16507, 16508, 16509, 16514, 16515, 16518, 16519, 16521, 16522, 16524, 16525, 16526, 16527, 16529, 16530, 16532, 16533, 16535, 16536, 16539, 16540, 16541, 16545, 16546, 16547, 16548, 16549, 16552, 16553, 16554, 16555, 16556, 16558, 16559, 16561, 16562, 16563, 16564, 16565, 16566, 16567, 16568, 16569, 16573, 16574, 16575, 16577, 16578, 16581, 16582, 16583, 16584, 16586, 16587, 16588, 16589, 16590, 16591, 16592, 16593, 16595, 16596, 16597, 16598, 16599, 16600, 16601, 16602, 16604, 16605, 16607, 16610, 16612, 16613, 16614, 16615, 16616, 16617, 16618, 16619, 16620, 16621, 16626, 16627, 16628, 16630, 16632, 16634, 16636, 16638, 16639, 16640, 16641, 16644, 16646, 16647, 16648, 16651, 16652, 16653, 16654, 16656, 16657, 16658, 16660, 16661, 16662, 16664, 16665, 16666, 16667, 16668, 16669, 16670, 16671, 16672, 16673, 16674, 16675, 16676, 16677, 16678, 16680, 16682, 16684, 16685, 16686, 16687, 16688, 16689, 16691, 16692, 16694, 16696, 16697, 16699, 16700, 16701, 16704, 16705, 16706, 16707, 16708, 16709, 16712, 16713, 16714, 16717, 16719, 16721, 16723, 16725, 16726, 16728, 16729, 16730, 16732, 16733, 16734, 16735, 16736, 16737, 16738, 16739, 16740, 16741, 16743, 16744, 16745, 16746, 16747, 16748, 16749, 16750, 16751, 16752, 16754, 16755, 16756, 16757, 16758, 16759, 16760, 16761, 16764, 16765, 16766, 16768, 16769, 16770, 16771, 16772, 16773, 16774, 16775, 16776, 16777, 16778, 16780, 16781, 16784, 16785, 16787, 16788, 16789, 16792, 16793, 16794, 16795, 16797, 16798, 16799, 16800, 16801, 16803, 16804, 16805, 16806, 16807, 16810, 16811, 16812, 16813, 16814, 16816, 16818, 16819, 16820, 16821, 16823, 16824, 16825, 16826, 16827, 16828, 16829, 16830, 16831, 16832, 16833, 16834, 16835, 16836, 16838, 16839, 16840, 16842, 16843, 16844, 16845, 16848, 16849, 16850, 16852, 16853, 16854, 16856, 16858, 16860, 16861, 16862, 16864, 16865, 16866, 16869, 16870, 16872, 16873, 16874, 16875, 16877, 16878, 16879, 16880, 16881, 16882, 16884, 16885, 16886, 16887, 16889, 16890, 16891, 16892, 16893, 16895, 16896, 16897, 16898, 16900, 16901, 16902, 16903, 16906, 16907, 16908, 16909, 16910, 16911, 16913, 16914, 16916, 16917, 16918, 16919, 16921, 16922, 16923, 16924, 16925, 16926, 16928, 16930, 16932, 16933, 16934, 16936, 16938, 16941, 16942, 16946, 16947, 16948, 16949, 16950, 16951, 16952, 16953, 16954, 16955, 16957, 16958, 16959, 16960, 16961, 16962, 16965, 16966, 16967, 16968, 16969, 16971, 16972, 16974, 16976, 16977, 16978, 16979, 16980, 16981, 16982, 16983, 16985, 16987, 16988, 16989, 16991, 16992, 16994, 16995, 16996, 16997, 16998, 16999, 17000, 17001, 17002, 17003, 17004, 17005, 17006, 17007, 17008, 17010, 17014, 17015, 17017, 17018, 17019, 17020, 17021, 17024, 17025, 17027, 17028, 17030, 17031, 17033, 17034, 17035, 17036, 17037, 17038, 17039, 17040, 17041, 17042, 17043, 17044, 17046, 17047, 17048, 17049, 17050, 17052, 17054, 17058, 17059, 17060, 17061, 17063, 17064, 17065, 17066, 17067, 17068, 17069, 17070, 17071, 17073, 17074, 17075, 17077, 17078, 17080, 17083, 17084, 17085, 17086, 17088, 17090, 17091, 17092, 17093, 17094, 17096, 17097, 17098, 17099, 17100, 17102, 17104, 17105, 17106, 17107, 17109, 17110, 17112, 17113, 17114, 17115, 17117, 17118, 17119, 17120, 17121, 17122, 17123, 17126, 17127, 17128, 17130, 17134, 17137, 17138, 17139, 17140, 17141, 17142, 17144, 17145, 17146, 17148, 17151, 17154, 17155, 17158, 17159, 17160, 17161, 17163, 17166, 17167, 17168, 17169, 17170, 17171, 17172, 17173, 17174, 17175, 17177, 17179, 17180, 17181, 17182, 17184, 17186, 17187, 17189, 17190, 17191, 17192, 17193, 17197, 17198, 17199, 17200, 17202, 17203, 17204, 17206, 17209, 17210, 17211, 17212, 17213, 17215, 17217, 17218, 17219, 17221, 17222, 17223, 17224, 17225, 17227, 17228, 17229, 17230, 17232, 17233, 17234, 17235, 17236, 17237, 17239, 17241, 17242, 17243, 17244, 17245, 17246, 17247, 17248, 17250, 17251, 17252, 17254, 17255, 17256, 17257, 17258, 17259, 17262, 17267, 17268, 17269, 17270, 17272, 17273, 17275, 17276, 17277, 17278, 17279, 17280, 17281, 17282, 17283, 17284, 17285, 17286, 17289, 17291, 17292, 17293, 17295, 17297, 17298, 17299, 17300, 17301, 17302, 17303, 17304, 17306, 17307, 17308, 17309, 17310, 17313, 17314, 17315, 17317, 17318, 17319, 17320, 17322, 17323, 17324, 17325, 17327, 17329, 17330, 17331, 17332, 17333, 17334, 17337, 17338, 17339, 17341, 17342, 17343, 17344, 17345, 17346, 17347, 17348, 17350, 17351, 17352, 17353, 17357, 17358, 17360, 17361, 17362, 17367, 17368, 17369, 17370, 17371, 17373, 17374, 17376, 17377, 17378, 17380, 17381, 17384, 17385, 17388, 17389, 17390, 17391, 17392, 17393, 17394, 17395, 17396, 17399, 17400, 17401, 17402, 17403, 17404, 17405, 17406, 17408, 17409, 17410, 17412, 17413, 17414, 17417, 17418, 17419, 17420, 17423, 17424, 17425, 17428, 17429, 17431, 17432, 17433, 17435, 17438, 17439, 17440, 17441, 17442, 17443, 17444, 17445, 17446, 17447, 17448, 17450, 17451, 17453, 17455, 17456, 17459, 17462, 17463, 17464, 17466, 17468, 17469, 17471, 17472, 17473, 17474, 17475, 17476, 17477, 17478, 17480, 17482, 17483, 17484, 17486, 17487, 17491, 17493, 17494, 17495, 17496, 17497, 17498, 17499, 17500, 17501, 17504, 17506, 17508, 17510, 17511, 17513, 17515, 17519, 17520, 17522, 17523, 17524, 17525, 17526, 17529, 17530, 17532, 17533, 17534, 17535, 17536, 17537, 17538, 17539, 17540, 17541, 17542, 17543, 17544, 17546, 17547, 17548, 17550, 17551, 17552, 17553, 17556, 17557, 17558, 17560, 17561, 17562, 17563, 17564, 17565, 17566, 17567, 17568, 17570, 17571, 17573, 17574, 17575, 17577, 17578, 17579, 17580, 17581, 17582, 17583, 17584, 17587, 17588, 17589, 17590, 17591, 17593, 17594, 17596, 17598, 17599, 17600, 17601, 17602, 17605, 17606, 17608, 17612, 17614, 17616, 17617, 17618, 17619, 17622, 17623, 17624, 17625, 17626, 17627, 17631, 17633, 17634, 17635, 17636, 17637, 17638, 17639, 17640, 17641, 17642, 17643, 17644, 17645, 17646, 17648, 17650, 17651, 17652, 17654, 17655, 17656, 17657, 17658, 17660, 17662, 17663, 17666, 17667, 17668, 17669, 17670, 17671, 17673, 17674, 17675, 17676, 17677, 17678, 17680, 17681, 17683, 17687, 17691, 17692, 17693, 17694, 17695, 17696, 17697, 17698, 17699, 17700, 17701, 17702, 17703, 17704, 17705, 17706, 17707, 17708, 17709, 17710, 17711, 17712, 17715, 17717, 17718, 17719, 17720, 17721, 17722, 17723, 17724, 17725, 17726, 17727, 17728, 17730, 17732, 17733, 17734, 17735, 17736, 17738, 17740, 17742, 17743, 17745, 17746, 17747, 17748, 17749, 17751, 17752, 17753, 17755, 17756, 17758, 17759, 17760, 17761, 17762, 17763, 17767, 17768, 17770, 17771, 17772, 17773, 17775, 17776, 17777, 17778, 17780, 17782, 17783, 17784, 17785, 17789, 17791, 17793, 17794, 17795, 17800, 17801, 17802, 17803, 17804, 17805, 17807, 17808, 17809, 17810, 17812, 17813, 17814, 17815, 17816, 17817, 17820, 17822, 17823, 17824, 17826, 17827, 17828, 17829, 17833, 17834, 17835, 17836, 17839, 17841, 17842, 17844, 17845, 17846, 17847, 17848, 17849, 17851, 17853, 17854, 17855, 17860, 17861, 17865, 17866, 17867, 17868, 17870, 17871, 17872, 17873, 17874, 17875, 17877, 17879, 17880, 17883, 17884, 17885, 17887, 17889, 17890, 17891, 17893, 17894, 17895, 17896, 17899, 17900, 17901, 17902, 17903, 17906, 17907, 17908, 17909, 17910, 17911, 17912, 17913, 17914, 17917, 17919, 17920, 17921, 17924, 17926, 17927, 17928, 17933, 17934, 17936, 17941, 17942, 17943, 17944, 17945, 17946, 17949, 17950, 17951, 17953, 17954, 17955, 17956, 17957, 17959, 17960, 17962, 17963, 17964, 17965, 17967, 17968, 17971, 17972, 17973, 17975, 17977, 17980, 17981, 17982, 17983, 17984, 17985, 17986, 17988, 17989, 17990, 17991, 17992, 17993, 17994, 17995, 17996, 17997, 17998, 17999, 18000, 18001, 18002, 18003, 18004, 18005, 18007, 18008, 18010, 18011, 18013, 18016, 18018, 18019, 18021, 18022, 18023, 18024, 18025, 18028, 18029, 18030, 18032, 18033, 18034, 18036, 18037, 18038, 18039, 18041, 18042, 18043, 18044, 18045, 18046, 18047, 18048, 18049, 18050, 18051, 18052, 18053, 18054, 18055, 18057, 18058, 18059, 18060, 18061, 18062, 18063, 18064, 18065, 18066, 18067, 18069, 18071, 18073, 18074, 18075, 18076, 18079, 18080, 18082, 18083, 18086, 18088, 18089, 18090, 18092, 18093, 18094, 18095, 18097, 18098, 18099, 18100, 18101, 18104, 18106, 18111, 18112, 18113, 18114, 18116, 18117, 18118, 18120, 18122, 18123, 18124, 18125, 18127, 18128, 18129, 18131, 18132, 18133, 18134, 18137, 18138, 18139, 18141, 18143, 18144, 18149, 18150, 18152, 18153, 18154, 18155, 18157, 18158, 18160, 18161, 18163, 18164, 18165, 18166, 18167, 18168, 18169, 18170, 18171, 18172, 18173, 18174, 18175, 18177, 18179, 18180, 18182, 18183, 18185, 18186, 18187, 18188, 18189, 18191, 18192, 18194, 18196, 18197, 18198, 18199, 18200, 18201, 18202, 18203, 18205, 18206, 18207, 18208, 18209, 18210, 18211, 18212, 18213, 18215, 18216, 18217, 18218, 18219, 18220, 18222, 18223, 18225, 18227, 18228, 18231, 18232, 18233, 18234, 18235, 18236, 18240, 18242, 18243, 18246, 18247, 18250, 18251, 18252, 18255, 18256, 18258, 18260, 18264, 18265, 18266, 18267, 18269, 18270, 18271, 18272, 18273, 18274, 18275, 18276, 18277, 18279, 18282, 18283, 18285, 18288, 18290, 18292, 18294, 18295, 18296, 18297, 18299, 18302, 18303, 18305, 18306, 18308, 18309, 18310, 18313, 18314, 18315, 18316, 18320, 18321, 18323, 18324, 18326, 18327, 18328, 18329, 18330, 18332, 18333, 18334, 18335, 18336, 18337, 18339, 18340, 18343, 18344, 18345, 18347, 18348, 18349, 18350, 18351, 18352, 18353, 18354, 18355, 18357, 18358, 18359, 18360, 18362, 18363, 18364, 18365, 18367, 18368, 18370, 18371, 18373, 18374, 18376, 18377, 18378, 18379, 18380, 18381, 18382, 18383, 18387, 18389, 18390, 18391, 18392, 18393, 18394, 18395, 18396, 18397, 18398, 18399, 18401, 18403, 18405, 18408, 18409, 18410, 18411, 18412, 18413, 18414, 18416, 18417, 18419, 18420, 18421, 18422, 18424, 18425, 18426, 18427, 18428, 18430, 18431, 18432, 18433, 18434, 18436, 18437, 18438, 18439, 18443, 18445, 18446, 18447, 18448, 18449, 18450, 18451, 18453, 18454, 18457, 18460, 18461, 18463, 18465, 18466, 18467, 18469, 18470, 18472, 18473, 18475, 18476, 18477, 18478, 18480, 18481, 18482, 18483, 18485, 18486, 18488, 18489, 18490, 18491, 18492, 18493, 18494, 18498, 18499, 18500, 18502, 18503, 18504, 18505, 18508, 18511, 18512, 18514, 18515, 18516, 18517, 18518, 18519, 18520, 18521, 18522, 18523, 18524, 18528, 18529, 18531, 18533, 18534, 18536, 18537, 18538, 18540, 18541, 18542, 18543, 18544, 18545, 18548, 18549, 18550, 18552, 18553, 18555, 18556, 18557, 18558, 18559, 18560, 18566, 18567, 18568, 18572, 18573, 18575, 18576, 18579, 18580, 18581, 18582, 18583, 18584, 18585, 18586, 18587, 18588, 18592, 18594, 18595, 18598, 18601, 18602, 18603, 18604, 18605, 18606, 18607, 18608, 18609, 18611, 18614, 18615, 18616, 18617, 18618, 18619, 18620, 18623, 18624, 18625, 18629, 18630, 18631, 18632, 18634, 18635, 18636, 18638, 18640, 18641, 18642, 18644, 18645, 18646, 18649, 18650, 18651, 18652, 18653, 18654, 18655, 18657, 18658, 18660, 18661, 18662, 18664, 18665, 18666, 18667, 18668, 18669, 18670, 18671, 18674, 18675, 18676, 18677, 18678, 18679, 18680, 18681, 18682, 18683, 18684, 18685, 18687, 18688, 18689, 18690, 18692, 18696, 18698, 18699, 18700, 18702, 18703, 18704, 18706, 18707, 18709, 18712, 18713, 18714, 18715, 18716, 18719, 18722, 18724, 18725, 18726, 18727, 18728, 18729, 18730, 18731, 18732, 18733, 18736, 18737, 18739, 18741, 18742, 18743, 18744, 18747, 18748, 18752, 18753, 18754, 18755, 18756, 18758, 18759, 18760, 18763, 18764, 18768, 18769, 18771, 18772, 18773, 18775, 18776, 18777, 18778, 18781, 18782, 18783, 18784, 18785, 18786, 18787, 18788, 18789, 18790, 18792, 18793, 18794, 18795, 18796, 18800, 18803, 18805, 18806, 18807, 18808, 18809, 18810, 18812, 18813, 18816, 18817, 18821, 18822, 18823, 18824, 18825, 18826, 18827, 18829, 18830, 18832, 18833, 18834, 18836, 18837, 18838, 18839, 18841, 18842, 18843, 18845, 18847, 18848, 18849, 18850, 18851, 18852, 18854, 18856, 18857, 18860, 18861, 18862, 18863, 18865, 18867, 18868, 18869, 18870, 18871, 18873, 18874, 18875, 18877, 18878, 18879, 18880, 18881, 18882, 18883, 18886, 18889, 18891, 18892, 18893, 18894, 18895, 18896, 18897, 18898, 18899, 18901, 18903, 18904, 18905, 18906, 18907, 18908, 18909, 18912, 18915, 18916, 18917, 18920, 18921, 18922, 18923, 18924, 18925, 18926, 18931, 18932, 18936, 18937, 18938, 18939, 18942, 18943, 18944, 18947, 18950, 18951, 18952, 18953, 18954, 18955, 18956, 18957, 18958, 18959, 18960, 18961, 18962, 18963, 18964, 18965, 18966, 18969, 18971, 18972, 18975, 18977, 18983, 18984, 18985, 18986, 18987, 18988, 18989, 18990, 18991, 18994, 18996, 18997, 18998, 18999, 19000, 19002, 19007, 19008, 19009, 19010, 19011, 19012, 19013, 19014, 19015, 19016, 19017, 19018, 19020, 19021, 19022, 19024, 19025, 19026, 19027, 19028, 19029, 19031, 19032, 19033, 19034, 19035, 19036, 19037, 19040, 19041, 19046, 19047, 19048, 19049, 19050, 19051, 19052, 19053, 19054, 19056, 19059, 19060, 19061, 19062, 19063, 19064, 19066, 19067, 19068, 19069, 19070, 19071, 19072, 19073, 19077, 19078, 19079, 19081, 19082, 19083, 19084, 19086, 19087, 19088, 19090, 19091, 19092, 19094, 19095, 19096, 19097, 19098, 19099, 19100, 19102, 19104, 19105, 19106, 19107, 19109, 19110, 19111, 19115, 19116, 19118, 19119, 19121, 19122, 19123, 19124, 19125, 19126, 19127, 19129, 19133, 19134, 19135, 19136, 19137, 19138, 19139, 19140, 19141, 19143, 19144, 19146, 19148, 19149, 19150, 19151, 19152, 19154, 19155, 19156, 19161, 19162, 19164, 19165, 19167, 19171, 19172, 19175, 19176, 19177, 19179, 19182, 19184, 19185, 19186, 19187, 19188, 19189, 19191, 19195, 19196, 19197, 19198, 19199, 19200, 19202, 19203, 19204, 19205, 19206, 19208, 19209, 19211, 19212, 19213, 19214, 19215, 19216, 19218, 19219, 19221, 19222, 19223, 19224, 19225, 19228, 19229, 19231, 19232, 19233, 19236, 19237, 19238, 19239, 19240, 19241, 19242, 19243, 19245, 19246, 19247, 19248, 19249, 19250, 19251, 19252, 19253, 19254, 19255, 19256, 19258, 19259, 19261, 19262, 19264, 19265, 19266, 19268, 19269, 19272, 19273, 19274, 19275, 19277, 19278, 19280, 19281, 19282, 19283, 19285, 19286, 19287, 19289, 19291, 19292, 19293, 19295, 19297, 19298, 19300, 19301, 19302, 19307, 19309, 19310, 19312, 19313, 19314, 19315, 19316, 19318, 19320, 19321, 19322, 19323, 19324, 19325, 19326, 19327, 19328, 19329, 19330, 19331, 19333, 19334, 19335, 19336, 19338, 19340, 19341, 19343, 19344, 19345, 19346, 19347, 19348, 19349, 19350, 19352, 19353, 19354, 19355, 19357, 19358, 19359, 19360, 19361, 19362, 19363, 19364, 19365, 19366, 19367, 19368, 19369, 19370, 19372, 19373, 19377, 19380, 19381, 19382, 19383, 19385, 19387, 19388, 19389, 19390, 19391, 19393, 19395, 19396, 19397, 19398, 19399, 19400, 19401, 19402, 19403, 19404, 19405, 19406, 19407, 19410, 19411, 19412, 19414, 19415, 19416, 19417, 19418, 19419, 19421, 19422, 19424, 19425, 19426, 19427, 19428, 19429, 19430, 19431, 19432, 19434, 19435, 19436, 19438, 19439, 19440, 19442, 19445, 19446, 19447, 19448, 19449, 19450, 19451, 19452, 19453, 19454, 19456, 19458, 19459, 19460, 19462, 19463, 19464, 19465, 19467, 19468, 19470, 19471, 19472, 19473, 19474, 19476, 19477, 19478, 19480, 19482, 19483, 19485, 19487, 19488, 19489, 19490, 19491, 19492, 19493, 19494, 19495, 19496, 19497, 19500, 19502, 19503, 19504, 19505, 19506, 19509, 19510, 19513, 19514, 19515, 19517, 19518, 19519, 19520, 19521, 19522, 19523, 19524, 19526, 19528, 19529, 19530, 19531, 19532, 19533, 19534, 19537, 19539, 19540, 19541, 19542, 19543, 19545, 19546, 19547, 19548, 19549, 19550, 19552, 19553, 19554, 19557, 19559, 19561, 19562, 19564, 19565, 19566, 19567, 19568, 19569, 19570, 19571, 19572, 19574, 19575, 19576, 19577, 19578, 19579, 19581, 19582, 19583, 19584, 19585, 19586, 19588, 19589, 19590, 19591, 19592, 19595, 19596, 19597, 19598, 19600, 19602, 19603, 19606, 19607, 19608, 19612, 19613, 19619, 19620, 19621, 19622, 19623, 19624, 19625, 19626, 19627, 19629, 19630, 19631, 19634, 19635, 19636, 19637, 19638, 19643, 19644, 19645, 19647, 19648, 19649, 19650, 19651, 19652, 19654, 19655, 19656, 19657, 19658, 19660, 19664, 19666, 19667, 19668, 19669, 19670, 19672, 19673, 19674, 19675, 19677, 19678, 19679, 19680, 19681, 19682, 19683, 19684, 19685, 19686, 19687, 19688, 19689, 19690, 19692, 19693, 19694, 19696, 19697, 19698, 19699, 19700, 19701, 19702, 19703, 19704, 19707, 19710, 19712, 19713, 19715, 19717, 19718, 19721, 19722, 19723, 19724, 19725, 19727, 19728, 19729, 19731, 19732, 19733, 19734, 19737, 19740, 19741, 19742, 19743, 19744, 19745, 19746, 19748, 19749, 19750, 19752, 19753, 19754, 19757, 19758, 19759, 19760, 19761, 19763, 19764, 19765, 19766, 19768, 19770, 19772, 19773, 19774, 19775, 19777, 19778, 19779, 19781, 19782, 19783, 19784, 19785, 19786, 19789, 19790, 19791, 19792, 19794, 19795, 19796, 19799, 19800, 19801, 19804, 19805, 19806, 19807, 19808, 19810, 19813, 19815, 19816, 19817, 19818, 19819, 19820, 19821, 19824, 19827, 19828, 19830, 19831, 19834, 19835, 19837, 19839, 19840, 19841, 19842, 19845, 19846, 19847, 19848, 19850, 19851, 19853, 19854, 19855, 19856, 19857, 19859, 19860, 19861, 19862, 19865, 19866, 19868, 19870, 19871, 19872, 19874, 19875, 19881, 19882, 19883, 19884, 19886, 19887, 19889, 19890, 19891, 19892, 19894, 19895, 19896, 19897, 19898, 19899, 19901, 19902, 19903, 19904, 19905, 19906, 19907, 19908, 19909, 19910, 19912, 19913, 19916, 19917, 19919, 19920, 19922, 19923, 19925, 19929, 19930, 19931, 19932, 19933, 19934, 19935, 19936, 19937, 19939, 19941, 19942, 19944, 19945, 19946, 19947, 19948, 19949, 19950, 19951, 19952, 19953, 19955, 19956, 19957, 19958, 19962, 19964, 19965, 19966, 19967, 19968, 19969, 19971, 19972, 19973, 19974, 19975, 19977, 19978, 19980, 19981, 19983, 19984, 19985, 19986, 19987, 19989, 19992, 19993, 19994, 19995, 19998, 19999, 20000, 20002, 20003, 20005, 20006, 20007, 20009, 20011, 20013, 20015, 20016, 20017, 20018, 20019, 20021, 20022, 20023, 20024, 20026, 20027, 20028, 20029, 20030, 20031, 20032, 20033, 20034, 20036, 20038, 20039, 20041, 20042, 20043, 20045, 20047, 20048, 20053, 20054, 20056, 20057, 20058, 20059, 20060, 20062, 20064, 20065, 20066, 20067, 20068, 20069, 20070, 20073, 20076, 20077, 20079, 20081, 20083, 20084, 20086, 20087, 20088, 20089, 20090, 20093, 20094, 20095, 20096, 20097, 20099, 20100, 20101, 20102, 20103, 20106, 20107, 20108, 20109, 20110, 20111, 20113, 20114, 20115, 20117, 20118, 20119, 20120, 20122, 20123, 20124, 20125, 20127, 20130, 20135, 20138, 20139, 20141, 20143, 20144, 20145, 20146, 20147, 20148, 20149, 20151, 20152, 20153, 20154, 20155, 20156, 20157, 20159, 20160, 20161, 20162, 20164, 20165, 20166, 20167, 20168, 20170, 20172, 20174, 20175, 20177, 20178, 20179, 20180, 20181, 20182, 20184, 20185, 20187, 20188, 20189, 20190, 20191, 20192, 20193, 20194, 20196, 20197, 20199, 20202, 20203, 20204, 20205, 20206, 20207, 20208, 20209, 20210, 20211, 20212, 20216, 20217, 20218, 20219, 20220, 20221, 20225, 20226, 20227, 20231, 20232, 20233, 20235, 20236, 20237, 20238, 20239, 20241, 20242, 20243, 20245, 20246, 20248, 20250, 20251, 20252, 20253, 20254, 20255, 20256, 20257, 20258, 20259, 20260, 20261, 20262, 20264, 20266, 20268, 20269, 20270, 20271, 20272, 20273, 20276, 20277, 20278, 20279, 20280, 20282, 20283, 20284, 20285, 20286, 20288, 20289, 20292, 20293, 20295, 20297, 20298, 20299, 20301, 20303, 20304, 20305, 20306, 20307, 20308, 20310, 20313, 20314, 20315, 20317, 20318, 20320, 20321, 20322, 20323, 20324, 20325, 20326, 20327, 20328, 20329, 20332, 20333, 20335, 20337, 20338, 20339, 20340, 20341, 20343, 20344, 20345, 20346, 20347, 20348, 20351, 20353, 20354, 20355, 20357, 20358, 20359, 20362, 20363, 20364, 20365, 20366, 20367, 20368, 20369, 20371, 20372, 20373, 20374, 20376, 20380, 20382, 20383, 20384, 20385, 20386, 20387, 20388, 20390, 20391, 20392, 20393, 20394, 20395, 20396, 20400, 20402, 20403, 20404, 20405, 20406, 20407, 20408, 20409, 20410, 20412, 20413, 20414, 20415, 20416, 20418, 20419, 20420, 20421, 20423, 20424, 20425, 20426, 20427, 20428, 20432, 20433, 20435, 20436, 20437, 20440, 20442, 20443, 20444, 20448, 20449, 20452, 20453, 20454, 20455, 20456, 20461, 20465, 20466, 20467, 20468, 20472, 20474, 20475, 20478, 20479, 20482, 20483, 20484, 20485, 20486, 20487, 20488, 20489, 20490, 20491, 20495, 20499, 20500, 20501, 20502, 20503, 20504, 20505, 20506, 20510, 20511, 20512, 20513, 20514, 20515, 20516, 20517, 20518, 20519, 20522, 20523, 20524, 20525, 20526, 20527, 20528, 20530, 20531, 20532, 20534, 20535, 20536, 20537, 20538, 20540, 20541, 20542, 20546, 20548, 20549, 20551, 20552, 20555, 20556, 20557, 20558, 20559, 20561, 20564, 20565, 20567, 20568, 20569, 20570, 20571, 20574, 20575, 20576, 20577, 20578, 20584, 20585, 20586, 20588, 20589, 20590, 20591, 20592, 20594, 20595, 20596, 20597, 20598, 20599, 20600, 20601, 20602, 20604, 20607, 20608, 20610, 20612, 20614, 20619, 20622, 20624, 20625, 20627, 20628, 20629, 20633, 20634, 20635, 20636, 20637, 20638, 20639, 20640, 20641, 20642, 20644, 20645, 20648, 20649, 20651, 20653, 20654, 20656, 20657, 20659, 20661, 20663, 20664, 20667, 20668, 20670, 20671, 20673, 20674, 20675, 20678, 20679, 20680, 20681, 20682, 20683, 20684, 20685, 20686, 20687, 20688, 20689, 20690, 20691, 20692, 20696, 20698, 20699, 20702, 20704, 20705, 20707, 20708, 20709, 20710, 20711, 20712, 20713, 20714, 20715, 20716, 20718, 20719, 20721, 20722, 20723, 20724, 20728, 20729, 20731, 20732, 20734, 20735, 20736, 20737, 20738, 20739, 20740, 20741, 20742, 20743, 20744, 20745, 20746, 20748, 20749, 20750, 20753, 20754, 20755, 20757, 20758, 20759, 20761, 20763, 20764, 20766, 20768, 20773, 20774, 20775, 20776, 20777, 20779, 20780, 20781, 20782, 20783, 20785, 20790, 20791, 20792, 20793, 20796, 20797, 20799, 20800, 20801, 20802, 20805, 20806, 20808, 20809, 20811, 20813, 20815, 20816, 20818, 20819, 20821, 20823, 20825, 20826, 20828, 20830, 20831, 20833, 20834, 20836, 20838, 20840, 20842, 20843, 20844, 20845, 20846, 20847, 20849, 20852, 20853, 20855, 20857, 20858, 20859, 20860, 20862, 20864, 20866, 20867, 20868, 20869, 20871, 20872, 20873, 20874, 20875, 20878, 20879, 20880, 20881, 20882, 20884, 20885, 20886, 20887, 20889, 20892, 20893, 20894, 20895, 20897, 20898, 20899, 20900, 20901, 20902, 20904, 20905, 20906, 20907, 20909, 20911, 20913, 20914, 20915, 20917, 20922, 20923, 20925, 20926, 20927, 20928, 20929, 20930, 20931, 20932, 20933, 20934, 20935, 20936, 20937, 20938, 20939, 20940, 20941, 20942, 20943, 20944, 20945, 20946, 20947, 20950, 20952, 20953, 20956, 20957, 20958, 20959, 20960, 20962, 20963, 20966, 20967, 20969, 20970, 20971, 20972, 20974, 20975, 20976, 20977, 20978, 20979, 20981, 20982, 20983, 20984, 20985, 20986, 20988, 20989, 20990, 20992, 20997, 20998, 20999, 21001, 21003, 21005, 21006, 21011, 21012, 21013, 21014, 21017, 21019, 21020, 21021, 21023, 21024, 21025, 21026, 21027, 21029, 21030, 21031, 21033, 21035, 21037, 21039, 21041, 21042, 21043, 21045, 21046, 21049, 21050, 21051, 21052, 21053, 21055, 21056, 21057, 21058, 21059, 21061, 21062, 21063, 21067, 21069, 21071, 21072, 21075, 21076, 21077, 21078, 21080, 21081, 21082, 21083, 21084, 21086, 21087, 21088, 21089, 21090, 21092, 21093, 21094, 21095, 21098, 21099, 21101, 21102, 21103, 21105, 21107, 21110, 21112, 21114, 21115, 21116, 21117, 21118, 21119, 21120, 21121, 21122, 21123, 21124, 21125, 21126, 21127, 21128, 21129, 21130, 21131, 21134, 21137, 21138, 21139, 21141, 21142, 21143, 21144, 21145, 21146, 21147, 21149, 21150, 21151, 21152, 21153, 21154, 21156, 21157, 21158, 21159, 21161, 21162, 21163, 21165, 21166, 21167, 21168, 21170, 21171, 21172, 21175, 21177, 21178, 21179, 21181, 21182, 21183, 21184, 21185, 21186, 21187, 21188, 21189, 21192, 21193, 21195, 21196, 21200, 21202, 21207, 21208, 21209, 21210, 21212, 21215, 21216, 21218, 21219, 21220, 21222, 21223, 21224, 21225, 21226, 21227, 21228, 21229, 21230, 21232, 21234, 21235, 21236, 21238, 21239, 21240, 21241, 21242, 21244, 21245, 21247, 21249, 21250, 21251, 21252, 21253, 21254, 21255, 21256, 21258, 21260, 21262, 21264, 21265, 21266, 21267, 21269, 21270, 21271, 21272, 21273, 21275, 21280, 21282, 21284, 21285, 21287, 21289, 21290, 21293, 21295, 21298, 21300, 21301, 21303, 21305, 21306, 21307, 21308, 21309, 21310, 21312, 21313, 21314, 21315, 21318, 21319, 21320, 21322, 21323, 21324, 21325, 21326, 21330, 21332, 21334, 21336, 21337, 21338, 21340, 21341, 21342, 21343, 21345, 21346, 21347, 21350, 21351, 21352, 21353, 21354, 21355, 21356, 21362, 21363, 21364, 21365, 21366, 21367, 21369, 21370, 21371, 21372, 21373, 21374, 21375, 21377, 21378, 21381, 21382, 21383, 21384, 21385, 21386, 21387, 21393, 21394, 21395, 21396, 21397, 21398, 21399, 21401, 21402, 21404, 21405, 21406, 21407, 21408, 21410, 21412, 21413, 21414, 21415, 21416, 21417, 21418, 21419, 21420, 21421, 21424, 21425, 21426, 21427, 21428, 21429, 21431, 21432, 21433, 21434, 21435, 21436, 21437, 21439, 21440, 21442, 21443, 21444, 21446, 21448, 21451, 21453, 21454, 21456, 21457, 21459, 21462, 21464, 21466, 21467, 21469, 21470, 21471, 21473, 21474, 21475, 21476, 21479, 21482, 21483, 21486, 21487, 21491, 21494, 21496, 21497, 21500, 21502, 21503, 21504, 21507, 21508, 21509, 21511, 21513, 21514, 21515, 21516, 21517, 21518, 21520, 21521, 21523, 21524, 21526, 21527, 21530, 21531, 21532, 21535, 21536, 21537, 21538, 21540, 21542, 21544, 21546, 21547, 21548, 21549, 21550, 21551, 21552, 21554, 21556, 21557, 21558, 21560, 21561, 21563, 21564, 21565, 21570, 21571, 21572, 21573, 21574, 21575, 21576, 21577, 21578, 21579, 21580, 21582, 21583, 21584, 21586, 21587, 21589, 21590, 21591, 21592, 21593, 21594, 21596, 21598, 21599, 21600, 21603, 21604, 21606, 21608, 21610, 21613, 21614, 21615, 21616, 21617, 21618, 21620, 21621, 21622, 21623, 21624, 21625, 21627, 21629, 21630, 21632, 21633, 21634, 21636, 21637, 21638, 21639, 21640, 21641, 21642, 21643, 21644, 21646, 21648, 21649, 21650, 21651, 21653, 21654, 21655, 21656, 21657, 21658, 21659, 21660, 21662, 21663, 21664, 21667, 21668, 21669, 21671, 21672, 21673, 21674, 21675, 21677, 21678, 21679, 21680, 21681, 21682, 21683, 21684, 21685, 21686, 21687, 21688, 21689, 21690, 21691, 21692, 21693, 21694, 21695, 21696, 21698, 21699, 21700, 21701, 21702, 21703, 21704, 21705, 21707, 21708, 21709, 21710, 21712, 21714, 21715, 21717, 21719, 21720, 21721, 21722, 21724, 21725, 21726, 21728, 21729, 21731, 21733, 21737, 21738, 21741, 21742, 21743, 21744, 21746, 21747, 21748, 21749, 21750, 21751, 21752, 21754, 21756, 21758, 21759, 21763, 21764, 21766, 21767, 21768, 21770, 21771, 21773, 21774, 21775, 21776, 21778, 21779, 21780, 21781, 21783, 21784, 21785, 21786, 21787, 21788, 21789, 21790, 21791, 21792, 21793, 21794, 21795, 21797, 21799, 21800, 21801, 21802, 21803, 21804, 21806, 21807, 21810, 21811, 21812, 21813, 21814, 21815, 21816, 21817, 21818, 21820, 21822, 21823, 21824, 21825, 21827, 21828, 21829, 21831, 21832, 21833, 21835, 21836, 21838, 21839, 21840, 21841, 21843, 21844, 21845, 21846, 21847, 21849, 21850, 21851, 21852, 21856, 21857, 21858, 21859, 21860, 21861, 21863, 21865, 21868, 21869, 21870, 21872, 21873, 21874, 21876, 21877, 21878, 21879, 21881, 21882, 21883, 21885, 21886, 21887, 21888, 21889, 21890, 21892, 21893, 21894, 21895, 21896, 21897, 21898, 21899, 21901, 21902, 21903, 21904, 21905, 21906, 21908, 21909, 21910, 21912, 21914, 21915, 21916, 21917, 21918, 21921, 21922, 21924, 21926, 21927, 21928, 21929, 21930, 21933, 21934, 21936, 21939, 21940, 21941, 21942, 21944, 21945, 21946, 21949, 21950, 21951, 21952, 21953, 21954, 21956, 21957, 21961, 21962, 21968, 21969, 21970, 21972, 21974, 21975, 21977, 21978, 21980, 21981, 21982, 21983, 21984, 21985, 21987, 21988, 21989, 21990, 21991, 21992, 21993, 21994, 21996, 21997, 21998, 21999, 22001, 22002, 22003, 22004, 22006, 22007, 22008, 22009, 22012, 22015, 22017, 22018, 22019, 22020, 22021, 22023, 22026, 22027, 22028, 22029, 22030, 22031, 22034, 22035, 22036, 22037, 22038, 22043, 22044, 22046, 22047, 22048, 22050, 22051, 22052, 22053, 22057, 22059, 22060, 22061, 22062, 22064, 22065, 22067, 22068, 22071, 22072, 22073, 22076, 22077, 22080, 22081, 22083, 22085, 22086, 22087, 22088, 22091, 22093, 22094, 22095, 22096, 22098, 22099, 22100, 22103, 22104, 22105, 22108, 22109, 22110, 22112, 22113, 22115, 22116, 22117, 22118, 22119, 22120, 22121, 22122, 22123, 22127, 22128, 22129, 22130, 22132, 22133, 22134, 22135, 22136, 22138, 22139, 22140, 22142, 22143, 22144, 22145, 22146, 22147, 22148, 22149, 22150, 22151, 22152, 22153, 22154, 22155, 22156, 22157, 22158, 22160, 22161, 22162, 22164, 22165, 22166, 22167, 22168, 22169, 22171, 22176, 22179, 22180, 22181, 22182, 22184, 22186, 22189, 22192, 22193, 22194, 22196, 22197, 22198, 22200, 22201, 22202, 22204, 22206, 22207, 22208, 22209, 22210, 22211, 22215, 22217, 22218, 22219, 22220, 22221, 22223, 22224, 22225, 22226, 22228, 22229, 22230, 22231, 22232, 22233, 22234, 22235, 22236, 22238, 22239, 22240, 22242, 22243, 22244, 22246, 22247, 22249, 22250, 22251, 22252, 22253, 22254, 22255, 22256, 22257, 22260, 22264, 22265, 22266, 22269, 22271, 22272, 22273, 22274, 22275, 22276, 22277, 22278, 22279, 22280, 22281, 22282, 22283, 22284, 22286, 22287, 22289, 22290, 22291, 22292, 22293, 22294, 22295, 22296, 22297, 22298, 22299, 22300, 22301, 22302, 22303, 22305, 22306, 22307, 22311, 22312, 22314, 22315, 22316, 22317, 22318, 22319, 22321, 22322, 22323, 22324, 22326, 22327, 22329, 22330, 22333, 22334, 22335, 22337, 22338, 22340, 22341, 22342, 22343, 22344, 22346, 22347, 22350, 22351, 22353, 22354, 22355, 22356, 22357, 22358, 22360, 22361, 22363, 22364, 22367, 22368, 22370, 22371, 22372, 22373, 22375, 22376, 22377, 22378, 22379, 22380, 22382, 22383, 22384, 22387, 22388, 22389, 22391, 22392, 22393, 22394, 22395, 22396, 22398, 22399, 22400, 22401, 22402, 22403, 22404, 22405, 22406, 22407, 22408, 22409, 22410, 22411, 22412, 22413, 22414, 22416, 22417, 22418, 22419, 22420, 22421, 22422, 22423, 22424, 22426, 22427, 22428, 22429, 22431, 22432, 22433, 22434, 22436, 22437, 22439, 22440, 22441, 22443, 22444, 22445, 22446, 22448, 22449, 22450, 22451, 22452, 22454, 22455, 22457, 22459, 22461, 22462, 22464, 22465, 22468, 22469, 22470, 22471, 22472, 22473, 22474, 22475, 22476, 22478, 22479, 22480, 22481, 22484, 22485, 22486, 22491, 22492, 22493, 22494, 22495, 22496, 22497, 22498, 22499, 22500, 22501, 22502, 22503, 22504, 22505, 22506, 22507, 22508, 22509, 22510, 22511, 22512, 22513, 22514, 22516, 22518, 22519, 22520, 22521, 22524, 22526, 22527, 22528, 22529, 22530, 22532, 22533, 22534, 22535, 22536, 22537, 22538, 22539, 22541, 22542, 22544, 22546, 22547, 22549, 22550, 22551, 22552, 22554, 22555, 22557, 22558, 22559, 22560, 22561, 22562, 22563, 22564, 22566, 22567, 22569, 22570, 22571, 22572, 22573, 22574, 22575, 22576, 22577, 22578, 22580, 22581, 22582, 22583, 22585, 22586, 22587, 22588, 22590, 22592, 22593, 22594, 22595, 22596, 22597, 22598, 22599, 22600, 22601, 22602, 22605, 22606, 22609, 22610, 22611, 22612, 22614, 22615, 22617, 22618, 22621, 22622, 22623, 22624, 22626, 22627, 22629, 22630, 22631, 22632, 22633, 22635, 22636, 22637, 22638, 22639, 22641, 22642, 22643, 22644, 22645, 22647, 22648, 22649, 22650, 22652, 22654, 22656, 22659, 22660, 22661, 22663, 22664, 22665, 22667, 22669, 22670, 22671, 22672, 22673, 22674, 22675, 22676, 22677, 22678, 22680, 22681, 22682, 22685, 22687, 22688, 22689, 22691, 22692, 22694, 22696, 22700, 22701, 22702, 22703, 22704, 22705, 22706, 22713, 22714, 22716, 22718, 22719, 22720, 22721, 22722, 22723, 22726, 22727, 22728, 22730, 22732, 22733, 22734, 22736, 22738, 22739, 22741, 22742, 22743, 22744, 22745, 22747, 22748, 22749, 22750, 22751, 22752, 22753, 22754, 22755, 22756, 22758, 22759, 22760, 22761, 22762, 22763, 22764, 22765, 22766, 22768, 22769, 22770, 22771, 22773, 22775, 22776, 22778, 22780, 22781, 22782, 22785, 22786, 22789, 22790, 22791, 22793, 22795, 22797, 22798, 22799, 22800, 22801, 22802, 22803, 22804, 22805, 22806, 22807, 22808, 22809, 22811, 22812, 22813, 22814, 22816, 22817, 22818, 22819, 22822, 22823, 22824, 22825, 22828, 22829, 22830, 22831, 22833, 22834, 22837, 22838, 22839, 22841, 22842, 22844, 22845, 22846, 22847, 22848, 22849, 22850, 22852, 22853, 22855, 22856, 22857, 22858, 22859, 22860, 22862, 22864, 22865, 22866, 22867, 22868, 22869, 22870, 22872, 22873, 22874, 22875, 22876, 22877, 22878, 22879, 22881, 22882, 22883, 22885, 22886, 22888, 22889, 22890, 22891, 22895, 22896, 22898, 22899, 22900, 22901, 22903, 22904, 22907, 22911, 22913, 22914, 22915, 22917, 22918, 22919, 22920, 22921, 22922, 22924, 22925, 22927, 22928, 22929, 22931, 22932, 22933, 22934, 22935, 22936, 22937, 22940, 22941, 22942, 22943, 22944, 22945, 22947, 22948, 22949, 22950, 22951, 22952, 22953, 22954, 22955, 22956, 22957, 22959, 22960, 22961, 22962, 22963, 22964, 22968, 22969, 22971, 22972, 22973, 22974, 22975, 22976, 22977, 22979, 22982, 22983, 22984, 22985, 22986, 22989, 22991, 22992, 22993, 22994, 22995, 22996, 22997, 22999, 23000, 23002, 23003, 23004, 23006, 23007, 23009, 23010, 23011, 23012, 23013, 23014, 23016, 23017, 23019, 23020, 23022, 23024, 23026, 23027, 23030, 23032, 23033, 23034, 23035, 23036, 23037, 23039, 23040, 23042, 23044, 23046, 23047, 23048, 23052, 23053, 23055, 23056, 23057, 23058, 23059, 23061, 23062, 23064, 23067, 23068, 23069, 23070, 23071, 23072, 23073, 23077, 23080, 23081, 23083, 23084, 23085, 23088, 23090, 23091, 23092, 23093, 23094, 23095, 23096, 23097, 23098, 23100, 23102, 23103, 23104, 23105, 23106, 23107, 23108, 23109, 23110, 23111, 23114, 23117, 23119, 23120, 23121, 23123, 23124, 23126, 23128, 23133, 23134, 23135, 23137, 23139, 23140, 23142, 23145, 23146, 23148, 23149, 23150, 23152, 23153, 23154, 23155, 23156, 23157, 23160, 23162, 23163, 23165, 23167, 23168, 23169, 23170, 23173, 23174, 23175, 23176, 23177, 23179, 23181, 23182, 23183, 23185, 23186, 23187, 23188, 23192, 23193, 23194, 23195, 23196, 23197, 23198, 23199, 23200, 23201, 23202, 23203, 23204, 23205, 23207, 23208, 23210, 23211, 23212, 23214, 23216, 23217, 23219, 23221, 23222, 23223, 23224, 23226, 23228, 23229, 23230, 23231, 23234, 23235, 23236, 23237, 23240, 23241, 23242, 23243, 23244, 23246, 23247, 23248, 23250, 23252, 23254, 23259, 23260, 23261, 23262, 23263, 23266, 23267, 23268, 23271, 23272, 23274, 23276, 23277, 23278, 23279, 23280, 23281, 23282, 23283, 23287, 23288, 23289, 23290, 23293, 23294, 23295, 23296, 23298, 23299, 23300, 23301, 23304, 23308, 23310, 23311, 23312, 23313, 23314, 23316, 23317, 23318, 23319, 23321, 23322, 23324, 23325, 23327, 23328, 23329, 23330, 23332, 23333, 23334, 23335, 23336, 23337, 23341, 23343, 23344, 23345, 23349, 23350, 23352, 23355, 23356, 23357, 23358, 23359, 23360, 23361, 23363, 23364, 23365, 23367, 23368, 23370, 23372, 23375, 23376, 23378, 23379, 23381, 23382, 23383, 23386, 23387, 23389, 23391, 23392, 23393, 23394, 23395, 23396, 23397, 23398, 23399, 23400, 23402, 23403, 23404, 23406, 23407, 23408, 23409, 23410, 23412, 23413, 23414, 23415, 23417, 23419, 23420, 23421, 23422, 23428, 23429, 23431, 23432, 23435, 23437, 23438, 23439, 23440, 23441, 23444, 23446, 23447, 23448, 23449, 23450, 23453, 23456, 23457, 23458, 23459, 23460, 23462, 23463, 23464, 23467, 23468, 23469, 23473, 23474, 23476, 23477, 23478, 23480, 23486, 23490, 23491, 23492, 23493, 23495, 23496, 23497, 23498, 23499, 23500, 23501, 23502, 23503, 23504, 23506, 23507, 23508, 23509, 23510, 23511, 23512, 23513, 23514, 23516, 23517, 23518, 23519, 23520, 23521, 23522, 23523, 23524, 23525, 23526, 23529, 23530, 23533, 23534, 23535, 23536, 23537, 23544, 23545, 23546, 23547, 23548, 23551, 23552, 23553, 23554, 23555, 23556, 23557, 23558, 23559, 23560, 23561, 23563, 23564, 23566, 23567, 23568, 23570, 23571, 23572, 23574, 23577, 23578, 23579, 23581, 23582, 23583, 23584, 23585, 23586, 23587, 23588, 23591, 23592, 23593, 23595, 23596, 23599, 23600, 23603, 23604, 23605, 23609, 23610, 23611, 23612, 23613, 23614, 23615, 23620, 23621, 23622, 23623, 23624, 23625, 23626, 23627, 23629, 23631, 23632, 23633, 23634, 23635, 23636, 23637, 23639, 23647, 23648, 23650, 23653, 23654, 23656, 23657, 23658, 23659, 23660, 23662, 23663, 23666, 23667, 23669, 23670, 23671, 23672, 23673, 23674, 23676, 23677, 23679, 23683, 23684, 23686, 23689, 23690, 23695, 23696, 23697, 23698, 23699, 23701, 23703, 23704, 23705, 23706, 23709, 23710, 23711, 23712, 23713, 23714, 23715, 23716, 23717, 23718, 23719, 23720, 23722, 23724, 23726, 23728, 23729, 23731, 23732, 23733, 23734, 23736, 23737, 23738, 23743, 23744, 23745, 23746, 23748, 23749, 23750, 23752, 23753, 23755, 23757, 23758, 23760, 23761, 23762, 23763, 23764, 23766, 23769, 23771, 23772, 23773, 23774, 23775, 23777, 23778, 23779, 23780, 23781, 23782, 23785, 23787, 23788, 23789, 23790, 23793, 23795, 23796, 23797, 23800, 23802, 23803, 23804, 23807, 23808, 23809, 23811, 23813, 23814, 23815, 23816, 23818, 23819, 23820, 23821, 23822, 23823, 23826, 23827, 23830, 23831, 23832, 23834, 23836, 23837, 23838, 23840, 23841, 23843, 23844, 23845, 23847, 23848, 23849, 23850, 23851, 23854, 23855, 23856, 23857, 23859, 23860, 23861, 23866, 23867, 23868, 23870, 23871, 23876, 23878, 23879, 23880, 23881, 23883, 23884, 23885, 23886, 23887, 23888, 23889, 23890, 23894, 23895, 23896, 23899, 23901, 23903, 23906, 23907, 23908, 23910, 23911, 23912, 23914, 23915, 23916, 23917, 23919, 23920, 23921, 23922, 23923, 23925, 23927, 23928, 23929, 23931, 23932, 23933, 23934, 23935, 23936, 23938, 23940, 23941, 23942, 23943, 23945, 23946, 23947, 23948, 23949, 23950, 23952, 23953, 23954, 23959, 23960, 23961, 23962, 23963, 23965, 23966, 23967, 23969, 23970, 23972, 23976, 23977, 23979, 23981, 23983, 23985, 23986, 23988, 23990, 23992, 23993, 23995, 23997, 24000, 24001, 24003, 24006, 24007, 24008, 24013, 24014, 24015, 24016, 24017, 24018, 24019, 24020, 24022, 24023, 24024, 24025, 24026, 24027, 24028, 24029, 24031, 24033, 24034, 24035, 24036, 24037, 24038, 24039, 24040, 24041, 24045, 24047, 24048, 24050, 24051, 24052, 24053, 24054, 24055, 24056, 24057, 24058, 24059, 24061, 24062, 24063, 24065, 24066, 24067, 24068, 24070, 24071, 24072, 24073, 24075, 24076, 24078, 24079, 24080, 24083, 24084, 24085, 24086, 24087, 24090, 24091, 24092, 24093, 24094, 24098, 24100, 24102, 24104, 24105, 24106, 24107, 24108, 24109, 24110, 24111, 24112, 24113, 24114, 24115, 24116, 24117, 24119, 24121, 24122, 24123, 24125, 24126, 24127, 24128, 24129, 24131, 24132, 24133, 24134, 24136, 24138, 24139, 24140, 24144, 24145, 24147, 24148, 24149, 24150, 24151, 24152, 24153, 24155, 24156, 24158, 24159, 24160, 24161, 24162, 24163, 24164, 24165, 24166, 24167, 24168, 24169, 24171, 24172, 24173, 24174, 24175, 24176, 24177, 24178, 24179, 24182, 24183, 24184, 24186, 24187, 24188, 24189, 24191, 24196, 24197, 24198, 24199, 24200, 24201, 24202, 24203, 24204, 24205, 24206, 24208, 24209, 24210, 24211, 24214, 24215, 24217, 24218, 24220, 24221, 24222, 24223, 24224, 24226, 24227, 24228, 24230, 24231, 24233, 24234, 24235, 24236, 24239, 24242, 24243, 24244, 24246, 24247, 24248, 24251, 24252, 24254, 24255, 24256, 24258, 24259, 24260, 24262, 24263, 24264, 24265, 24266, 24267, 24268, 24271, 24272, 24273, 24274, 24276, 24278, 24280, 24281, 24282, 24284, 24285, 24286, 24287, 24289, 24291, 24292, 24293, 24295, 24296, 24297, 24298, 24299, 24300, 24301, 24303, 24304, 24306, 24310, 24311, 24314, 24317, 24318, 24319, 24325, 24326, 24328, 24330, 24331, 24332, 24333, 24334, 24335, 24337, 24338, 24340, 24341, 24344, 24345, 24346, 24347, 24351, 24352, 24353, 24355, 24356, 24358, 24359, 24360, 24361, 24362, 24363, 24365, 24366, 24367, 24368, 24370, 24372, 24373, 24375, 24376, 24378, 24379, 24380, 24383, 24384, 24386, 24387, 24388, 24390, 24392, 24394, 24395, 24396, 24397, 24398, 24399, 24401, 24403, 24404, 24406, 24407, 24408, 24411, 24412, 24414, 24415, 24417, 24418, 24419, 24420, 24421, 24422, 24423, 24424, 24425, 24428, 24429, 24430, 24431, 24432, 24436, 24438, 24440, 24441, 24444, 24448, 24450, 24451, 24452, 24453, 24454, 24455, 24457, 24460, 24461, 24462, 24463, 24464, 24465, 24466, 24467, 24468, 24469, 24470, 24471, 24473, 24474, 24481, 24482, 24483, 24484, 24485, 24487, 24488, 24489, 24492, 24493, 24494, 24495, 24497, 24498, 24500, 24502, 24503, 24505, 24506, 24507, 24508, 24509, 24510, 24511, 24516, 24517, 24519, 24521, 24523, 24525, 24526, 24528, 24530, 24531, 24533, 24534, 24536, 24537, 24539, 24541, 24542, 24543, 24545, 24546, 24547, 24548, 24551, 24552, 24553, 24554, 24555, 24560, 24562, 24563, 24564, 24565, 24566, 24568, 24569, 24570, 24571, 24572, 24576, 24577, 24578, 24579, 24580, 24581, 24582, 24583, 24585, 24586, 24587, 24589, 24590, 24592, 24593, 24594, 24596, 24597, 24598, 24599, 24600, 24601, 24602, 24603, 24604, 24605, 24606, 24607, 24610, 24614, 24616, 24617, 24618, 24619, 24620, 24621, 24622, 24624, 24625, 24626, 24630, 24631, 24632, 24633, 24637, 24639, 24640, 24641, 24642, 24643, 24644, 24645, 24646, 24647, 24650, 24651, 24652, 24653, 24654, 24655, 24656, 24657, 24658, 24659, 24660, 24661, 24663, 24664, 24665, 24666, 24667, 24668, 24669, 24670, 24671, 24672, 24673, 24676, 24678, 24680, 24681, 24682, 24683, 24685, 24686, 24688, 24689, 24690, 24692, 24693, 24694, 24695, 24696, 24698, 24699, 24700, 24701, 24704, 24705, 24706, 24708, 24709, 24712, 24713, 24714, 24715, 24717, 24718, 24721, 24722, 24724, 24725, 24726, 24727, 24728, 24729, 24730, 24731, 24732, 24734, 24735, 24736, 24737, 24740, 24741, 24744, 24745, 24746, 24747, 24748, 24750, 24751, 24752, 24753, 24754, 24756, 24757, 24759, 24760, 24762, 24763, 24764, 24765, 24767, 24768, 24769, 24770, 24771, 24772, 24773, 24774, 24775, 24776, 24777, 24778, 24779, 24781, 24782, 24783, 24784, 24786, 24788, 24789, 24790, 24792, 24793, 24795, 24797, 24799, 24800, 24801, 24802, 24803, 24804, 24805, 24806, 24807, 24808, 24810, 24811, 24813, 24814, 24815, 24816, 24817, 24819, 24820, 24821, 24822, 24823, 24824, 24825, 24826, 24828, 24830, 24832, 24833, 24834, 24835, 24836, 24838, 24839, 24841, 24842, 24843, 24844, 24846, 24848, 24849, 24850, 24851, 24852, 24855, 24857, 24859, 24861, 24862, 24865, 24866, 24867, 24868, 24870, 24873, 24874, 24875, 24876, 24877, 24878, 24879, 24880, 24881, 24882, 24883, 24884, 24885, 24886, 24887, 24888, 24890, 24891, 24892, 24893, 24894, 24895, 24896, 24897, 24898, 24899, 24900, 24901, 24902, 24904, 24905, 24907, 24908, 24909, 24910, 24911, 24913, 24914, 24915, 24917, 24919, 24920, 24921, 24925, 24927, 24928, 24929, 24930, 24933, 24935, 24939, 24940, 24941, 24942, 24943, 24946, 24947, 24950, 24953, 24954, 24955, 24956, 24958, 24962, 24964, 24965, 24966, 24967, 24968, 24970, 24971, 24973, 24974, 24975, 24977, 24979, 24980, 24981, 24982, 24983, 24984, 24985, 24986, 24987, 24988, 24989, 24990, 24991, 24993, 24994, 24995, 24996, 24997, 24999, 25000, 25001, 25002, 25003, 25004, 25006, 25008, 25009, 25010, 25013, 25015, 25017, 25018, 25019, 25021, 25025, 25027, 25028, 25029, 25030, 25032, 25034, 25035, 25037, 25038, 25039, 25041, 25042, 25043, 25044, 25045, 25046, 25047, 25049, 25050, 25051, 25052, 25053, 25054, 25056, 25057, 25058, 25060, 25062, 25064, 25065, 25066, 25067, 25068, 25069, 25070, 25071, 25072, 25073, 25074, 25076, 25079, 25082, 25083, 25084, 25086, 25087, 25088, 25091, 25092, 25093, 25094, 25095, 25096, 25098, 25100, 25101, 25102, 25103, 25105, 25106, 25108, 25110, 25111, 25114, 25115, 25116, 25117, 25120, 25121, 25122, 25124, 25125, 25126, 25127, 25128, 25129, 25130, 25131, 25133, 25134, 25135, 25136, 25137, 25138, 25140, 25141, 25142, 25143, 25144, 25145, 25146, 25148, 25150, 25151, 25153, 25154, 25155, 25157, 25158, 25162, 25163, 25164, 25165, 25167, 25168, 25169, 25170, 25172, 25173, 25174, 25176, 25177, 25178, 25181, 25182, 25183, 25184, 25185, 25186, 25187, 25189, 25190, 25191, 25192, 25193, 25194, 25195, 25196, 25197, 25198, 25201, 25202, 25203, 25204, 25206, 25207, 25208, 25209, 25210, 25212, 25213, 25214, 25216, 25217, 25219, 25220, 25221, 25223, 25224, 25226, 25227, 25230, 25232, 25233, 25234, 25236, 25237, 25238, 25239, 25241, 25242, 25243, 25244, 25245, 25247, 25248, 25251, 25252, 25254, 25256, 25258, 25259, 25260, 25261, 25262, 25263, 25264, 25265, 25266, 25267, 25270, 25271, 25272, 25273, 25275, 25277, 25279, 25280, 25281, 25282, 25283, 25284, 25285, 25286, 25288, 25290, 25291, 25292, 25293, 25295, 25297, 25298, 25299, 25300, 25301, 25302, 25303, 25304, 25305, 25306, 25308, 25309, 25310, 25311, 25312, 25316, 25317, 25318, 25319, 25320, 25321, 25322, 25324, 25325, 25326, 25328, 25329, 25330, 25331, 25333, 25335, 25337, 25339, 25340, 25341, 25342, 25343, 25344, 25345, 25346, 25347, 25349, 25351, 25352, 25354, 25355, 25357, 25358, 25359, 25360, 25361, 25362, 25363, 25365, 25366, 25367, 25369, 25370, 25371, 25372, 25373, 25374, 25375, 25377, 25378, 25379, 25381, 25382, 25384, 25387, 25388, 25389, 25390, 25392, 25393, 25394, 25395, 25397, 25398, 25399, 25400, 25401, 25402, 25403, 25404, 25406, 25407, 25408, 25409, 25410, 25411, 25413, 25414, 25416, 25418, 25420, 25421, 25424, 25425, 25426, 25427, 25428, 25429, 25430, 25431, 25432, 25433, 25434, 25435, 25436, 25439, 25440, 25441, 25442, 25443, 25444, 25446, 25448, 25449, 25450, 25451, 25452, 25453, 25456, 25457, 25460, 25461, 25463, 25464, 25465, 25466, 25467, 25469, 25470, 25471, 25472, 25473, 25474, 25475, 25477, 25479, 25480, 25481, 25482, 25485, 25486, 25487, 25488, 25489, 25493, 25494, 25495, 25498, 25499, 25500, 25503, 25504, 25506, 25507, 25508, 25509, 25510, 25511, 25514, 25515, 25516, 25517, 25518, 25519, 25522, 25523, 25524, 25529, 25530, 25531, 25532, 25537, 25538, 25540, 25541, 25543, 25544, 25545, 25546, 25549, 25550, 25551, 25553, 25554, 25556, 25557, 25558, 25559, 25560, 25561, 25562, 25564, 25565, 25567, 25568, 25569, 25571, 25572, 25573, 25574, 25575, 25576, 25577, 25578, 25579, 25580, 25581, 25583, 25584, 25585, 25586, 25587, 25588, 25590, 25591, 25593, 25594, 25595, 25596, 25598, 25599, 25600, 25601, 25602, 25603, 25604, 25605, 25606, 25607, 25608, 25610, 25611, 25613, 25615, 25617, 25618, 25619, 25621, 25623, 25624, 25625, 25626, 25627, 25628, 25629, 25630, 25631, 25632, 25633, 25634, 25635, 25636, 25637, 25639, 25641, 25642, 25643, 25644, 25646, 25647, 25649, 25650, 25653, 25654, 25659, 25660, 25661, 25663, 25665, 25666, 25667, 25668, 25669, 25670, 25672, 25673, 25674, 25675, 25676, 25677, 25678, 25679, 25680, 25681, 25683, 25684, 25685, 25686, 25689, 25690, 25691, 25692, 25696, 25698, 25699, 25701, 25703, 25704, 25706, 25707, 25708, 25709, 25710, 25711, 25712, 25714, 25718, 25719, 25721, 25722, 25724, 25725, 25726, 25727, 25728, 25729, 25732, 25733, 25734, 25735, 25736, 25738, 25739, 25740, 25741, 25742, 25743, 25745, 25747, 25748, 25749, 25750, 25751, 25752, 25754, 25755, 25757, 25758, 25760, 25761, 25762, 25763, 25765, 25766, 25769, 25770, 25771, 25772, 25774, 25775, 25777, 25778, 25779, 25780, 25781, 25782, 25783, 25784, 25785, 25786, 25787, 25789, 25793, 25797, 25799, 25800, 25801, 25803, 25804, 25805, 25806, 25807, 25808, 25810, 25812, 25813, 25814, 25815, 25817, 25818, 25819, 25820, 25821, 25822, 25823, 25824, 25825, 25826, 25827, 25828, 25829, 25830, 25832, 25833, 25836, 25837, 25838, 25840, 25842, 25844, 25845, 25846, 25847, 25848, 25849, 25850, 25852, 25853, 25854, 25856, 25857, 25858, 25860, 25861, 25862, 25863, 25864, 25865, 25866, 25867, 25868, 25869, 25872, 25873, 25874, 25875, 25876, 25877, 25880, 25881, 25882, 25883, 25884, 25885, 25887, 25888, 25890, 25891, 25892, 25895, 25896, 25897, 25898, 25901, 25902, 25903, 25905, 25906, 25907, 25908, 25911, 25912, 25914, 25915, 25916, 25917, 25918, 25919, 25920, 25921, 25923, 25924, 25928, 25929, 25930, 25931, 25932, 25933, 25934, 25935, 25936, 25938, 25942, 25943, 25945, 25946, 25947, 25948, 25949, 25950, 25951, 25952, 25953, 25954, 25955, 25958, 25959, 25960, 25961, 25962, 25963, 25965, 25966, 25968, 25969, 25971, 25972, 25973, 25974, 25976, 25977, 25978, 25979, 25980, 25985, 25986, 25988, 25989, 25990, 25991, 25992, 25993, 25996, 25997, 25998, 26000, 26002, 26003, 26004, 26005, 26012, 26013, 26014, 26015, 26016, 26017, 26018, 26019, 26020, 26022, 26023, 26025, 26026, 26027, 26028, 26029, 26030, 26031, 26032, 26033, 26034, 26035, 26036, 26037, 26038, 26040, 26041, 26043, 26044, 26045, 26050, 26053, 26055, 26057, 26058, 26060, 26062, 26063, 26064, 26066, 26067, 26068, 26069, 26071, 26072, 26074, 26075, 26076, 26078, 26079, 26080, 26081, 26082, 26083, 26084, 26085, 26086, 26088, 26091, 26092, 26093, 26094, 26095, 26096, 26097, 26099, 26100, 26101, 26104, 26106, 26107, 26109, 26111, 26112, 26113, 26114, 26117, 26118, 26119, 26120, 26121, 26123, 26126, 26130, 26132, 26133, 26138, 26139, 26142, 26143, 26144, 26146, 26147, 26148, 26149, 26151, 26153, 26154, 26155, 26157, 26158, 26159, 26160, 26161, 26163, 26166, 26167, 26169, 26170, 26171, 26172, 26173, 26174, 26175, 26176, 26178, 26179, 26180, 26181, 26182, 26183, 26184, 26186, 26187, 26189, 26191, 26192, 26195, 26196, 26197, 26199, 26200, 26201, 26202, 26203, 26204, 26205, 26206, 26208, 26209, 26210, 26211, 26213, 26214, 26215, 26216, 26217, 26218, 26220, 26221, 26223, 26224, 26225, 26226, 26227, 26228, 26229, 26230, 26232, 26233, 26234, 26236, 26239, 26241, 26243, 26244, 26245, 26246, 26247, 26249, 26250, 26251, 26252, 26254, 26255, 26256, 26257, 26258, 26259, 26260, 26261, 26262, 26263, 26264, 26265, 26266, 26267, 26268, 26269, 26270, 26272, 26273, 26274, 26276, 26277, 26279, 26281, 26283, 26285, 26287, 26288, 26290, 26291, 26292, 26293, 26294, 26295, 26296, 26297, 26298, 26299, 26300, 26303, 26304, 26305, 26306, 26307, 26308, 26310, 26312, 26313, 26315, 26316, 26317, 26318, 26320, 26322, 26323, 26324, 26327, 26329, 26330, 26331, 26332, 26333, 26334, 26335, 26337, 26338, 26340, 26341, 26342, 26343, 26344, 26345, 26346, 26348, 26349, 26350, 26352, 26353, 26355, 26356, 26357, 26358, 26359, 26360, 26361, 26364, 26365, 26366, 26367, 26368, 26370, 26372, 26373, 26375, 26377, 26379, 26380, 26383, 26384, 26385, 26386, 26389, 26390, 26392, 26394, 26395, 26397, 26400, 26402, 26403, 26404, 26406, 26407, 26408, 26409, 26411, 26412, 26413, 26414, 26415, 26417, 26419, 26420, 26422, 26425, 26426, 26427, 26428, 26429, 26430, 26431, 26432, 26433, 26434, 26436, 26438, 26439, 26440, 26441, 26442, 26443, 26444, 26447, 26448, 26449, 26450, 26451, 26454, 26455, 26456, 26457, 26458, 26459, 26460, 26461, 26463, 26466, 26467, 26468, 26469, 26472, 26475, 26476, 26477, 26478, 26479, 26480, 26481, 26482, 26483, 26484, 26485, 26486, 26487, 26488, 26489, 26490, 26491, 26497, 26498, 26499, 26501, 26503, 26505, 26506, 26507, 26508, 26509, 26510, 26511, 26516, 26517, 26518, 26520, 26522, 26523, 26525, 26526, 26528, 26529, 26531, 26532, 26533, 26534, 26535, 26536, 26538, 26539, 26540, 26541, 26544, 26545, 26546, 26547, 26549, 26551, 26554, 26555, 26556, 26557, 26560, 26561, 26562, 26563, 26564, 26565, 26566, 26571, 26572, 26573, 26574, 26575, 26576, 26577, 26579, 26580, 26581, 26583, 26584, 26585, 26586, 26589, 26590, 26591, 26592, 26594, 26598, 26601, 26602, 26603, 26605, 26606, 26607, 26608, 26611, 26612, 26613, 26614, 26615, 26616, 26619, 26620, 26621, 26622, 26623, 26627, 26628, 26629, 26630, 26632, 26634, 26636, 26637, 26638, 26640, 26641, 26643, 26644, 26645, 26647, 26648, 26649, 26651, 26653, 26654, 26655, 26656, 26657, 26658, 26659, 26660, 26661, 26663, 26664, 26665, 26667, 26669, 26670, 26671, 26672, 26673, 26675, 26676, 26678, 26679, 26680, 26682, 26683, 26684, 26685, 26686, 26687, 26688, 26689, 26690, 26691, 26693, 26694, 26695, 26696, 26697, 26698, 26700, 26701, 26702, 26703, 26706, 26707, 26710, 26715, 26716, 26717, 26718, 26719, 26721, 26722, 26725, 26728, 26730, 26733, 26735, 26737, 26738, 26740, 26742, 26743, 26744, 26745, 26746, 26747, 26749, 26752, 26753, 26754, 26755, 26756, 26757, 26759, 26763, 26764, 26766, 26767, 26768, 26769, 26770, 26771, 26772, 26773, 26774, 26775, 26777, 26778, 26779, 26781, 26782, 26783, 26784, 26785, 26786, 26789, 26790, 26792, 26793, 26794, 26795, 26796, 26797, 26798, 26800, 26801, 26802, 26803, 26804, 26805, 26806, 26807, 26808, 26809, 26810, 26811, 26813, 26814, 26815, 26819, 26821, 26823, 26824, 26825, 26826, 26830, 26831, 26833, 26834, 26835, 26836, 26837, 26838, 26839, 26841, 26842, 26843, 26844, 26845, 26846, 26847, 26848, 26850, 26851, 26852, 26854, 26855, 26856, 26858, 26860, 26861, 26862, 26863, 26864, 26866, 26867, 26868, 26869, 26870, 26872, 26873, 26874, 26875, 26877, 26879, 26880, 26881, 26882, 26885, 26886, 26887, 26888, 26889, 26890, 26892, 26895, 26897, 26898, 26899, 26900, 26902, 26903, 26905, 26906, 26910, 26913, 26914, 26915, 26917, 26918, 26919, 26920, 26921, 26922, 26926, 26928, 26929, 26930, 26931, 26932, 26933, 26934, 26935, 26936, 26937, 26939, 26940, 26941, 26942, 26943, 26944, 26945, 26946, 26948, 26949, 26950, 26951, 26952, 26953, 26954, 26955, 26956, 26957, 26958, 26959, 26960, 26961, 26962, 26963, 26965, 26966, 26968, 26969, 26970, 26971, 26972, 26973, 26974, 26975, 26976, 26977, 26978, 26979, 26980, 26982, 26983, 26986, 26987, 26988, 26989, 26991, 26992, 26993, 26994, 26996, 26998, 27000, 27002, 27004, 27007, 27008, 27009, 27010, 27011, 27012, 27014, 27015, 27017, 27018, 27019, 27020, 27021, 27023, 27025, 27027, 27029, 27030, 27033, 27034, 27035, 27036, 27038, 27041, 27042, 27044, 27046, 27048, 27053, 27054, 27055, 27056, 27057, 27059, 27060, 27061, 27063, 27065, 27066, 27068, 27069, 27070, 27071, 27074, 27075, 27076, 27077, 27079, 27080, 27083, 27084, 27085, 27086, 27087, 27088, 27089, 27090, 27091, 27092, 27093, 27094, 27096, 27097, 27098, 27099, 27101, 27102, 27103, 27104, 27105, 27107, 27109, 27111, 27112, 27114, 27115, 27118, 27119, 27121, 27122, 27126, 27128, 27131, 27132, 27133, 27135, 27137, 27139, 27140, 27142, 27143, 27144, 27145, 27146, 27147, 27148, 27150, 27151, 27152, 27153, 27154, 27155, 27158, 27159, 27160, 27161, 27163, 27166, 27167, 27168, 27172, 27173, 27175, 27177, 27178, 27179, 27180, 27181, 27183, 27186, 27187, 27189, 27191, 27192, 27193, 27194, 27195, 27196, 27198, 27201, 27202, 27203, 27204, 27205, 27206, 27208, 27209, 27210, 27211, 27213, 27216, 27217, 27218, 27219, 27220, 27221, 27223, 27225, 27226, 27227, 27228, 27229, 27231, 27234, 27235, 27236, 27237, 27238, 27239, 27241, 27242, 27243, 27245, 27247, 27248, 27249, 27250, 27253, 27254, 27256, 27257, 27259, 27260, 27262, 27263, 27265, 27267, 27272, 27274, 27275, 27276, 27277, 27278, 27280, 27281, 27283, 27285, 27286, 27290, 27291, 27292, 27293, 27294, 27295, 27297, 27298, 27300, 27301, 27304, 27305, 27306, 27307, 27311, 27312, 27313, 27315, 27316, 27317, 27319, 27322, 27323, 27324, 27325, 27326, 27327, 27328, 27329, 27332, 27333, 27334, 27335, 27336, 27337, 27338, 27339, 27340, 27341, 27343, 27344, 27345, 27346, 27347, 27348, 27350, 27351, 27352, 27353, 27356, 27357, 27358, 27359, 27360, 27361, 27362, 27363, 27364, 27365, 27366, 27367, 27369, 27371, 27373, 27374, 27375, 27376, 27377, 27378, 27379, 27380, 27381, 27382, 27383, 27384, 27385, 27386, 27387, 27388, 27389, 27391, 27392, 27393, 27394, 27395, 27396, 27397, 27400, 27401, 27403, 27404, 27406, 27407, 27408, 27409, 27412, 27413, 27415, 27416, 27417, 27418, 27419, 27420, 27421, 27422, 27423, 27424, 27425, 27426, 27427, 27428, 27430, 27432, 27433, 27434, 27435, 27436, 27437, 27438, 27439, 27441, 27442, 27443, 27444, 27445, 27447, 27448, 27449, 27450, 27451, 27453, 27454, 27456, 27459, 27460, 27463, 27465, 27466, 27467, 27471, 27472, 27473, 27475, 27476, 27477, 27478, 27479, 27482, 27483, 27484, 27485, 27486, 27487, 27488, 27490, 27494, 27495, 27496, 27497, 27499, 27500, 27501, 27502, 27504, 27505, 27508, 27509, 27510, 27511, 27513, 27515, 27516, 27517, 27518, 27520, 27522, 27523, 27524, 27525, 27526, 27527, 27528, 27529, 27530, 27531, 27533, 27534, 27535, 27537, 27538, 27539, 27541, 27542, 27546, 27547, 27548, 27549, 27550, 27551, 27552, 27553, 27554, 27555, 27556, 27558, 27559, 27562, 27564, 27565, 27566, 27569, 27571, 27572, 27573, 27574, 27575, 27576, 27577, 27578, 27579, 27581, 27582, 27584, 27586, 27587, 27590, 27591, 27592, 27594, 27597, 27599, 27600, 27601, 27602, 27603, 27604, 27605, 27607, 27608, 27609, 27610, 27612, 27613, 27617, 27618, 27619, 27620, 27622, 27624, 27625, 27626, 27627, 27628, 27631, 27632, 27633, 27634, 27637, 27638, 27639, 27640, 27641, 27642, 27643, 27644, 27645, 27646, 27647, 27648, 27649, 27650, 27652, 27653, 27655, 27656, 27657, 27658, 27659, 27660, 27664, 27667, 27668, 27669, 27674, 27675, 27676, 27677, 27678, 27679, 27680, 27681, 27683, 27684, 27685, 27687, 27688, 27689, 27691, 27692, 27693, 27694, 27695, 27696, 27697, 27699, 27700, 27702, 27703, 27704, 27705, 27708, 27709, 27710, 27712, 27713, 27714, 27715, 27717, 27719, 27720, 27721, 27723, 27725, 27726, 27727, 27728, 27729, 27730, 27733, 27734, 27735, 27736, 27737, 27739, 27740, 27741, 27743, 27744, 27745, 27746, 27747, 27748, 27749, 27750, 27752, 27753, 27754, 27759, 27760, 27761, 27764, 27765, 27766, 27767, 27768, 27769, 27770, 27773, 27774, 27777, 27778, 27779, 27781, 27783, 27784, 27785, 27790, 27791, 27792, 27794, 27795, 27796, 27797, 27799, 27800, 27801, 27802, 27803, 27804, 27805, 27806, 27807, 27808, 27811, 27812, 27813, 27814, 27817, 27819, 27820, 27821, 27824, 27826, 27827, 27828, 27829, 27830, 27831, 27832, 27833, 27834, 27837, 27839, 27840, 27844, 27845, 27847, 27849, 27850, 27851, 27852, 27853, 27854, 27855, 27856, 27857, 27859, 27860, 27862, 27863, 27865, 27866, 27867, 27868, 27869, 27871, 27872, 27873, 27874, 27876, 27877, 27878, 27880, 27883, 27884, 27885, 27887, 27889, 27892, 27895, 27896, 27897, 27898, 27899, 27900, 27901, 27903, 27909, 27910, 27912, 27913, 27914, 27915, 27916, 27918, 27919, 27920, 27921, 27924, 27925, 27926, 27928, 27929, 27930, 27931, 27932, 27935, 27936, 27937, 27939, 27941, 27943, 27944, 27946, 27947, 27948, 27949, 27950, 27951, 27952, 27954, 27955, 27956, 27957, 27959, 27961, 27962, 27963, 27964, 27965, 27966, 27967, 27968, 27970, 27971, 27974, 27975, 27976, 27977, 27978, 27980, 27982, 27983, 27984, 27986, 27988, 27990, 27992, 27993, 27995, 28000, 28002, 28003, 28004, 28005, 28006, 28007, 28008, 28009, 28011, 28012, 28013, 28014, 28015, 28016, 28018, 28020, 28021, 28022, 28023, 28024, 28026, 28027, 28029, 28030, 28031, 28032, 28035, 28036, 28038, 28039, 28041, 28042, 28044, 28046, 28047, 28048, 28049, 28050, 28053, 28054, 28055, 28056, 28058, 28059, 28060, 28061, 28062, 28064, 28065, 28066, 28069, 28071, 28074, 28076, 28077, 28079, 28081, 28082, 28083, 28087, 28088, 28089, 28090, 28091, 28092, 28093, 28095, 28096, 28098, 28099, 28100, 28101, 28102, 28103, 28105, 28109, 28110, 28112, 28113, 28117, 28118, 28119, 28120, 28121, 28125, 28126, 28127, 28128, 28129, 28130, 28132, 28134, 28136, 28137, 28139, 28140, 28141, 28142, 28143, 28144, 28145, 28147, 28148, 28150, 28151, 28152, 28154, 28155, 28156, 28157, 28160, 28162, 28163, 28164, 28165, 28166, 28168, 28170, 28171, 28172, 28173, 28175, 28176, 28177, 28178, 28179, 28180, 28182, 28183, 28184, 28185, 28187, 28188, 28189, 28190, 28193, 28194, 28196, 28201, 28203, 28206, 28208, 28209, 28210, 28212, 28213, 28214, 28215, 28216, 28217, 28219, 28221, 28223, 28226, 28227, 28229, 28230, 28231, 28232, 28234, 28235, 28237, 28238, 28240, 28242, 28243, 28245, 28246, 28247, 28248, 28252, 28255, 28257, 28258, 28261, 28264, 28265, 28267, 28269, 28270, 28271, 28272, 28273, 28274, 28275, 28276, 28277, 28278, 28280, 28282, 28283, 28284, 28285, 28286, 28287, 28290, 28292, 28293, 28294, 28295, 28297, 28301, 28302, 28304, 28305, 28306, 28307, 28308, 28309, 28310, 28311, 28312, 28313, 28314, 28315, 28316, 28317, 28318, 28319, 28321, 28322, 28324, 28325, 28326, 28328, 28329, 28330, 28331, 28332, 28333, 28334, 28336, 28337, 28338, 28339, 28340, 28341, 28342, 28343, 28344, 28345, 28346, 28347, 28348, 28350, 28353, 28354, 28356, 28357, 28358, 28359, 28361, 28362, 28364, 28365, 28366, 28367, 28368, 28369, 28370, 28372, 28373, 28374, 28375, 28377, 28378, 28379, 28381, 28383, 28384, 28386, 28387, 28388, 28389, 28390, 28392, 28395, 28396, 28399, 28400, 28401, 28404, 28405, 28406, 28407, 28408, 28409, 28410, 28411, 28412, 28413, 28416, 28417, 28420, 28421, 28422, 28423, 28427, 28428, 28429, 28431, 28432, 28433, 28435, 28436, 28437, 28438, 28439, 28440, 28441, 28442, 28444, 28445, 28446, 28448, 28449, 28450, 28452, 28454, 28455, 28456, 28457, 28458, 28459, 28460, 28462, 28468, 28469, 28470, 28471, 28472, 28473, 28474, 28475, 28476, 28478, 28479, 28480, 28482, 28483, 28485, 28486, 28488, 28489, 28490, 28491, 28492, 28493, 28495, 28496, 28499, 28501, 28502, 28506, 28507, 28508, 28509, 28510, 28511, 28512, 28513, 28514, 28515, 28517, 28519, 28520, 28521, 28522, 28523, 28524, 28525, 28526, 28527, 28531, 28532, 28533, 28534, 28535, 28537, 28539, 28542, 28544, 28549, 28551, 28552, 28553, 28554, 28555, 28556, 28558, 28559, 28560, 28562, 28563, 28564, 28566, 28568, 28570, 28571, 28574, 28575, 28576, 28577, 28578, 28579, 28580, 28582, 28585, 28587, 28588, 28590, 28591, 28593, 28595, 28599, 28601, 28602, 28603, 28604, 28606, 28608, 28609, 28610, 28611, 28612, 28613, 28614, 28615, 28616, 28617, 28618, 28619, 28620, 28621, 28623, 28624, 28628, 28629, 28630, 28631, 28632, 28633, 28634, 28636, 28637, 28638, 28639, 28641, 28645, 28646, 28648, 28652, 28653, 28654, 28655, 28659, 28661, 28662, 28663, 28665, 28667, 28668, 28669, 28670, 28671, 28673, 28674, 28675, 28676, 28678, 28679, 28680, 28681, 28683, 28684, 28685, 28686, 28687, 28688, 28690, 28692, 28694, 28695, 28696, 28698, 28699, 28700, 28701, 28702, 28703, 28706, 28710, 28715, 28716, 28719, 28721, 28722, 28723, 28724, 28725, 28726, 28727, 28728, 28729, 28730, 28731, 28732, 28733, 28735, 28736, 28738, 28740, 28741, 28743, 28745, 28746, 28747, 28748, 28749, 28750, 28751, 28752, 28753, 28755, 28756, 28757, 28759, 28760, 28761, 28762, 28763, 28765, 28766, 28767, 28769, 28773, 28774, 28775, 28776, 28777, 28778, 28779, 28781, 28782, 28784, 28785, 28787, 28788, 28789, 28790, 28791, 28793, 28794, 28795, 28797, 28799, 28800, 28801, 28802, 28803, 28805, 28807, 28809, 28811, 28812, 28813, 28814, 28815, 28817, 28818, 28819, 28822, 28824, 28825, 28828, 28829, 28830, 28831, 28834, 28835, 28837, 28838, 28839, 28841, 28842, 28843, 28845, 28847, 28848, 28849, 28850, 28851, 28852, 28855, 28856, 28858, 28860, 28861, 28862, 28863, 28865, 28866, 28867, 28868, 28869, 28870, 28871, 28872, 28873, 28876, 28877, 28879, 28880, 28881, 28882, 28884, 28886, 28887, 28888, 28889, 28890, 28891, 28892, 28893, 28895, 28896, 28900, 28901, 28902, 28903, 28905, 28907, 28908, 28912, 28913, 28916, 28917, 28919, 28920, 28922, 28923, 28925, 28926, 28927, 28929, 28930, 28932, 28933, 28934, 28936, 28937, 28939, 28943, 28944, 28945, 28946, 28947, 28948, 28950, 28954, 28955, 28956, 28957, 28958, 28959, 28960, 28961, 28964, 28966, 28967, 28968, 28969, 28970, 28971, 28972, 28974, 28976, 28977, 28980, 28981, 28982, 28983, 28984, 28985, 28986, 28987, 28989, 28991, 28992, 28993, 28994, 28998, 28999, 29000, 29001, 29002, 29003, 29005, 29007, 29008, 29009, 29010, 29011, 29012, 29013, 29015, 29016, 29019, 29020, 29021, 29024, 29025, 29026, 29027, 29029, 29030, 29031, 29032, 29033, 29035, 29036, 29038, 29039, 29040, 29043, 29044, 29045, 29047, 29048, 29049, 29051, 29052, 29053, 29054, 29056, 29057, 29058, 29059, 29060, 29061, 29063, 29069, 29072, 29073, 29074, 29077, 29080, 29081, 29082, 29083, 29086, 29088, 29089, 29090, 29092, 29093, 29099, 29100, 29101, 29104, 29105, 29107, 29108, 29109, 29110, 29112, 29113, 29115, 29116, 29118, 29119, 29121, 29122, 29124, 29125, 29126, 29127, 29128, 29129, 29130, 29132, 29135, 29136, 29138, 29140, 29141, 29142, 29143, 29144, 29147, 29148, 29149, 29150, 29152, 29153, 29155, 29156, 29157, 29158, 29160, 29161, 29162, 29163, 29164, 29165, 29166, 29167, 29168, 29169, 29171, 29173, 29174, 29175, 29176, 29177, 29179, 29182, 29184, 29186, 29187, 29188, 29191, 29193, 29194, 29196, 29197, 29199, 29200, 29201, 29203, 29204, 29205, 29206, 29207, 29208, 29209, 29211, 29212, 29213, 29214, 29215, 29216, 29218, 29219, 29220, 29221, 29222, 29224, 29227, 29228, 29229, 29230, 29232, 29234, 29235, 29236, 29237, 29238, 29239, 29240, 29242, 29244, 29245, 29246, 29247, 29248, 29249, 29250, 29251, 29252, 29254, 29255, 29258, 29260, 29261, 29264, 29266, 29268, 29271, 29272, 29275, 29276, 29278, 29279, 29282, 29283, 29284, 29286, 29287, 29288, 29289, 29290, 29291, 29292, 29293, 29294, 29295, 29296, 29297, 29298, 29299, 29300, 29301, 29302, 29303, 29306, 29307, 29308, 29309, 29311, 29313, 29315, 29316, 29317, 29319, 29322, 29325, 29326, 29327, 29329, 29330, 29331, 29332, 29333, 29335, 29336, 29337, 29339, 29341, 29342, 29343, 29344, 29347, 29348, 29349, 29351, 29352, 29353, 29354, 29356, 29357, 29358, 29359, 29361, 29362, 29364, 29366, 29367, 29368, 29369, 29371, 29375, 29376, 29378, 29379, 29380, 29381, 29382, 29383, 29384, 29385, 29387, 29388, 29389, 29392, 29395, 29396, 29397, 29398, 29399, 29400, 29401, 29402, 29403, 29404, 29405, 29406, 29407, 29408, 29409, 29413, 29416, 29417, 29418, 29419, 29420, 29422, 29424, 29425, 29427, 29429, 29430, 29431, 29432, 29433, 29434, 29435, 29436, 29437, 29440, 29441, 29442, 29443, 29444, 29446, 29447, 29448, 29449, 29450, 29451, 29452, 29453, 29455, 29456, 29457, 29458, 29459, 29461, 29462, 29464, 29465, 29468, 29471, 29472, 29473, 29474, 29476, 29477, 29478, 29480, 29481, 29482, 29484, 29485, 29486, 29487, 29489, 29490, 29491, 29493, 29494, 29496, 29497, 29498, 29499, 29500, 29503, 29505, 29506, 29507, 29508, 29509, 29510, 29512, 29513, 29514, 29517, 29519, 29521, 29523, 29525, 29526, 29527, 29528, 29529, 29530, 29531, 29533, 29534, 29535, 29536, 29537, 29538, 29539, 29540, 29543, 29544, 29545, 29547, 29549, 29551, 29552, 29553, 29554, 29555, 29557, 29558, 29559, 29560, 29561, 29562, 29563, 29564, 29566, 29567, 29568, 29570, 29572, 29573, 29574, 29575, 29577, 29578, 29579, 29580, 29581, 29582, 29584, 29585, 29586, 29588, 29591, 29592, 29594, 29595, 29596, 29597, 29599, 29600, 29602, 29604, 29605, 29606, 29607, 29608, 29610, 29611, 29612, 29613, 29614, 29615, 29619, 29620, 29621, 29622, 29624, 29627, 29628, 29629, 29630, 29631, 29632, 29633, 29634, 29635, 29637, 29639, 29640, 29642, 29643, 29644, 29645, 29646, 29648, 29649, 29650, 29652, 29654, 29655, 29656, 29657, 29658, 29659, 29660, 29662, 29664, 29665, 29667, 29669, 29670, 29671, 29672, 29673, 29674, 29675, 29676, 29677, 29678, 29679, 29681, 29682, 29683, 29686, 29687, 29688, 29689, 29690, 29691, 29692, 29693, 29694, 29695, 29696, 29697, 29699, 29700, 29701, 29702, 29704, 29706, 29707, 29708, 29710, 29712, 29713, 29714, 29716, 29717, 29718, 29719, 29720, 29721, 29723, 29724, 29725, 29726, 29727, 29728, 29730, 29731, 29733, 29735, 29736, 29739, 29740, 29741, 29743, 29744, 29745, 29746, 29748, 29750, 29751, 29752, 29753, 29755, 29756, 29757, 29758, 29760, 29761, 29763, 29764, 29766, 29767, 29769, 29770, 29773, 29774, 29775, 29776, 29778, 29779, 29780, 29781, 29782, 29783, 29784, 29785, 29787, 29788, 29789, 29790, 29793, 29794, 29795, 29797, 29798, 29800, 29802, 29804, 29806, 29807, 29808, 29809, 29811, 29812, 29813, 29814, 29815, 29816, 29817, 29820, 29821, 29822, 29823, 29824, 29829, 29830, 29831, 29832, 29833, 29834, 29835, 29837, 29838, 29840, 29841, 29842, 29844, 29845, 29846, 29847, 29848, 29850, 29851, 29852, 29853, 29854, 29855, 29856, 29857, 29858, 29860, 29861, 29862, 29863, 29864, 29865, 29866, 29867, 29868, 29869, 29870, 29871, 29874, 29875, 29876, 29878, 29879, 29880, 29882, 29884, 29885, 29886, 29887, 29888, 29889, 29890, 29892, 29894, 29895, 29896, 29897, 29898, 29901, 29902, 29904, 29905, 29909, 29910, 29911, 29912, 29915, 29916, 29917, 29918, 29919, 29920, 29922, 29926, 29927, 29929, 29931, 29932, 29933, 29934, 29935, 29936, 29937, 29939, 29940, 29941, 29943, 29944, 29946, 29948, 29950, 29952, 29956, 29957, 29958, 29960, 29962, 29964, 29965, 29966, 29967, 29968, 29969, 29970, 29973, 29975, 29978, 29979, 29980, 29982, 29983, 29985, 29987, 29988, 29989, 29990, 29991, 29994, 29995, 29996, 29997, 29998, 30000, 30001, 30002, 30003, 30004, 30005, 30007, 30008, 30009, 30010, 30011, 30012, 30014, 30015, 30016, 30017, 30018, 30019, 30020, 30021, 30022, 30023, 30024, 30025, 30026, 30027, 30028, 30029, 30030, 30031, 30032, 30033, 30036, 30038, 30039, 30040, 30042, 30043, 30045, 30046, 30047, 30049, 30050, 30052, 30053, 30054, 30059, 30060, 30062, 30065, 30066, 30068, 30069, 30071, 30073, 30075, 30076, 30077, 30080, 30082, 30083, 30084, 30085, 30086, 30087, 30088, 30089, 30091, 30092, 30093, 30094, 30095, 30096, 30097, 30099, 30100, 30103, 30104, 30106, 30108, 30111, 30112, 30113, 30114, 30115, 30117, 30119, 30122, 30123, 30124, 30125, 30127, 30128, 30129, 30130, 30132, 30133, 30134, 30135, 30136, 30137, 30139, 30140, 30141, 30142, 30143, 30144, 30147, 30150, 30151, 30152, 30154, 30155, 30157, 30158, 30159, 30162, 30165, 30166, 30167, 30168, 30169, 30170, 30171, 30172, 30173, 30174, 30175, 30176, 30178, 30179, 30180, 30181, 30182, 30183, 30184, 30185, 30186, 30187, 30188, 30189, 30190, 30194, 30197, 30198, 30200, 30202, 30203, 30206, 30207, 30208, 30209, 30210, 30211, 30212, 30213, 30214, 30215, 30216, 30218, 30219, 30220, 30222, 30223, 30224, 30225, 30226, 30227, 30228, 30229, 30230, 30231, 30232, 30233, 30234, 30235, 30236, 30237, 30239, 30240, 30243, 30245, 30246, 30247, 30248, 30249, 30250, 30251, 30253, 30255, 30256, 30259, 30260, 30262, 30263, 30265, 30270, 30272, 30273, 30275, 30276, 30279, 30281, 30283, 30284, 30287, 30288, 30289, 30291, 30292, 30293, 30294, 30295, 30296, 30298, 30302, 30303, 30305, 30306, 30307, 30308, 30310, 30311, 30312, 30316, 30317, 30318, 30319, 30320, 30321, 30324, 30325, 30326, 30327, 30328, 30329, 30330, 30332, 30333, 30336, 30338, 30340, 30342, 30343, 30344, 30347, 30348, 30349, 30350, 30351, 30353, 30355, 30356, 30357, 30358, 30359, 30360, 30361, 30363, 30364, 30365, 30366, 30367, 30370, 30371, 30372, 30373, 30374, 30378, 30379, 30380, 30381, 30382, 30383, 30384, 30385, 30387, 30388, 30389, 30391, 30392, 30395, 30396, 30397, 30398, 30399, 30400, 30401, 30402, 30404, 30406, 30407, 30408, 30409, 30410, 30413, 30414, 30416, 30417, 30419, 30421, 30424, 30425, 30426, 30427, 30428, 30430, 30431, 30432, 30433, 30434, 30435, 30436, 30437, 30438, 30439, 30441, 30442, 30445, 30447, 30448, 30449, 30450, 30451, 30452, 30454, 30456, 30457, 30458, 30460, 30463, 30465, 30467, 30468, 30469, 30470, 30471, 30472, 30474, 30475, 30477, 30478, 30479, 30481, 30482, 30483, 30484, 30486, 30487, 30488, 30489, 30490, 30491, 30492, 30493, 30494, 30495, 30496, 30497, 30498, 30499, 30500, 30501, 30502, 30504, 30505, 30508, 30510, 30512, 30513, 30514, 30515, 30516, 30518, 30519, 30521, 30522, 30523, 30524, 30525, 30526, 30527, 30528, 30529, 30530, 30531, 30532, 30533, 30534, 30535, 30536, 30538, 30540, 30541, 30543, 30545, 30546, 30547, 30549, 30551, 30552, 30553, 30554, 30555, 30556, 30558, 30559, 30561, 30564, 30565, 30566, 30567, 30568, 30569, 30570, 30572, 30573, 30575, 30578, 30579, 30580, 30581, 30584, 30587, 30588, 30591, 30592, 30595, 30598, 30599, 30601, 30602, 30603, 30605, 30606, 30607, 30608, 30610, 30613, 30615, 30616, 30617, 30618, 30619, 30620, 30621, 30622, 30623, 30626, 30627, 30629, 30630, 30631, 30632, 30633, 30634, 30635, 30638, 30639, 30640, 30641, 30642, 30644, 30645, 30646, 30648, 30649, 30650, 30651, 30652, 30654, 30656, 30657, 30659, 30660, 30662, 30663, 30665, 30666, 30667, 30668, 30669, 30670, 30671, 30672, 30673, 30674, 30675, 30676, 30680, 30681, 30682, 30683, 30685, 30686, 30687, 30688, 30689, 30691, 30692, 30693, 30694, 30696, 30699, 30700, 30701, 30703, 30704, 30705, 30706, 30708, 30710, 30711, 30712, 30713, 30714, 30715, 30716, 30717, 30718, 30720, 30721, 30723, 30724, 30725, 30726, 30727, 30728, 30730, 30731, 30733, 30734, 30735, 30737, 30738, 30739, 30740, 30741, 30742, 30744, 30745, 30747, 30748, 30749, 30753, 30755, 30756, 30757, 30758, 30759, 30762, 30763, 30764, 30765, 30766, 30769, 30770, 30772, 30773, 30774, 30775, 30776, 30777, 30778, 30779, 30780, 30781, 30782, 30783, 30785, 30786, 30788, 30791, 30792, 30795, 30796, 30797, 30798, 30799, 30800, 30801, 30806, 30807, 30808, 30809, 30811, 30813, 30815, 30817, 30819, 30821, 30824, 30827, 30828, 30829, 30831, 30833, 30834, 30836, 30838, 30839, 30840, 30841, 30842, 30844, 30845, 30846, 30847, 30848, 30849, 30850, 30851, 30852, 30853, 30854, 30855, 30857, 30858, 30859, 30860, 30861, 30862, 30863, 30864, 30865, 30867, 30869, 30870, 30871, 30872, 30873, 30875, 30876, 30877, 30878, 30879, 30881, 30882, 30887, 30888, 30889, 30891, 30893, 30894, 30895, 30896, 30897, 30899, 30901, 30902, 30903, 30905, 30907, 30908, 30910, 30911, 30913, 30914, 30915, 30916, 30920, 30922, 30923, 30924, 30925, 30926, 30927, 30928, 30929, 30930, 30931, 30933, 30935, 30936, 30937, 30938, 30939, 30941, 30944, 30945, 30946, 30948, 30949, 30950, 30951, 30954, 30955, 30956, 30958, 30959, 30960, 30961, 30962, 30963, 30965, 30966, 30969, 30970, 30971, 30972, 30973, 30974, 30975, 30976, 30977, 30978, 30979, 30981, 30982, 30983, 30984, 30987, 30988, 30989, 30990, 30993, 30994, 30996, 30998, 31000, 31001, 31002, 31003, 31004, 31005, 31006, 31007, 31010, 31011, 31012, 31014, 31015, 31016, 31017, 31018, 31019, 31021, 31022, 31024, 31025, 31026, 31027, 31028, 31029, 31030, 31031, 31032, 31033, 31034, 31036, 31037, 31038, 31039, 31040, 31041, 31042, 31043, 31044, 31045, 31046, 31048, 31049, 31050, 31051, 31052, 31053, 31057, 31058, 31059, 31061, 31062, 31063, 31064, 31067, 31068, 31069, 31070, 31071, 31072, 31073, 31074, 31075, 31076, 31077, 31078, 31079, 31080, 31081, 31082, 31085, 31086, 31087, 31088, 31089, 31090, 31092, 31093, 31094, 31095, 31096, 31098, 31100, 31101, 31102, 31103, 31104, 31105, 31106, 31107, 31108, 31110, 31113, 31114, 31115, 31116, 31117, 31118, 31119, 31120, 31121, 31122, 31123, 31124, 31125, 31127, 31128, 31130, 31131, 31132, 31133, 31134, 31135, 31137, 31138, 31139, 31140, 31141, 31142, 31143, 31144, 31147, 31148, 31150, 31151, 31152, 31153, 31154, 31156, 31158, 31159, 31162, 31166, 31167, 31168, 31169, 31171, 31173, 31174, 31176, 31177, 31178, 31179, 31180, 31184, 31187, 31190, 31193, 31194, 31196, 31197, 31199, 31200, 31203, 31204, 31206, 31207, 31208, 31209, 31210, 31211, 31212, 31214, 31217, 31221, 31222, 31223, 31224, 31225, 31227, 31228, 31230, 31231, 31232, 31233, 31234, 31235, 31236, 31239, 31242, 31243, 31245, 31246, 31247, 31248, 31250, 31251, 31254, 31255, 31257, 31258, 31259, 31260, 31261, 31263, 31265, 31267, 31270, 31271, 31272, 31273, 31274, 31275, 31276, 31277, 31280, 31281, 31284, 31285, 31286, 31287, 31288, 31289, 31290, 31292, 31293, 31294, 31295, 31298, 31299, 31300, 31301, 31302, 31303, 31304, 31305, 31306, 31307, 31308, 31309, 31310, 31311, 31317, 31319, 31320, 31322, 31324, 31325, 31326, 31328, 31329, 31330, 31332, 31333, 31334, 31337, 31338, 31339, 31340, 31341, 31342, 31343, 31344, 31345, 31346, 31348, 31349, 31351, 31352, 31353, 31357, 31358, 31359, 31360, 31361, 31363, 31364, 31365, 31366, 31367, 31369, 31370, 31371, 31373, 31374, 31375, 31376, 31378, 31379, 31380, 31381, 31384, 31385, 31387, 31388, 31390, 31391, 31392, 31393, 31394, 31395, 31396, 31398, 31399, 31400, 31401, 31402, 31403, 31404, 31406, 31407, 31409, 31410, 31411, 31412, 31413, 31415, 31417, 31418, 31421, 31422, 31424, 31426, 31429, 31431, 31432, 31433, 31436, 31438, 31439, 31440, 31441, 31442, 31443, 31444, 31446, 31447, 31450, 31451, 31452, 31453, 31455, 31456, 31457, 31458, 31459, 31460, 31463, 31464, 31466, 31467, 31468, 31469, 31471, 31472, 31473, 31476, 31477, 31480, 31481, 31482, 31483, 31484, 31485, 31487, 31488, 31489, 31490, 31491, 31492, 31494, 31495, 31496, 31497, 31498, 31500, 31501, 31504, 31505, 31506, 31508, 31509, 31510, 31511, 31512, 31513, 31514, 31515, 31516, 31517, 31518, 31519, 31521, 31522, 31523, 31526, 31528, 31529, 31530, 31532, 31534, 31535, 31536, 31537, 31539, 31540, 31541, 31542, 31543, 31544, 31546, 31547, 31549, 31550, 31551, 31553, 31554, 31556, 31557, 31559, 31562, 31563, 31565, 31566, 31567, 31569, 31571, 31572, 31573, 31574, 31576, 31577, 31578, 31581, 31584, 31585, 31587, 31589, 31590, 31591, 31593, 31595, 31597, 31598, 31599, 31601, 31602, 31604, 31605, 31606, 31607, 31608, 31610, 31611, 31613, 31615, 31616, 31617, 31618, 31620, 31621, 31622, 31623, 31624, 31625, 31626, 31628, 31630, 31631, 31633, 31634, 31637, 31638, 31639, 31642, 31644, 31647, 31649, 31650, 31652, 31653, 31654, 31655, 31656, 31658, 31661, 31662, 31664, 31665, 31666, 31667, 31668, 31671, 31672, 31674, 31676, 31677, 31678, 31679, 31680, 31681, 31682, 31683, 31684, 31686, 31687, 31688, 31690, 31691, 31692, 31694, 31695, 31696, 31697, 31698, 31699, 31701, 31705, 31706, 31707, 31709, 31710, 31711, 31712, 31713, 31714, 31716, 31718, 31719, 31720, 31721, 31724, 31725, 31726, 31727, 31728, 31730, 31732, 31733, 31734, 31735, 31737, 31738, 31739, 31740, 31742, 31744, 31745, 31746, 31748, 31750, 31751, 31752, 31753, 31754, 31755, 31757, 31758, 31759, 31761, 31762, 31763, 31764, 31765, 31766, 31767, 31770, 31771, 31772, 31773, 31774, 31775, 31777, 31778, 31779, 31780, 31781, 31782, 31784, 31787, 31789, 31790, 31791, 31793, 31794, 31795, 31796, 31797, 31798, 31799, 31800, 31804, 31805, 31806, 31807, 31808, 31809, 31810, 31813, 31817, 31818, 31820, 31822, 31823, 31824, 31826, 31827, 31828, 31830, 31834, 31835, 31837, 31839, 31840, 31841, 31843, 31845, 31846, 31848, 31849, 31850, 31853, 31854, 31856, 31857, 31858, 31859, 31860, 31862, 31863, 31864, 31865, 31866, 31868, 31870, 31871, 31873, 31874, 31876, 31877, 31879, 31880, 31882, 31884, 31885, 31886, 31887, 31889, 31890, 31891, 31892, 31893, 31894, 31895, 31899, 31900, 31901, 31902, 31903, 31904, 31905, 31908, 31909, 31910, 31911, 31912, 31914, 31915, 31916, 31917, 31918, 31920, 31921, 31923, 31925, 31926, 31927, 31928, 31929, 31931, 31932, 31933, 31934, 31935, 31936, 31937, 31938, 31939, 31940, 31941, 31944, 31945, 31947, 31949, 31954, 31957, 31958, 31961, 31963, 31965, 31967, 31968, 31970, 31973, 31974, 31975, 31976, 31977, 31980, 31981, 31982, 31984, 31985, 31986, 31987, 31988, 31989, 31990, 31992, 31993, 31994, 31997, 31998, 31999, 32001, 32006, 32008, 32009, 32010, 32012, 32015, 32016, 32018, 32020, 32021, 32022, 32023, 32025, 32027, 32029, 32030, 32032, 32033, 32034, 32035, 32037, 32042, 32044, 32046, 32047, 32048, 32049, 32052, 32053, 32054, 32055, 32056, 32058, 32059, 32060, 32061, 32063, 32064, 32066, 32067, 32068, 32069, 32071, 32073, 32074, 32075, 32076, 32079, 32081, 32082, 32083, 32084, 32085, 32086, 32087, 32088, 32089, 32092, 32093, 32095, 32097, 32098, 32100, 32101, 32103, 32105, 32107, 32108, 32109, 32110, 32115, 32116, 32117, 32118, 32119, 32122, 32123, 32124, 32125, 32127, 32128, 32129, 32130, 32131, 32133, 32134, 32138, 32142, 32144, 32145, 32148, 32149, 32150, 32151, 32152, 32153, 32155, 32156, 32158, 32159, 32160, 32161, 32162, 32163, 32164, 32166, 32168, 32169, 32170, 32171, 32175, 32176, 32177, 32178, 32180, 32181, 32183, 32184, 32188, 32190, 32191, 32192, 32193, 32194, 32196, 32198, 32199, 32200, 32201, 32202, 32203, 32204, 32205, 32207, 32209, 32211, 32212, 32214, 32215, 32216, 32217, 32218, 32219, 32220, 32221, 32222, 32225, 32226, 32229, 32235, 32236, 32237, 32239, 32240, 32241, 32243, 32244, 32245, 32246, 32247, 32248, 32251, 32252, 32253, 32254, 32255, 32256, 32257, 32260, 32263, 32264, 32265, 32266, 32267, 32269, 32270, 32271, 32272, 32273, 32275, 32277, 32278, 32279, 32280, 32281, 32282, 32283, 32284, 32286, 32287, 32288, 32289, 32290, 32292, 32294, 32295, 32296, 32298, 32299, 32301, 32302, 32304, 32305, 32306, 32308, 32310, 32311, 32312, 32313, 32314, 32315, 32316, 32317, 32318, 32320, 32322, 32323, 32324, 32325, 32326, 32327, 32328, 32331, 32333, 32334, 32335, 32336, 32337, 32338, 32340, 32342, 32344, 32345, 32347, 32348, 32349, 32350, 32351, 32352, 32353, 32355, 32357, 32358, 32359, 32361, 32362, 32364, 32365, 32368, 32369, 32370, 32371, 32372, 32374, 32375, 32376, 32377, 32378, 32379, 32380, 32383, 32384, 32386, 32388, 32391, 32392, 32394, 32397, 32398, 32399, 32402, 32403, 32404, 32408, 32409, 32410, 32411, 32412, 32413, 32414, 32417, 32419, 32420, 32421, 32422, 32424, 32425, 32426, 32427, 32428, 32431, 32433, 32434, 32436, 32439, 32441, 32442, 32444, 32446, 32447, 32449, 32451, 32452, 32453, 32455, 32456, 32458, 32461, 32463, 32464, 32465, 32466, 32467, 32469]
train_changed = train_changed.iloc[idx]
train_changed

,id,sentence,subject_entity,object_entity,label,source
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...","{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '박정부', 'start_idx': 22, 'end_idx': 24...","{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...",org:top_members/employees,wikitree
5,5,": 유엔, 유럽 의회, 북대서양 조약 기구 (NATO), 국제이주기구, 세계 보건 ...","{'word': 'NATO', 'start_idx': 25, 'end_idx': 2...","{'word': '북대서양 조약 기구', 'start_idx': 13, 'end_i...",org:alternate_names,wikipedia
7,7,"박용오(朴容旿, 1937년 4월 29일(음력 3월 19일)(음력 3월 19일) ~ ...","{'word': '1937년 4월 29일', 'start_idx': 9, 'end_...","{'word': '박용오', 'start_idx': 0, 'end_idx': 2, ...",per:date_of_birth,wikipedia
8,8,중공군에게 온전히 대항할 수 없을 정도로 약해진 국민당은 타이베이로 수도를 옮기는 ...,"{'word': '타이베이', 'start_idx': 32, 'end_idx': 3...","{'word': '중화민국', 'start_idx': 59, 'end_idx': 6...",org:place_of_headquarters,wikipedia
...,...,...,...,...,...,...
32464,32464,KIA타이거즈 외야수 이창진이 롯데백화점 광주점이 시상하는 9월 월간 MVP에 선정...,"{'word': '외야수', 'start_idx': 8, 'end_idx': 10,...","{'word': '이창진', 'start_idx': 12, 'end_idx': 14...",per:title,wikitree
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...","{'word': '유기준', 'start_idx': 93, 'end_idx': 95...",per:employee_of,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...",per:colleagues,wikipedia
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...","{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...",org:top_members/employees,wikitree


In [8]:
#i = 2

for i in idx:
    # 1. 원본 라벨 확인
    origin_label = train_changed['label'][i]
    #print(origin_label)

    # 2. type 추출
    idx = train_changed['label'][i].find(':')
    type1 = train_changed['label'][i][:idx]
    type2 = train_changed['label'][i][idx+1:]
    #print(type1)
    #print(type2)
    
    # 3. type 교체
    new_label = type2 + ':' + type1
    train_changed['label'][i] = new_label
    #print(new_label)
    
train_changed

C:\Users\2jeon\AppData\Local\Temp\ipykernel_27444\1067947575.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_changed['label'][i] = new_label


,id,sentence,subject_entity,object_entity,label,source
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...","{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...",member_of:org,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '박정부', 'start_idx': 22, 'end_idx': 24...","{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...",top_members/employees:org,wikitree
5,5,": 유엔, 유럽 의회, 북대서양 조약 기구 (NATO), 국제이주기구, 세계 보건 ...","{'word': 'NATO', 'start_idx': 25, 'end_idx': 2...","{'word': '북대서양 조약 기구', 'start_idx': 13, 'end_i...",alternate_names:org,wikipedia
7,7,"박용오(朴容旿, 1937년 4월 29일(음력 3월 19일)(음력 3월 19일) ~ ...","{'word': '1937년 4월 29일', 'start_idx': 9, 'end_...","{'word': '박용오', 'start_idx': 0, 'end_idx': 2, ...",date_of_birth:per,wikipedia
8,8,중공군에게 온전히 대항할 수 없을 정도로 약해진 국민당은 타이베이로 수도를 옮기는 ...,"{'word': '타이베이', 'start_idx': 32, 'end_idx': 3...","{'word': '중화민국', 'start_idx': 59, 'end_idx': 6...",place_of_headquarters:org,wikipedia
...,...,...,...,...,...,...
32464,32464,KIA타이거즈 외야수 이창진이 롯데백화점 광주점이 시상하는 9월 월간 MVP에 선정...,"{'word': '외야수', 'start_idx': 8, 'end_idx': 10,...","{'word': '이창진', 'start_idx': 12, 'end_idx': 14...",title:per,wikitree
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...","{'word': '유기준', 'start_idx': 93, 'end_idx': 95...",employee_of:per,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...",colleagues:per,wikipedia
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...","{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...",top_members/employees:org,wikitree


In [9]:
result = pd.concat([train, train_changed])
result

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
32464,32464,KIA타이거즈 외야수 이창진이 롯데백화점 광주점이 시상하는 9월 월간 MVP에 선정...,"{'word': '외야수', 'start_idx': 8, 'end_idx': 10,...","{'word': '이창진', 'start_idx': 12, 'end_idx': 14...",title:per,wikitree
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...","{'word': '유기준', 'start_idx': 93, 'end_idx': 95...",employee_of:per,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...",colleagues:per,wikipedia
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...","{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...",top_members/employees:org,wikitree


In [10]:
result.to_csv('./train_aug_ver1-1.csv', index=False)